## Funcion de covarianza generalizada

In [1]:
import numpy as np              
from time import time
from os import listdir
import cv2
import os 
import scipy.linalg as sg
from PIL import Image
from math import sqrt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from PIL import Image
from math import sqrt

%matplotlib inline

In [4]:
## Funcion de flujo optico

In [2]:
def flujo_optico(imgBGR ):
    
    tem = cv2.cvtColor(imgBGR,cv2.COLOR_BGR2YCrCb)
    f,c,ch = tem.shape
    imgYcrcb = np.zeros((f,c,ch))
    direction_matrix = np.zeros((f,c))
   
    for row in range(0,tem.shape[0]):         #y
        for col in range(0,tem.shape[1]):     #x
            
            imgYcrcb[row,col,1] = tem[row,col,1] + (-128.0)                 #Cr
            imgYcrcb[row,col,2] = tem[row,col,2] + (-128.0)                 #Cb
            imgYcrcb[row,col,0] = tem[row,col,0]                          #Y
                                  
            if imgYcrcb[row,col,1] > 0 and imgYcrcb[row,col,2] > 0:
                angle = (np.arctan(imgYcrcb[row][col][1]/imgYcrcb[row][col][2])* (180.0/np.pi)) + 270
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] > 0 and imgYcrcb[row,col,2] < 0:
                angle = np.arctan(imgYcrcb[row][col][2]/imgYcrcb[row][col][1])* (180.0/np.pi)
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] < 0 and imgYcrcb[row,col,2] < 0:
                angle = (np.arctan(imgYcrcb[row][col][1]/imgYcrcb[row][col][2])* (180.0/np.pi))+ 90
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] < 0 and imgYcrcb[row,col,2] > 0:
                angle = (np.arctan(imgYcrcb[row][col][2]/imgYcrcb[row][col][1])* (180.0/np.pi)) + 180
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] > 0 and imgYcrcb[row,col,2] == 0:
                angle = 0.0 
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] < 0 and imgYcrcb[row,col,2] == 0:
                angle = 180.0 
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] == 0 and imgYcrcb[row,col,2] > 0:
                angle = 270.0
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] == 0 and imgYcrcb[row,col,2] < 0:
                angle = 90.0 
                direction_matrix[row][col] = angle

            elif imgYcrcb[row,col,1] == 0 and imgYcrcb[row,col,2] == 0:
                angle = 0.0 
                direction_matrix[row][col] = angle   
    
    mag,d=cv2.cartToPolar(imgYcrcb[:,:,2],imgYcrcb[:,:,1])    #cartTopolar(x,y)
    return imgYcrcb,mag,abs(direction_matrix) 


## Funcion para pintar

In [3]:
def pintar(cov):
    new_cov=np.copy(cov)
    for i in range(cov.shape[1]):
        for j in range(cov.shape[0]):
            if(new_cov[i,j]<=0):
                new_cov[i,j]=-1*(new_cov[i,j]/np.min(cov))
            else:
                new_cov[i,j]=new_cov[i,j]/np.max(cov)
    return new_cov

## Funcion para pintar normalizada

In [4]:
def normalized_matrices(covs):
    d=covs[0][0].shape[0]
    min_matrix=np.zeros((d,d))
    max_matrix=np.zeros((d,d))
    for i in range(len(covs)):
        #for de frames
        for j in range(len(covs[i])):
            #for de trayectorias en los frames
            new_cov=covs[i][j]
            for k in range(covs[i][j].shape[0]):
                for l in range(covs[i][j].shape[1]):
                    #for's para las dimensiones de las covarianzas(# de primitivas)
                    if(new_cov[k,l]<min_matrix[k,l]):
                        min_matrix[k,l]=new_cov[k,l]
                    if(new_cov[k,l]>max_matrix[k,l]):
                        max_matrix[k,l]=new_cov[k,l]
    return max_matrix,min_matrix

In [5]:
def normalized_draw(cov,max_matrix,min_matrix):
    new_cov=np.copy(cov)
    for i in range(cov.shape[1]):
        for j in range(cov.shape[0]):
            if(new_cov[i,j]<=0):
                new_cov[i,j]=-1*(new_cov[i,j]/min_matrix[i,j])
            else:
                new_cov[i,j]=new_cov[i,j]/max_matrix[i,j]
    return new_cov

## Funcion para calcular mis primitivas integrales

In [6]:
def int_great_array(img,imgf):
# nuestrass 4 caracteristicas de la imagen, puede tenerse en cuenta el color

    gris=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sobelx=cv2.Sobel(gris,cv2.CV_64F,1,0,ksize=3)
    sobely=cv2.Sobel(gris,cv2.CV_64F,0,1,ksize=3)
    sobelyx=((cv2.Sobel(sobelx,cv2.CV_64F,1,0,ksize=3)+cv2.Sobel(sobely,cv2.CV_64F,0,1,ksize=3))/2)
    laplacian=cv2.Laplacian(gris,cv2.CV_64F,ksize=3)
    imgyrcv,mag,direction=flujo_optico(imgf)
    cr=imgyrcv[:,:,1]
    cb=imgyrcv[:,:,2]
    
    ch_array=np.array([sobelx,sobely,sobelyx,laplacian,cr,cb,mag])
    #Matriz de d*w*h
    d=ch_array.shape[0]
    W=ch_array.shape[1]
    H=ch_array.shape[2]
    #Tensor P
    f_mat=np.zeros((d,W+1,H+1))
    for i in range(d):
        img_int=cv2.integral(ch_array[i],sdepth=cv2.CV_64F)
        f_mat[i]=img_int 
    #Tensor Q    
    nmatrices=((d * (d + 1)) / 2 )
    matrix_product=np.zeros((int(nmatrices),W+1,H+1))              
    pos=0
    for i in range(d):
        for j in range(i,d):
            mult=ch_array[i]*ch_array[j]                      #producto elemento a elemento entre tipos arrays
            img_int=cv2.integral(mult,sdepth=cv2.CV_64F)
            matrix_product[pos] = img_int
            pos=pos+1
        

    return ch_array,f_mat,matrix_product

## Funcion Cortar utilizada en video_covs

In [7]:
def cortar(x):
    #x = trayectorias en texto
    tr=[]
    gtr=[]
    count = 0
    n = 0
    for i in range(0,len(x)):
        if(x[i]==' '):
            count=count+1
            tr.append(float(x[n:i]))
            n=i
        if(count==34):
            gtr.append(tr)
            count=0
            tr=[]
    return gtr

## Funcon para crear la matriz de covarianza

In [8]:
def matriz_covarianza(P,Q,y1,x1,y2,x2):
    #El tensor P
    #El tensor Q
    #Las coordenadas de los puntos para las regiones de interes
    d=P.shape[0]
    col=x2-x1
    fil=y2-y1
    m_cov=np.zeros((d,d))
    n1=(1/((fil*col)-1))
    n2=(1/(fil*col))
    pos=0
    for i in range(d):
        for j in range(i,d):
            q=Q[pos][y1,x1]+Q[pos][y1+fil,x1+col]-Q[pos][y1,x1+col]-Q[pos][y1+fil,x1]
            p_1=(P[i][y1,x1]+P[i][y1+fil,x1+col]-P[i][y1,x1+col]-P[i][fil+y1,x1])/(fil*col)
            p_2=(P[j][y1,x1]+P[j][y1+fil,x1+col]-P[j][y1,x1+col]-P[j][fil+y1,x1])/(fil*col)
            m_cov[i,j]=(n1)*(q-(n2*p_1*p_2))
            pos=pos+1
            if(i!=j):
                m_cov[j,i]=m_cov[i,j]
                
    return m_cov

In [9]:
def matriz_covarianza_x16(Px16,Qx16,T_area):
    #El tensor P
    #El tensor Q
    #Las coordenadas de los puntos para las regiones de interes
    d=Px16[0].shape[0]
    m_cov=np.zeros((d,d))
    pos=0
    p_1=0
    p_2=0
    q=0
    
    for i in range(d):
        for j in range(i,d):
            for k in range(len(Qx16)):
                y1=T_area[k][0]
                x1=T_area[k][1]
                y2=T_area[k][2]
                x2=T_area[k][3]
                col=x2-x1
                fil=y2=-y1
                q=Qx16[k][pos][y1,x1]+Qx16[k][pos][y1+fil,x1+col]-Qx16[k][pos][y1,x1+col]-Qx16[k][pos][y1+fil,x1]+q
                p_1=(Px16[k][i][y1,x1]+Px16[k][i][y1+fil,x1+col]-Px16[k][i][y1,x1+col]-Px16[k][i][fil+y1,x1])/(fil*col)+p_1
                p_2=(Px16[k][j][y1,x1]+Px16[k][j][y1+fil,x1+col]-Px16[k][j][y1,x1+col]-Px16[k][j][fil+y1,x1])/(fil*col)+p_2
                #print(k)
            n1=(1/((fil*col)-1))*16
            n2=(1/(fil*col))*16    
            m_cov[i,j]=(n1)*(q-(n2*p_1*p_2))
            pos=pos+1
            if(i!=j):
                m_cov[j,i]=m_cov[i,j]

                
    return m_cov

In [12]:
test=[]
n1=0
n2=1
n3=2

test.append(np.zeros((4,3)))
test.append(np.zeros((4,2)))
test.append(n1,n2,n3)
len(test)

TypeError: append() takes exactly one argument (3 given)

## Funcion para Obtener las covarianzas del video
- Recibe el video(el path ) , las trayectorias en texto y el area de la covarianza de entrada
- Calcula mis tensores Q, P
- Devuelve Lista de listas con las covarianzas organizadas, segun la matriz de trayectorias, y la matriz de trayectorias.

In [10]:
trayectorias="16 31 22 137 22.0509 136.951 22.08 137.066 21.0317 137.621 19.6865 140.775 18.2122 142.948 17.5346 146.148 17.4286 151.478 17.7006 153.091 18.8177 158.471 20.7017 162.986 22.0004 169.79 23.3447 176.793 24.4449 179.655 25.4684 181.9 26.1483 182.712 16 31 22 157 21.9934 156.983 21.9994 157.007 21.9805 157.038 22.0067 157.009 21.8544 157.135 21.8525 159.206 21.836 164.497 22.5334 170.133 22.9572 175.397 24.3987 180.963 26.2425 187.322 26.6474 193.078 27.1124 200.357 27.1264 207.872 26.9729 216.499 16 31 27 107 26.2381 109.566 25.4565 112.655 23.9643 117.292 22.6305 121.673 21.584 126.6 21.0033 131.794 20.9738 136.831 21.5505 142.323 22.8098 148.037 24.7264 154.42 26.1026 162.084 27.2724 170.052 28.7253 176.436 29.34 183.123 30.1224 189.226 16 31 27 112 25.6172 116.398 24.3675 120.454 23.0586 124.904 21.7606 129.018 20.7895 133.438 20.3588 137.944 20.2627 143.156 20.4921 149.557 21.7178 155.667 23.6642 161.908 25.1282 169.09 26.6483 176.333 27.6014 183.987 27.7592 191.864 27.7116 199.448 16 31 27 117 25.5565 120.697 24.3386 124.492 22.9121 129.219 21.5251 133.509 20.5465 137.562 20.0679 143.2 19.9552 148.611 20.3433 153.16 21.5264 160.325 23.5886 166.094 25.0554 173.141 26.5837 179.947 27.508 188.084 27.5182 196.356 27.5413 203.895 16 31 27 122 25.3826 125.342 24.031 128.949 22.4394 133.273 21.0319 137.08 19.9851 141.291 19.5065 147.108 19.4427 152.92 19.8609 157.468 20.8616 163.001 22.9878 169.337 24.3531 175.016 25.9143 181.689 26.7684 189.706 26.7915 197.915 26.8106 205.964 16 31 27 127 25.4033 130.661 24.0638 134.668 22.389 138.452 21.0438 142.457 19.4194 144.899 18.7966 149.557 18.6922 155.281 19.0981 160.178 19.8759 164.64 22.0272 171.198 23.2309 176.611 24.825 183.225 25.5249 190.061 25.5352 198.056 25.5529 205.704 16 31 27 132 26.4398 134.286 25.0119 138.029 23.2696 141.869 22.0424 146.073 20.3359 149.226 19.8486 154.137 19.5545 159.385 20.0215 164.84 21.2117 170.625 22.888 176.723 24.336 182.141 25.7229 189.559 26.1414 196.487 26.1626 204.014 26.1372 212.852 16 31 27 137 26.8052 137.143 25.292 139.604 23.4548 142.304 22.2277 146.509 20.5212 149.614 19.8881 154.526 19.5283 159.685 19.9949 165.119 21.1851 170.904 22.8615 177.003 24.3094 182.42 25.6963 189.839 26.1148 196.767 26.1343 204.207 26.109 213.046 16 31 27 142 27.047 141.978 26.519 142.148 24.6505 145.283 23.4234 149.46 22.3619 153.666 22.1393 158.506 22.1229 163.797 22.8203 169.432 23.5732 175.099 25.1473 181.055 27.2534 188.15 27.6367 194.157 28.1964 201.458 28.2695 209.149 28.1693 217.948 16 31 27 147 27.1521 146.952 27.0937 147.045 26.6978 147.556 23.0524 150.003 22.1659 153.796 21.9434 158.636 21.927 163.927 22.6243 169.562 23.0482 174.827 24.4896 180.392 26.5003 187.209 27.0059 193.361 27.4708 200.641 27.4558 208.045 27.3023 216.672 16 31 27 162 27.0062 162.015 27.0215 162.013 26.9903 161.968 27.0266 162.007 27.0004 162.013 26.8635 163.555 27.1002 168.939 27.4757 174.316 28.1477 180.307 30.7924 186.706 31.4807 192.612 32.9359 199.423 33.833 206.444 34.0059 214.122 34.1424 223.131 16 31 32 97 31.8301 99.9584 31.3277 103.552 31.5283 107.403 31.6608 110.958 30.9222 115.269 31.0922 119.931 31.2485 124.658 31.6749 130.597 32.3797 136.651 34.3269 142.875 36.6105 149.121 38.1555 156.355 38.5738 162.149 38.2065 165.477 38.2739 165.573 16 31 32 102 30.7074 106.143 31.2972 108.516 30.5323 112.546 29.2274 116.744 29.2542 120.554 28.9235 125.455 29.4524 130.264 29.9424 136.137 31.0133 142.094 32.8744 148.668 34.542 155.476 35.1839 163.454 35.7182 171.134 35.6515 179.279 35.4851 186.696 16 31 32 107 30.7399 110.844 29.8873 114.409 28.5054 118.978 27.163 123.443 26.1328 128.354 25.6679 133.564 25.5929 138.795 26.1746 144.455 27.5333 150.073 29.3118 156.635 30.7212 163.996 32.0696 171.398 33.2185 178.128 33.3696 185.987 33.5139 193.398 16 31 32 112 30.6344 116.505 29.4508 120.643 28.1348 124.899 26.8561 128.923 25.83 133.687 25.4322 138.412 25.4144 143.472 25.7401 149.645 27.1582 155.246 29.0745 161.449 30.4713 168.683 31.8364 175.757 32.8073 183.136 32.9566 190.916 32.9047 198.714 16 31 32 122 30.4722 125.852 29.1697 130.152 27.6741 134.259 26.5659 139.006 25.4986 143.312 25.1026 148.66 25.0098 154.294 25.4947 159.635 26.5963 165.692 28.6024 172.224 30.0096 177.931 31.5033 185.368 32.2141 192.702 32.215 200.237 32.301 209.342 16 31 32 127 30.7087 131.428 29.3486 135.774 27.8934 140.186 26.6338 144.781 25.6489 149.527 24.9436 154.307 24.652 159.597 25.2159 165.178 26.3029 171.324 28.1961 177.68 30.2188 184.779 31.0975 191.236 31.6118 198.508 31.7144 206.44 31.8206 215.513 16 31 32 132 30.3973 136.042 29.0232 140.399 27.4289 144.545 25.9328 148.735 25.3128 153.9 24.7984 158.747 24.749 164.255 25.3758 169.911 26.1803 175.751 27.9275 181.872 29.8948 188.879 30.5096 195.331 31.1653 202.644 31.4328 210.574 31.439 219.572 16 31 32 137 30.6649 138.975 29.8277 143.267 27.9328 147.186 25.0523 150.723 24.5294 155.645 24.2618 160.419 24.3941 166.002 25.1483 171.608 25.598 177.181 27.3956 183.279 29.2386 190.164 29.7631 196.501 30.7688 203.952 31.0906 211.772 31.1644 220.757 16 31 32 142 32.0038 142.105 31.2341 144.73 29.2458 148.038 26.0446 151.186 25.6314 156.274 25.2673 161.002 25.4752 166.593 26.0626 172.115 26.7347 178.042 28.8294 184.521 29.8445 190.732 30.7543 197.334 31.6527 204.747 31.9539 212.546 32.1473 221.555 16 31 32 147 32.0008 147.035 31.8428 147.138 30.0081 149.107 26.7963 152.088 26.3832 157.127 26.0421 161.779 26.2504 167.325 26.7618 172.852 27.5794 179.078 30.1471 185.65 30.8775 191.575 32.2964 198.485 33.1935 205.727 33.4215 213.355 33.5581 222.391 16 31 32 152 32.1037 152.044 32.0395 152.005 32.0036 152.192 30.5628 153.825 29.1548 158.351 28.409 162.87 28.5693 168.3 28.9928 173.833 30.3903 180.476 32.958 187.025 34.1882 193.517 36.0701 200.98 37.1874 207.953 37.3717 215.572 36.3839 224.131 16 31 32 167 32.0561 166.723 32.0242 166.682 32.145 166.815 32.0364 166.726 31.9727 166.692 32.0038 167.187 32.1811 170.821 32.6214 175.244 33.7123 181.367 35.6186 187.781 37.3881 194.412 39.1939 201.851 40.5617 208.707 40.2713 216.321 39.0217 224.955 16 31 37 97 35.7977 101.176 34.614 105.37 33.3794 109.771 32.272 114.178 31.4764 118.625 31.1757 123.546 31.5266 128.468 31.981 134.487 32.8185 140.581 34.9257 147.041 36.8511 153.795 38.5069 161.342 38.8107 169.099 38.4791 177.346 37.447 185.303 16 31 37 102 35.8457 106.113 34.7039 110.151 33.7003 114.325 32.3587 118.699 32.1008 122.599 31.6852 127.54 32.0645 132.474 32.5368 138.441 33.5852 144.53 35.525 150.889 37.2879 157.677 38.2395 165.337 38.7418 173.034 38.5621 181.21 38.0171 188.954 16 31 37 122 36.5518 123.624 36.5412 123.647 36.5389 123.65 36.5348 123.65 36.5099 123.793 36.1539 127.53 36.033 132.663 36.5341 138.441 37.6323 144.614 39.5583 150.614 41.6651 157.028 44.0281 164.022 44.9172 171.613 45.6997 179.53 45.1318 187.113 16 31 37 127 35.5222 130.911 34.4835 135.256 33.0238 139.441 31.7569 143.783 31.1006 148.568 30.3819 153.208 30.21 158.363 30.797 163.74 31.8715 169.701 33.6856 175.783 35.4454 181.923 36.3924 187.776 36.88 193.175 36.843 194.06 36.8792 194.635 16 31 37 132 34.9503 135.801 34.0759 140.443 32.6237 144.734 31.1613 149.015 30.2739 153.913 29.5588 158.556 29.2324 163.873 29.7665 169.506 31.2519 175.982 33.3541 182.525 35.1092 189.489 36.8045 196.76 37.5749 203.853 37.6184 211.657 37.2255 220.567 16 31 37 137 35.5854 140.381 34.5034 144.847 32.7101 148.961 31.6741 153.098 30.1301 157.664 29.3174 162.196 29.2644 167.581 29.6879 173.114 31.1507 179.756 33.536 186.25 35.0801 193.072 36.8907 200.613 38.0684 207.622 38.2179 215.223 37.3346 223.871 16 31 37 142 37.2364 142.887 35.9383 146.178 34.3037 149.943 33.2804 153.953 31.6726 158.495 30.8687 163.07 30.5039 168.37 31.0663 173.945 32.529 180.521 34.5073 186.959 36.1475 193.702 38.0438 201.211 39.249 208.213 39.3657 215.823 38.1146 224.445 16 31 37 147 36.995 147.096 36.906 147.346 35.5787 150.262 34.5256 154.513 33.0874 158.937 32.5278 163.475 32.0766 168.754 32.657 174.249 33.5914 180.648 35.4976 187.062 37.1378 193.805 38.9437 201.245 40.2388 208.214 40.3243 215.841 39.0747 224.474 16 31 37 152 36.989 152.145 36.9841 152.016 36.7412 152.435 35.7178 155.272 34.5897 159.742 34.1881 164.087 33.7369 169.372 34.3489 174.867 35.2833 180.972 37.0848 187.354 38.8544 194.28 40.095 201.552 41.5189 208.379 41.2286 216.051 40.0905 224.706 16 31 37 157 36.9844 157.141 36.9726 156.983 36.9063 157.209 36.8057 157.503 37.3358 159.583 37.0023 163.873 36.7382 169.169 37.0658 174.795 38.0104 180.937 40.0697 187.272 41.8649 194.178 41.6521 201.199 43.1827 208.131 42.7457 215.901 41.8878 224.623 16 31 37 162 36.9983 161.922 37.0128 161.888 37.0124 162.016 36.9565 161.997 36.9725 162.367 36.8402 165.166 36.4543 170.09 36.6868 175.29 37.6315 181.432 39.6908 187.767 41.48 194.361 41.5551 201.462 43.0857 208.394 42.6486 216.164 41.7908 224.886 16 31 37 167 37.0065 166.956 37.0128 166.892 37.0245 166.987 37.0281 166.902 37.0276 166.9 37.0049 167.128 36.4193 170.19 36.6519 175.391 37.5965 181.533 39.5284 187.625 41.3177 194.219 41.3927 201.321 42.8192 208.234 42.3822 216.004 41.3234 224.695 16 31 42 92 41.2632 95.8794 40.2619 99.9757 38.683 104.466 37.6257 108.748 36.4428 113.281 36.0759 118.029 35.2384 123.229 35.5352 129.056 36.1708 134.973 38.0664 140.458 39.9354 145.659 40.1619 149.165 40.0623 149.288 40.0717 149.155 40.0522 149.201 16 31 42 97 40.8027 101.18 39.5761 105.409 38.0548 109.866 36.9399 114.265 35.856 118.84 35.4377 123.791 34.9196 129.163 35.3843 135.166 36.2219 141.376 38.6447 147.605 41.07 154.243 43.8699 161.282 44.7664 168.889 45.6113 176.837 45.8751 184.39 16 31 42 102 41.4751 105.334 40.3286 109.486 39.0041 113.891 37.8771 117.988 36.8726 122.432 36.4625 127.329 36.1675 132.631 36.6687 138.41 37.7669 144.582 39.6929 150.582 41.7997 156.997 44.1627 163.99 45.0518 171.581 45.8343 179.498 45.5573 187.081 16 31 42 122 41.9777 122.081 41.977 122.083 41.9767 122.085 41.9717 122.089 41.5849 123.406 40.8144 127.926 40.5656 132.629 40.813 138.099 41.4108 144.168 42.9496 150.227 45.5275 156.644 46.2285 163.525 47.2799 171.013 48.2075 178.849 48.2336 186.331 16 31 42 127 40.5948 130.955 39.1823 135.339 38.0238 139.528 37.0075 143.958 36.2618 148.3 35.8453 152.71 35.9677 157.525 36.4904 162.8 36.8139 166.231 36.8224 166.237 36.8131 166.249 40.3075 166.988 40.9592 174.303 40.662 182.346 40.0326 190.043 16 31 42 132 40.4984 136.014 38.9717 140.53 37.6725 145.026 36.6618 149.49 35.5322 154.056 35.1016 158.693 34.9374 163.966 35.4577 169.723 36.1523 176.092 38.2542 182.521 39.9368 189.625 40.8629 196.859 42.1533 203.942 42.2427 211.816 41.5876 220.749 16 31 42 137 41.4201 140.854 39.4565 145.229 38.2003 149.568 36.9951 153.858 35.8181 158.416 35.3721 163.026 34.9861 168.323 35.5602 174.046 36.3093 180.402 38.3139 186.84 40.1032 193.765 40.5942 200.977 42.0207 207.89 41.7304 215.562 40.6717 224.253 16 31 42 152 41.9818 152.197 42.0179 152.067 42.0604 152.569 41.9346 154.211 41.4696 158.756 41.3775 163.02 41.3977 168.146 41.9731 173.66 42.692 179.955 44.9022 186.345 47.1455 193.048 47.7917 199.649 48.2259 207.079 48.7572 214.778 48.7625 223.288 16 31 42 157 41.9849 157.125 41.9749 157.068 41.9582 157.062 41.8822 157.114 42.2298 159.362 42.2486 163.429 42.3308 168.489 42.9673 173.879 43.6942 180.071 45.8303 186.403 47.9043 192.962 48.2649 199.584 48.6991 207.014 49.4903 214.665 49.4956 223.174 16 31 47 92 46.3089 95.8946 45.2541 99.9582 44.4093 104.31 43.4614 108.6 42.7681 113.083 42.2961 117.827 42.2251 122.865 42.8842 128.439 43.1569 133.063 44.2825 136.724 44.1555 137.636 44.2449 137.783 44.153 137.777 44.2312 137.779 44.1793 137.7 16 31 47 97 45.4733 101.127 44.255 105.341 42.8757 109.798 41.8555 114.256 41.0887 118.806 40.4645 123.748 40.6674 129.037 41.3392 134.982 41.9332 141.233 43.6101 147.415 46.1973 153.829 48.5017 160.904 49.3042 168.185 49.9386 175.832 51.1534 183.312 16 31 47 102 45.7185 104.793 44.4013 108.586 43.7556 112.737 42.8111 117.055 41.9704 121.577 41.2435 126.4 41.4574 131.636 41.889 137.473 42.483 143.722 43.9363 149.781 46.4961 156.201 48.1148 163.186 49.1106 170.493 49.5569 178.174 50.4872 185.617 16 31 47 122 45.8831 125.594 44.6981 129.561 43.5089 133.58 42.469 137.497 42.2103 138.799 42.1728 139.314 42.1749 139.333 42.1773 140.493 42.5158 145.56 42.5781 150.724 44.8757 156.999 45.5767 163.848 46.6281 171.335 47.5557 179.172 47.5818 186.653 16 31 47 127 45.7204 131.118 44.2729 135.536 42.8627 140.009 41.5442 144.484 40.6216 149.417 39.9891 154.409 40.2399 159.513 40.5744 165.427 41.1988 171.814 42.971 178.46 44.7665 185.414 45.5473 192.602 46.7118 200.012 47.1552 207.909 46.8338 216.432 16 31 47 132 45.6949 135.899 44.3155 140.432 42.8654 144.891 41.4148 149.418 40.7094 153.955 40.369 158.587 40.353 163.783 40.8453 169.495 41.4835 175.921 43.6848 182.358 45.5306 189.386 46.4567 196.311 48.1178 203.478 48.6491 211.317 48.9268 219.949 16 31 47 137 45.926 140.756 46.1117 144.67 45.479 148.115 44.0274 152.377 43.2109 156.985 43.0067 161.473 43.0255 166.583 43.6068 171.999 44.4438 178.416 46.6556 184.829 48.4451 191.543 49.2646 198.237 49.9202 205.906 50.7114 213.61 50.9526 222.167 16 31 47 142 47.0262 142.189 47.8449 145.283 47.327 147.983 46.3333 151.67 45.2528 156.291 44.8801 160.893 44.8897 165.978 45.3902 171.394 46.3433 177.805 48.1152 184.31 49.4025 191.284 50.1669 198.342 50.3516 206.134 51.1428 213.837 51.384 222.395 16 31 47 147 47.042 147.083 47.1368 147.206 46.7847 148.803 45.8488 151.935 44.7683 156.556 44.4803 160.828 44.4991 165.919 45.0306 171.415 45.9837 177.826 47.7556 184.331 49.0429 191.305 49.8073 198.363 49.992 206.155 50.7832 213.858 51.0244 222.416 16 31 47 152 46.9698 152.185 47.0755 152.122 46.4648 152.215 45.9635 153.649 45.0854 157.044 44.7975 161.316 44.8071 166.4 45.3076 171.816 46.2608 178.21 48.0327 184.715 49.3229 191.308 50.0873 198.366 50.2719 206.158 51.0631 213.861 51.3043 222.419 16 31 47 157 46.9277 157.141 46.8965 157.202 46.6417 156.78 46.6062 156.762 46.2211 157.608 45.9226 161.33 45.9322 166.45 46.3859 171.779 47.4274 178.111 49.0627 184.714 50.1014 191.145 50.8172 198.312 50.8528 206.22 51.6787 214.059 51.7552 222.569 16 31 47 162 47.0099 162.039 46.9813 162.115 46.8339 162.058 46.7952 162.084 46.7853 162.331 47.0483 164.93 47.7158 169.506 48.6874 173.929 50.3374 179.595 51.9916 184.469 53.0067 190.97 53.7579 198.541 53.4258 205.358 53.962 213.823 53.7828 222.116 16 31 52 92 50.6296 96.0474 49.2867 100.227 47.6704 104.509 46.6096 108.958 46.2611 113.495 45.5367 118.29 45.1834 123.27 45.9797 128.873 47.1039 134.925 48.5939 141.178 50.1678 147.257 51.3755 153.043 51.4891 157.44 51.5654 157.4 51.6431 157.421 16 31 52 97 50.7295 101.225 49.4513 105.457 47.9119 109.969 46.8957 114.499 46.2167 119.098 45.8582 124.199 45.7137 129.406 46.7415 135.449 48.4252 141.994 49.4638 148.179 51.4677 154.435 54.2979 161.704 55.2509 169.069 55.1295 176.762 53.2782 183.749 16 31 52 102 51.1995 106.153 50.1285 110.228 48.7198 114.548 47.7568 118.865 46.9146 123.3 46.6675 128.317 46.6417 133.495 47.5091 139.145 48.9389 145.45 50.3934 151.526 51.3785 157.307 53.8126 164.442 54.6149 171.662 53.8735 178.935 53.7082 185.658 16 31 52 112 50.9942 114.847 50.6708 116.416 50.7803 117.883 50.3836 120.48 49.4334 124.888 49.1922 129.604 49.2778 134.627 50.1058 140.208 51.5958 146.513 53.6393 152.875 55.3485 159.124 56.4287 165.743 56.1853 172.396 54.6322 179.579 52.4846 185.931 16 31 52 122 50.7013 126.128 49.4642 130.464 48.0688 134.942 46.7928 139.328 45.8136 144.315 45.4597 149.417 45.721 154.467 46.356 160.421 47.4721 166.611 49.1075 173.435 50.1879 180.779 51.3563 188.087 51.8443 196.15 52.5401 204.472 52.8324 213.256 16 31 52 127 50.6395 131.234 49.4649 135.417 47.954 140.004 46.7077 144.456 45.4773 149.643 44.7904 154.68 44.7367 159.83 45.1613 165.787 46.0511 172.142 47.644 179.029 48.844 186.272 49.5776 193.676 49.9682 201.861 50.4116 210.219 50.7454 218.935 16 31 52 132 50.6863 136.205 49.3133 140.535 47.8481 144.959 46.7428 149.131 45.4679 154.309 44.8884 159.247 44.8497 164.411 45.2656 170.175 46.1883 176.587 47.9253 183.294 49.2126 190.343 49.977 197.527 50.1616 205.319 50.8649 213.216 51.2125 221.847 16 31 52 137 51.037 139.778 50.1989 143.974 49.529 146.527 48.4802 150.074 47.1751 155.176 46.4961 160.066 46.4826 165.251 46.8985 170.848 47.9401 177.243 49.5098 184.074 50.5559 190.884 51.2717 198.253 51.3073 206.161 52.1332 214 52.2097 222.51 16 31 52 142 51.6041 141.823 51.7585 144.866 51.0937 147.347 49.8315 151.063 48.435 155.973 47.756 160.546 47.804 165.777 48.1973 171.274 49.3251 177.668 50.8876 183.929 51.9259 190.431 52.6629 197.97 52.4361 205.452 52.9999 213.737 52.8768 222.237 16 31 52 147 51.9409 146.853 52.169 147.219 51.5907 148.982 50.2713 152.1 48.9476 156.855 48.254 161.167 48.302 166.398 48.6953 171.895 50.0287 177.965 51.5984 183.707 52.6136 190.209 53.4618 197.789 53.2349 205.271 53.7711 213.737 53.5919 222.03 16 31 52 152 51.9317 152.011 51.9358 152.021 51.7066 152.585 51.3428 154.035 50.1824 158.335 49.4756 162.421 49.6146 167.691 50.0011 173.071 51.4288 178.737 53.1443 183.805 54.1588 190.478 55.007 198.108 54.7581 205.143 55.3769 213.127 55.2538 221.444 16 31 52 162 51.9993 162.003 51.9759 162.096 51.9258 161.983 51.9156 162.036 51.9641 162.133 51.7988 163.976 52.1768 168.759 52.4557 174.025 53.9154 179.506 55.8485 184.495 56.4229 191.785 57.1745 199.475 56.9356 206.565 57.4958 213.872 57.721 221.387 16 31 57 92 55.9799 96.1871 54.9862 100.637 53.7932 105.161 53.3766 109.761 53.0021 114.256 52.8283 119.439 51.9517 124.533 52.0988 129.85 53.6435 136.184 55.7041 142.692 57.5571 148.816 59.0378 154.939 59.0776 161.778 58.2807 169.101 57.6327 175.612 16 31 57 97 55.8632 101.281 54.8676 105.745 53.5316 110.288 52.9017 114.935 52.1741 119.516 51.8477 124.714 51.3174 129.956 51.7732 135.585 53.2738 142.07 55.434 148.551 57.174 154.789 58.4631 161.412 58.527 168.2 57.4401 175.524 55.7616 182.543 16 31 57 102 55.9031 106.257 54.9324 110.654 53.6933 115.123 52.9843 119.726 51.9881 124.197 51.7058 129.224 51.2924 134.454 51.9426 140.097 53.4374 146.469 55.5226 152.97 57.3973 159.311 58.4774 165.951 58.2847 172.604 56.5935 179.646 54.4459 186.324 16 31 57 107 56.3705 109.912 55.6004 113.313 54.4806 117.458 53.7409 121.776 52.7447 126.153 52.765 130.894 52.3685 135.882 53.1886 141.525 54.6903 147.66 56.7595 154.022 58.6891 160.361 59.7492 167.064 60.0986 173.664 59.3349 180.583 58.2734 187.254 16 31 57 112 55.6204 116.41 54.3416 120.816 53.3065 125.485 52.0616 130.117 51.1821 134.605 50.9351 140.222 51.3241 145.428 51.8806 151.005 53.0478 157.291 54.5514 164.157 55.7723 171.971 56.4768 180.233 57.2056 188.294 57.6683 196.469 57.7291 203.835 16 31 57 117 55.5512 121.343 54.2337 125.787 53.1259 130.527 51.7796 135.182 50.9147 139.795 50.614 145.449 50.9341 150.714 51.5369 156.388 52.5487 162.747 54.0263 169.601 54.9751 177.67 55.9001 185.776 56.331 194.393 57.0692 202.868 57.5707 211.815 16 31 57 122 55.6421 126.153 54.445 130.571 53.09 135.015 51.7987 139.468 50.7996 144.142 50.4787 149.631 50.6534 154.69 51.2949 160.569 52.3251 166.844 53.7899 173.675 54.6233 181.241 55.5037 189.079 55.9734 197.168 56.7737 205.512 57.5776 213.538 16 31 57 127 55.6853 131.265 54.5251 135.291 53.0819 139.727 51.8384 144.088 50.4488 149.293 49.774 154.285 49.747 159.36 50.2594 165.243 51.2541 171.612 52.8011 178.572 53.6422 186.004 54.4815 193.501 54.8657 201.734 55.8396 210.2 56.8241 218.511 16 31 57 132 56.5865 134.683 55.4853 137.128 53.708 141.532 52.5429 145.598 50.8181 151.064 49.9756 156.065 49.9238 161.2 50.174 167.097 51.0989 173.66 52.6458 180.648 53.6329 188.098 54.4912 195.664 54.4648 203.788 55.0836 212.288 55.285 220.808 16 31 57 137 56.2436 139.937 54.6241 143.405 53.3695 146.491 52.1699 150.845 50.6711 155.688 49.8447 160.4 49.9111 165.655 50.1769 171.425 51.3416 177.624 52.9535 183.143 53.9873 190.29 54.8355 197.921 54.5866 204.955 55.2054 212.939 55.0823 221.256 16 31 57 147 57.023 146.992 57.092 147.155 57.6248 148.118 57.7014 150.32 56.4163 154.83 55.6561 159.526 54.6753 165.344 54.6706 171.344 55.2302 177.765 56.6486 184.058 56.9804 191.492 57.3854 199.189 57.1465 206.28 57.7478 213.657 58.1421 220.691 16 31 57 152 56.9883 151.946 57.0135 151.932 57.133 152.007 57.6744 152.204 57.0624 155.23 56.2923 159.682 55.3114 165.5 55.3271 171.38 55.8866 177.801 57.2092 184.243 57.541 191.677 57.479 199.367 57.2401 206.457 57.8414 213.835 58.2357 220.869 16 31 57 157 57.0052 156.909 56.9505 156.825 56.972 156.989 56.9557 156.881 56.9672 158.097 56.7912 160.485 55.7844 166.097 55.7871 171.82 56.326 178.069 57.6486 184.51 57.6976 191.842 57.6356 199.531 57.4037 206.778 57.9639 214.084 58.3582 221.118 16 31 62 92 61.238 96.2591 60.2966 100.587 59.5172 105.208 59.667 109.514 59.1498 114.546 58.9139 120.118 57.699 124.963 59.4501 128.196 61.8677 131.819 65.7535 135.502 68.4869 139.959 71.3021 144.128 72.9699 147.786 73.8596 150.692 73.4099 153.714 16 31 62 97 61.1341 101.234 60.0484 105.741 59.2216 110.114 58.8679 114.561 58.5575 119.047 58.4261 124.14 57.6047 129.151 57.711 134.494 59.0653 140.713 61.4013 147.103 63.4495 153.261 64.9369 159.713 64.9809 166.671 64.3098 173.771 63.7636 180.23 16 31 62 102 60.9812 106.139 59.8526 110.541 58.8496 114.887 58.129 119.318 57.4663 123.803 57.2445 128.702 56.8537 133.746 57.5007 139.348 58.8175 145.731 60.9586 152.086 62.9938 158.343 64.0283 165.229 64.5619 171.802 64.1176 178.76 63.599 185.063 16 31 62 107 60.7495 110.931 59.6309 115.047 58.621 119.183 57.6257 123.072 56.9624 126.918 56.9098 130.98 56.6009 135.598 57.4214 140.768 58.7231 146.961 60.7566 153.206 62.7254 159.449 63.6636 166.404 64.2041 172.952 63.936 179.851 63.3957 186.038 16 31 62 112 60.4784 116.532 59.2946 121.157 58.3251 125.954 57.2354 131.149 56.8655 136.237 56.3577 141.472 56.4886 146.617 56.9029 152.237 57.6823 158.249 58.9362 164.93 60.2413 172.617 60.7902 180.693 61.6066 188.453 61.9438 196.657 61.72 204.697 16 31 62 117 60.216 121.506 59.0896 126.133 58 131.057 56.4814 136.292 55.6596 141.465 55.1422 147.105 54.8465 152.344 55.0213 157.875 55.5177 163.959 56.7676 170.617 57.6472 178.594 58.425 186.684 58.8792 195.194 59.6942 203.549 60.383 212.266 16 31 62 122 60.3659 126.361 59.188 131.036 57.6615 135.642 55.9481 140.068 54.3689 144.827 53.9019 150.504 53.2593 155.95 53.588 161.871 53.8027 168.293 55.0525 175.19 55.8719 182.756 56.689 190.438 57.1589 198.283 58.0353 206.371 58.8641 213.984 16 31 62 127 60.6571 131.277 59.2763 135.049 57.5431 139.617 55.9453 143.767 54.2233 148.549 53.3083 153.705 52.8212 158.982 52.5959 164.807 53.0391 171.37 54.2733 178.419 55.0368 185.868 55.8732 193.408 56.3111 201.121 57.2914 209.102 58.276 216.361 16 31 62 132 61.4367 135.377 60.4708 138.654 59.263 142.942 57.8339 147.039 56.1789 151.942 55.2312 157.018 54.2947 162.597 54.2309 168.712 54.7699 175.285 55.8781 182.195 56.5723 189.749 57.1535 197.434 56.9302 204.681 57.568 212.169 58.2358 219.195 16 31 62 137 61.4892 140.058 60.9547 142.503 60.167 145.207 59.1264 149.218 57.5676 153.813 56.6272 158.264 55.6384 163.764 55.5746 169.708 56.0534 176.129 57.1616 182.904 57.6216 190.354 57.8659 197.939 57.6408 205.185 58.2654 212.52 58.7172 219.546 16 31 62 142 61.9211 142.146 61.4422 144.262 60.6545 146.347 59.753 150.349 58.1942 154.786 57.404 159.057 56.392 164.604 56.3551 170.418 56.834 176.839 57.9043 183.51 57.9533 190.946 58.0369 198.632 57.805 205.849 58.3894 213.135 58.8412 220.161 16 31 62 147 62.0127 147.026 62.0618 147.186 61.3326 147.563 60.9039 150.966 59.302 155.232 58.3967 159.435 57.3669 164.742 57.33 170.352 57.7209 176.671 58.592 183.335 57.9345 190.785 58.0182 198.47 57.793 205.716 58.3774 213.002 58.8292 220.028 16 31 62 152 61.9287 151.955 61.8968 151.936 61.6746 151.906 61.5831 152.158 59.9356 155.886 59.1343 160.066 58.0933 165.061 58.0887 170.488 58.3015 176.736 59.1727 183.401 58.5152 190.85 58.2694 198.418 58.0443 205.665 58.6286 212.95 59.1171 219.599 16 31 62 157 61.9697 156.941 61.8934 156.863 61.9199 157.004 62.003 157.01 61.5515 158.032 61.6154 161.937 60.8444 166.326 61.1472 171.345 60.8845 177.567 60.1418 184.039 58.6882 191.501 58.3204 199.102 58.0885 206.32 58.6728 213.606 59.1246 220.351 16 31 67 92 66.6436 96.087 65.4348 100.438 64.8962 104.946 64.9716 109.259 65.0462 114.138 67.0589 119.058 71.2724 121.777 74.7097 124.306 77.7555 126.904 86.7305 129.976 87.2271 134.746 87.3423 139.7 87.432 143.884 87.1674 147.306 86.7692 149.997 16 31 67 97 66.3079 101.024 65.3245 105.397 64.8426 109.876 64.534 113.978 64.6431 118.753 65.1307 123.326 65.9364 126.856 66.5401 129.656 70.4019 132.128 71.3869 137.683 74.501 141.671 78.2817 145.055 81.7122 147.408 83.4006 149.876 83.4265 152.513 16 31 67 107 65.7518 111.241 64.5923 115.905 63.6649 120.27 62.8908 122.958 62.7389 124.169 62.6112 126.063 62.5532 130.163 62.8589 132.531 64.021 136.521 65.9606 141.913 68.3006 146.114 69.6092 151.519 70.4173 155.943 70.2238 160.372 69.1174 164.942 16 31 67 112 65.4943 116.858 63.9341 122.354 62.185 128.327 60.4984 134.333 58.8554 140.382 57.5075 145.951 56.5037 151.228 56.4722 156.637 56.7406 162.585 57.8957 169.148 58.7905 177.135 59.4341 185.257 59.916 193.765 60.7162 202.115 61.1235 211.049 16 31 67 117 65.3975 121.793 63.8752 127.187 62.0385 132.912 60.2914 138.663 58.4999 144.958 57.4862 150.628 56.5039 156.361 56.175 162.282 55.8766 168.634 56.6475 175.605 57.4467 183.189 58.256 190.87 58.7158 198.436 59.5817 205.995 60.364 212.093 16 31 67 122 65.3358 126.376 63.7239 131.369 61.8297 136.696 60.0695 141.446 58.2849 147.172 57.3227 152.011 56.3419 157.693 56.049 163.475 55.8121 169.888 56.5323 176.919 57.3314 184.502 58.1444 192.073 58.5826 199.587 59.459 206.73 60.302 212.828 16 31 67 127 65.3155 131.42 63.7037 135.602 61.7388 140.163 59.9899 144.657 58.2986 148.896 57.3906 152.956 56.4231 158.541 56.1707 164.084 55.9843 170.504 56.7045 177.552 57.4764 185.147 58.2893 192.718 58.6019 200.224 59.4782 207.366 60.3213 213.464 16 31 67 132 65.598 135.862 64.2979 139.706 62.85 143.661 61.4302 147.671 59.6566 152.118 58.4849 156.243 57.5217 161.337 57.3362 166.526 57.4059 172.683 58.0666 179.6 58.7388 187.199 59.2483 194.683 59.0969 202.046 59.8243 209.16 60.6311 215.041 16 31 67 137 67.5676 140.132 66.424 143.394 64.7733 146.752 64.3446 150.823 62.45 155.02 61.0626 159.201 59.8485 164.026 59.9488 169.277 59.8887 175.526 59.7115 182.213 58.6716 189.674 58.5895 197.158 58.3662 204.443 59.0305 211.778 59.6144 218.426 16 31 67 142 66.699 142.523 65.2654 144.851 63.4768 146.905 62.916 150.928 61.0451 155.241 59.7614 159.428 58.5605 164.341 58.5237 169.662 58.5004 175.91 58.6526 182.597 57.9951 190.047 58.2394 197.631 58.0143 204.878 58.6388 212.212 59.2227 218.861 16 31 67 147 66.965 147.082 66.9818 147.352 65.1932 148.727 64.8239 152.797 62.391 155.901 61.2498 160.08 60.2088 164.873 60.476 170.125 60.4037 176.373 60.2265 183.06 58.8281 190.52 58.5823 198.088 58.3587 205.369 58.9833 212.703 59.4718 219.352 16 31 67 152 66.9582 151.994 66.9557 152.014 66.653 151.974 65.9333 153.626 63.3807 156.561 62.7599 160.705 61.7189 165.306 61.8788 170.346 61.6161 176.594 60.8811 183.238 59.4275 190.793 59.1818 198.361 58.9582 205.642 59.5296 212.849 60.0181 219.252 16 31 72 92 70.9589 96.0739 69.4376 100.314 69.0667 104.865 68.5761 109.162 67.8453 113.496 68.5172 117.546 73.6912 120 76.0613 122.57 78.9569 125.306 87.5124 128.605 87.9136 133.216 88.0642 138.276 88.0799 142.388 87.8319 145.629 87.4337 148.224 16 31 72 107 71.3109 109.077 71.2511 109.62 70.687 111.706 69.619 115.524 68.0536 119.09 67.5313 122.641 68.8068 125.408 70.5999 128.141 75.5179 130.234 83.4399 133.26 85.1366 137.715 84.8721 142.581 85.0705 146.535 85.4629 149.417 85.274 152.108 16 31 72 112 70.5019 116.607 68.9215 122.039 67.2791 127.946 65.8829 134.083 64.5326 140.118 63.2337 146.187 62.2815 151.568 61.8612 156.25 61.8602 160.216 62.5587 165.893 63.8186 172.094 64.5504 177.004 64.989 181.195 65.1516 186.474 64.6998 191.348 16 31 72 117 70.4928 121.947 68.8195 127.134 66.9772 133.142 65.3489 139.035 63.5609 145.16 62.1685 150.771 60.6241 156.297 59.542 161.674 59.0307 167.788 59.3974 174.63 60.067 182.217 60.6402 189.756 61.0514 197.127 61.8858 204.532 62.487 211.636 16 31 72 122 70.3792 126.675 68.7089 131.299 66.7622 136.96 65.0684 141.301 63.252 146.142 61.8799 151.356 60.3355 156.882 59.3582 162.295 58.8469 168.433 59.2136 175.275 59.8832 182.861 60.4564 190.329 60.9126 197.792 61.7469 204.846 62.3482 211.95 16 31 72 127 70.3557 131.364 68.7057 135.791 66.792 139.671 65.1743 143.519 63.3403 147.859 62.0146 152.271 60.4497 157.571 59.7027 162.974 59.2379 169.111 59.5993 176.017 60.2093 183.639 60.7758 191.097 61.1854 198.45 62.0198 205.504 62.6611 211.397 16 31 72 132 71.2152 135.535 69.9128 139.04 68.1957 142.572 66.7077 146.367 64.897 150.701 63.6356 154.873 62.2123 159.747 61.736 165.064 61.7142 171.277 61.8609 178.2 61.253 186.048 61.2519 193.175 61.3946 200.488 62.1005 207.417 62.7562 213.148 16 31 72 137 71.8967 140.213 70.3152 143.349 67.8673 146.597 66.7938 150.474 64.3876 154.274 63.0984 158.42 61.8971 163.226 61.8641 168.408 61.7847 174.728 61.3931 181.557 59.9527 189.112 59.6666 196.412 59.4579 203.724 60.0825 210.931 60.7503 217.207 16 31 72 142 71.7861 142.737 70.2596 145.368 67.6693 148.332 66.8196 152.208 63.7462 155.713 62.5192 159.859 61.4269 164.606 61.6941 169.783 61.4314 176.031 60.9738 182.758 59.5203 190.313 59.1525 197.659 58.9289 204.94 59.5302 212.166 60.0894 218.484 16 31 72 147 71.9985 147.008 72.0021 147.804 71.0115 150.642 70.1827 154.025 67.2413 157.508 66.3818 161.292 64.8244 165.783 61.0262 169.824 60.8871 176.074 60.4296 182.801 59.0312 190.26 58.9491 197.744 58.7255 205.025 59.3268 212.251 59.9107 218.9 16 31 72 157 72.0121 157.008 72.0502 157.001 71.9836 156.936 72.0008 157.109 70.9458 158.959 69.6772 162.252 65.8261 166.231 61.3686 170.178 61.2295 176.428 60.7719 183.155 59.3184 190.71 59.0726 198.278 58.849 205.559 59.4205 212.766 59.909 219.414 16 31 72 162 72.0108 162 72.019 162.02 72.0137 162.011 72.0131 162.046 71.9651 162.321 71.2863 164.335 67.7474 167.875 62.8488 171.33 62.0597 177.101 61.3247 183.744 59.8442 191.335 59.3839 198.768 59.1606 206.053 59.7321 213.26 60.2205 219.664 16 31 72 232 72.1013 232.087 72.0847 232.337 72.1107 232.232 72.1109 232.235 72.1124 232.121 72.0403 232.025 72.0394 232.024 72.1381 232.492 56.1387 248.398 44.462 256.26 36.713 259.428 35.6233 254.586 39.1767 246.619 48.6877 233.376 60.2027 223.809 16 31 77 92 76.4762 96.4465 75.9596 101.461 75.576 105.921 73.9496 109.842 71.6021 113.362 68.9476 116.434 73.8496 118.858 76.1983 121.386 78.7011 124.139 86.7651 127.504 86.9616 132.541 87.1049 137.588 87.1224 141.795 86.8744 145.036 86.4533 147.687 16 31 77 97 76.1433 101.54 75.7561 106.586 75.3684 111.003 73.8482 114.991 71.9069 118.636 70.6704 121.57 71.7507 123.977 74.6352 126.507 77.9799 128.918 86.5262 131.951 87.4137 136.576 87.4589 141.304 87.6092 145.496 87.3446 148.97 87.224 151.526 16 31 77 102 76.5184 103.135 76.1312 108.059 75.7434 112.391 74.572 116.352 73.5731 119.961 74.0576 122.824 74.6325 125.096 76.2374 127.608 79.5777 130 86.7053 132.976 87.9036 137.554 88.048 142.049 88.2299 146.19 87.9701 149.664 88.0408 152.127 16 31 77 112 76.8456 112.532 76.7301 113.138 76.6044 113.85 76.1995 116.988 76.2498 120.229 76.9188 123.078 78.2213 125.434 79.4944 128.03 81.7418 130.333 87.347 133.218 88.5454 137.797 88.7506 142.231 88.9326 146.247 88.6679 149.691 88.7291 152.107 16 31 77 142 77.2679 143.27 77.2515 146.651 76.1728 150.023 75.1782 153.561 73.4788 156.88 71.0224 160.15 66.9179 164.112 62.1553 167.954 62.0759 174.274 61.8057 181.139 60.3654 188.891 60.0793 196.191 59.8705 203.503 60.4716 210.618 61.1394 216.894 16 31 77 157 77.0295 157.034 77.0541 157.069 77.0767 157.115 77.092 157.493 76.5114 160.536 76.0169 163.709 72.7507 168.081 65.9642 170.429 63.1992 174.768 62.7016 181.606 61.2207 189.476 60.62 196.66 60.4066 203.982 61.0077 211.097 61.6196 217.236 16 31 77 162 77.0032 162.007 77.0226 162.029 77.0291 162.054 77.0265 162.011 76.9799 162.587 76.8567 165.244 73.7745 170.42 66.1206 172.277 62.9312 176.21 62.2944 182.937 60.8143 190.59 60.1653 197.89 59.944 205.194 60.5283 212.327 61.0704 218.586 16 31 77 227 77.0527 226.748 77.0525 226.809 76.9725 226.881 76.9972 226.914 77.0026 226.907 77.0105 226.918 76.9739 226.876 74.3302 230.201 58.4587 245.995 46.6541 254.486 38.6874 257.324 38.3172 250.397 41.3903 244.28 49.7739 233.455 61.285 224.03 16 31 82 87 81.6638 91.5196 81.7897 96.99 81.9027 102.394 81.9948 106.07 81.4565 109.396 81.1731 112.467 81.2459 114.93 81.2781 117.488 81.5543 119.317 81.7387 121.73 81.6482 124.337 81.6851 124.555 81.6659 124.597 81.7059 124.533 81.7035 124.628 16 31 82 92 82.0268 96.3118 82.4485 101.618 84.0365 106.696 84.559 110.39 84.429 113.962 84.6709 117.477 84.5171 120.531 84.5989 123.491 84.4933 125.95 84.6418 129.288 84.6177 134.565 84.5745 139.541 84.7557 143.785 84.6007 147.153 84.1472 150.124 16 31 82 102 81.7432 102.921 81.8202 107.222 82.4908 111.511 82.1461 114.975 83.1062 118.341 83.4726 121.739 83.2722 124.282 83.7169 127.184 83.7405 129.564 87.2653 132.381 88.1528 137.006 88.281 141.639 88.463 145.78 88.2031 149.254 88.0826 151.766 16 31 82 137 83.0987 140.545 82.4351 144.28 81.0011 148.661 81.1013 153.028 80.183 157.9 80.1138 163.276 76.4568 168.419 65.388 167.82 64.0667 173.415 63.7711 180.338 62.3762 188.528 61.5633 195.71 61.3815 203.033 61.9865 210.036 62.6277 215.958 16 31 82 142 82.153 144.644 81.5844 148.259 80.9407 152.404 81.2979 156.689 80.8545 161.642 80.6822 167.271 77.0309 172.165 64.0792 171.443 62.4916 176.745 61.7566 183.389 60.2766 191.042 59.8163 198.475 59.595 205.778 60.1552 212.911 60.6437 219.315 16 31 82 147 82.0946 146.771 81.8549 148.481 81.2113 152.625 81.5555 156.714 81.1201 162.093 80.9477 167.722 77.5514 172.616 63.3354 172.088 62.1811 177.518 61.2844 183.904 59.8039 191.495 59.3436 198.929 59.1203 206.214 59.6917 213.421 60.1802 219.825 16 31 82 152 81.976 152.198 81.9637 152.546 81.6211 152.738 81.9334 157.024 81.4979 162.403 81.3256 168.032 77.9293 172.926 63.7133 172.398 61.7378 177.089 61.0028 183.732 59.5223 191.323 59.062 198.756 58.8387 206.041 59.4101 213.248 59.8986 219.897 16 31 82 157 81.9243 157.266 81.9493 157.298 81.6997 156.647 81.797 158.525 81.4412 162.74 81.2577 168.352 77.8614 173.246 63.6454 172.717 61.2779 177.131 60.6788 183.774 59.1983 191.365 58.9525 198.933 58.7292 206.218 59.3006 213.425 59.7891 220.074 16 31 82 162 81.9628 162.464 81.9988 162.111 81.9104 162.307 81.9405 162.275 81.6173 164.588 81.4553 169.907 79.0419 174.51 67.1101 174.98 63.2063 178.719 62.0417 184.471 60.5611 192.153 59.7978 199.524 59.5914 206.927 60.1437 214.022 60.6173 220.626 16 31 82 167 82.0732 166.71 82.0909 166.209 82.0949 166.453 82.0962 165.995 82.0622 166.038 81.9585 170.764 79.219 175.451 67.2872 175.921 63.3833 179.661 62.0001 184.761 60.5196 192.414 59.7562 199.786 59.5498 207.188 60.1021 214.283 60.5757 220.887 16 31 82 172 82.0566 171.976 82.0711 171.484 82.0224 171.995 82.0062 171.54 81.9661 171.143 82.086 172.367 79.9112 176.555 71.0195 177.655 63.4218 176.229 62.785 182.956 61.3265 190.784 60.6774 198.084 60.464 205.417 61.0483 212.551 61.5735 218.868 16 31 82 177 82.0092 177.149 81.9727 177.075 81.9825 177.386 81.9352 177.134 81.9262 177.048 81.9689 177.423 80.937 173.467 65.9327 173.449 62.5418 177.382 61.7541 183.95 60.2736 191.633 59.6659 199.152 59.4503 206.482 60.0218 213.689 60.4954 220.293 16 31 82 217 82.0608 217.044 82.0526 217.49 82.0725 217.447 82.0918 217.382 82.0895 217.404 82.0905 217.366 82.2592 216.4 74.8953 226.805 62.6398 238.09 50.021 248.004 43.5465 248.935 37.9395 252.172 41.0625 246.017 50.296 232.844 61.8072 223.418 16 31 87 87 86.5835 92.1935 86.2957 98.4359 86.269 104.323 86.2048 108.19 86.0217 111.761 86.0666 115.349 85.9079 118.433 85.9299 121.369 85.6472 123.82 85.7123 127.126 85.6688 132.387 85.8054 137.381 85.8443 141.818 85.5963 145.119 85.164 147.913 16 31 87 92 86.4606 97.2567 86.7446 102.949 86.9779 108.63 87.1711 112.687 86.9364 116.501 86.8912 120.137 86.3061 123.282 86.4622 126.21 86.0935 128.745 87.3136 131.561 88.2011 136.187 88.3444 140.911 88.4394 145.064 88.1748 148.538 88.0543 151.049 16 31 87 97 85.9827 100.906 86.0217 105.94 86.3462 111.141 86.7335 114.847 86.5273 118.62 86.612 122.153 85.8199 125.138 86.1154 127.962 85.675 130.409 87.2763 133.216 88.4746 137.794 88.6189 142.289 88.8009 146.304 88.5363 149.749 88.5974 152.165 16 31 87 137 87.0388 139.869 86.1371 143.418 84.8954 147.895 84.9439 152.683 84.6848 158.076 84.5474 163.837 75.1459 167.598 66.0016 168.049 63.7388 172.92 63.4431 179.843 62.0295 187.749 61.2166 194.877 61.0532 202.189 61.675 209.159 62.3307 214.931 16 31 87 142 87.0035 143.606 86.5533 146.806 85.9235 151.844 85.8741 156.587 85.6044 162.144 85.597 167.79 76.0129 172.551 65.9458 173.254 62.5549 177.187 61.7672 183.755 60.2867 191.438 59.8264 198.871 59.6108 206.202 60.171 213.334 60.6595 219.738 16 31 87 147 87.0089 147.444 86.8758 148.986 86.4297 154.123 86.3532 158.016 86.0818 163.566 86.1505 169.116 76.7378 174.041 66.0454 174.715 62.5908 178.809 61.4263 184.56 59.9457 192.154 59.3381 199.673 59.1225 206.976 59.6748 214.121 60.1484 220.726 16 31 87 152 87.0123 152.246 86.9899 152.18 86.6643 152.483 86.6026 157.441 86.3489 162.991 86.37 168.584 76.9573 173.509 66.2649 174.182 62.8103 178.115 61.8469 184.398 60.3663 192.081 59.7587 199.6 59.5522 207.002 60.1046 214.097 60.5782 220.701 16 31 87 157 87.0186 157.16 87.0045 157.144 86.896 156.444 86.8181 159.202 86.5601 164.524 86.7177 169.854 77.305 174.997 69.2743 175.854 64.0165 177.587 62.9734 183.621 61.5149 191.362 60.702 198.617 60.4986 206.02 61.0588 213.152 61.584 219.47 16 31 87 162 87.0433 162.27 87.0458 161.754 86.9644 162.356 86.9714 161.899 86.7802 163.89 86.9056 169.32 77.2303 174.378 66.5379 175.051 62.634 178.791 61.4695 184.542 59.9889 192.136 59.3813 199.655 59.1657 206.959 59.718 214.104 60.1916 220.708 16 31 87 167 87.0958 166.874 87.1016 166.365 87.0564 166.838 87.0622 166.339 87.1385 167.122 87.3144 172.051 78.927 178.96 72.5513 179.253 63.1398 175.85 62.503 182.577 61.0445 190.406 60.433 197.669 60.2117 204.973 60.7961 212.106 61.3381 218.365 16 31 87 172 87.0504 171.715 87.0562 171.214 87.0277 171.741 87.0353 171.285 87.0292 170.998 87.1882 173.453 79.0463 180.362 72.9448 181.806 62.7642 177.546 61.8007 183.829 60.3202 191.512 59.7125 199.031 59.4969 206.361 60.0684 213.568 60.542 220.172 16 31 87 187 87.0039 186.978 86.9835 187.523 86.9812 187.512 86.9749 187.623 86.9989 187.479 87.0261 187.256 86.9114 181.646 73.7515 184.233 62.3946 179.248 61.3499 185.281 59.8693 192.875 59.2203 200.409 59.0047 207.713 59.548 214.712 60.0777 221.457 16 31 87 207 87.0743 206.963 87.07 206.996 87.0835 206.744 87.0999 206.738 87.0854 206.84 87.0695 206.697 86.6222 213.058 72.6659 230.611 56.4346 246.458 46.4422 251.482 40.3612 252.352 36.713 251.551 39.6925 245.647 48.9261 232.475 60.1118 223.662 16 31 92 82 91.6535 87.7311 91.3343 94.4048 91.1492 100.586 90.9263 104.778 90.8101 108.22 90.7794 111.953 90.7205 115.478 90.7183 118.281 90.4368 120.921 90.6355 123.217 90.5055 127.041 90.554 131.2 90.5599 133.552 90.5456 133.284 90.5385 133.036 16 31 92 87 91.6923 92.6925 91.4488 99.3192 91.1637 105.777 90.853 110.076 90.7348 113.876 90.7512 117.716 90.5569 121.331 90.7316 124.201 90.4549 127.75 91.0826 130.333 91.4139 134.013 91.7067 138.725 91.2392 142.363 90.9825 145.768 90.6061 148.208 16 31 92 92 91.3188 97.3452 91.0144 103.702 90.7316 109.888 90.5324 114.349 90.3309 118.237 90.2556 122.036 89.647 125.343 90.0207 128.204 90.0597 131.503 91.5448 134.153 94.5481 138.162 97.9448 142.467 99.9948 145.681 100.734 148.291 100.18 149.991 16 31 92 112 91.9407 112.039 91.9421 112.053 91.9622 112.24 92.1779 115.09 92.1234 118.926 92.3261 122.488 92.4156 125.96 93.2889 128.787 97.501 133.575 102.837 139.728 105.858 145.348 108.275 149.629 109.945 152.424 110.667 154.421 110.816 156.754 16 31 92 137 91.8834 138.802 92.7181 141.774 93.439 146.416 92.778 150.946 93.1909 156.106 94.185 162.092 77.9518 166.154 66.0118 165.555 64.6905 170.962 64.5256 177.925 63.5143 185.958 62.8107 192.938 62.9534 200.149 63.5498 206.758 64.1073 212.412 16 31 92 142 92.0771 143.765 92.3035 146.463 92.2141 151.288 91.8054 156.269 91.4584 161.633 91.7838 166.96 77.8289 171.535 63.0397 170.521 62.2507 176.292 61.7062 183.069 60.2262 190.722 59.7659 198.155 59.5446 205.458 60.1289 212.592 60.6174 218.996 16 31 92 147 91.9105 148.229 91.8593 149.864 91.5197 154.221 91.2792 159.127 90.9117 164.516 91.1684 169.276 78.8347 174.311 65.4172 174.23 62.4081 178.56 61.3633 184.594 59.8828 192.188 59.2751 199.707 59.0595 207.01 59.6119 214.155 60.0855 220.76 16 31 92 152 91.9853 152.163 92.0007 152.098 91.7156 155.044 91.5328 159.635 91.1032 164.73 91.3599 169.49 79.0262 174.525 67.0944 174.995 63.1905 178.735 62.0259 184.486 60.5454 192.168 59.782 199.54 59.5756 206.942 60.1279 214.037 60.6015 220.641 16 31 92 157 92.012 157.014 92.0086 156.993 92.016 157.113 91.9452 159.764 91.5155 164.859 91.7723 169.502 79.8773 174.782 67.0626 175.277 63.1587 179.017 61.9941 184.769 60.5136 192.422 59.7502 199.793 59.5438 207.196 60.0961 214.29 60.5697 220.895 16 31 92 162 91.9707 162.184 91.9698 161.812 91.9502 162.388 91.9757 162.017 91.6577 163.89 91.9599 168.823 79.3449 173.776 65.9274 173.696 62.4728 177.629 61.5761 184.016 60.0956 191.698 59.4879 199.217 59.2646 206.502 59.8169 213.647 60.2905 220.252 16 31 92 177 92.0119 176.822 92.0609 176.998 92.0415 176.965 92.0426 176.927 92.0485 176.732 92.0485 177.313 82.7203 181.141 72.6414 182.006 62.4608 177.746 61.5641 184.133 60.0836 191.815 59.4759 199.334 59.2526 206.619 59.805 213.764 60.2786 220.368 16 31 97 87 96.9914 92.6637 98.2649 99.3976 104.913 105.959 102.767 110.017 102.185 113.947 102.437 117.501 102.626 121.325 104.535 125.283 105.763 130.711 104.653 135.554 107.702 141.144 110.177 145.222 111.958 148.128 112.613 150.344 112.6 152.645 16 31 97 97 96.9597 98.1395 99.0639 104.515 102.252 109.79 101.476 113.502 101.02 117.325 101.199 120.803 100.706 124.459 101.847 128.084 102.483 133.135 104.869 139.025 107.847 144.615 110.129 148.83 111.887 151.619 112.663 153.662 112.854 155.965 16 31 97 142 96.8926 143.585 96.8724 145.423 99.2452 150.069 99.9655 154.332 100.569 158.819 98.9558 163.658 81.9215 168.252 66.0094 167.028 64.0015 172.166 63.8366 179.123 62.589 187.156 61.7552 194.222 61.7303 201.501 62.3352 208.43 62.9909 214.162 16 31 97 147 96.9871 148.507 96.3315 149.618 96.3376 152.913 96.4253 157.669 96.9162 162.384 95.7853 166.873 79.6141 171.425 63.2893 169.96 62.7389 176.023 62.1021 182.75 60.6221 190.403 60.0106 197.666 59.7893 204.969 60.3736 212.103 60.9329 218.42 16 31 97 152 96.9825 152.206 96.6543 152.574 96.4788 153.945 96.4038 158.255 96.8947 162.969 95.4135 167.29 79.9036 171.687 64.0935 170.698 62.5059 176 61.8691 182.727 60.389 190.38 60.0212 197.726 59.7999 205.03 60.3843 212.163 60.9435 218.48 16 31 97 162 96.9609 162.082 96.9552 162.047 96.9442 162.038 96.9284 162.095 96.2717 164.741 94.9462 168.538 81.1192 173.794 66.894 174.28 62.9901 177.944 62.0267 184.228 60.5462 191.91 59.7828 199.282 59.5672 206.612 60.1195 213.707 60.5931 220.311 16 31 97 167 97.0434 167.151 96.9497 167.126 96.9506 167.123 97.0146 166.997 96.9641 167.228 95.485 169.205 81.6581 174.461 67.8425 175.582 63.1974 178.493 62.2339 184.776 60.7534 192.429 59.99 199.801 59.7836 207.204 60.3359 214.298 60.8095 220.903 16 31 97 172 97.2655 171.812 97.0069 171.853 96.9988 171.866 97.182 171.725 97.2634 171.666 97.0198 171.842 87.0712 179.501 72.9794 182.649 62.7988 178.23 61.8354 184.513 60.3548 192.166 59.7472 199.685 59.5407 207.088 60.093 214.182 60.5666 220.787 16 31 97 177 97.0404 176.91 97.0107 176.922 96.9719 176.904 96.9658 176.898 96.9769 176.862 96.9682 176.991 90.8748 185.619 78.6208 190.771 71.3271 190.471 65.0811 184.156 63.6651 192.026 62.7183 199.231 62.5496 206.845 63.1681 213.915 63.8106 220.264 16 31 97 197 96.9914 197.027 96.9919 197.036 96.9598 197.058 96.9576 197.083 96.9587 197.081 96.9631 197.003 84.0376 214.159 75.4543 227.711 61.856 240.798 50.6027 249.162 42.9778 251.596 37.8783 253.842 41.4316 246.194 50.6652 233.022 62.0434 223.098 16 31 97 202 96.9706 201.947 96.9763 202.013 96.9357 201.927 96.9153 201.945 96.9267 202.007 96.8185 201.64 85.1087 213.543 75.0038 229.175 59.668 244.16 48.0162 252.594 40.2337 255.227 39.7246 248.559 42.7856 242.526 51.0438 231.758 62.1589 223.095 16 31 102 87 101.699 92.6844 102.021 99.4438 111.43 105.869 107.9 109.978 105.212 114.201 105.432 117.735 105.875 121.745 107.532 125.793 108.707 131.291 109.453 136.127 112.187 141.007 113.684 145.336 114.972 148.59 115.532 150.923 115.613 153.019 16 31 102 92 102.568 95.7778 103.759 101.856 109.959 108.595 106.636 112.786 104.719 116.903 104.77 120.389 105.16 124.306 106.84 128.291 107.989 133.97 106.74 138.88 109.815 144.025 111.953 148.194 113.719 151.115 114.463 153.287 114.642 155.545 16 31 102 147 100.755 148.185 99.5384 149.363 98.6584 151.663 98.3393 155.496 98.7244 159.862 97.1584 164.458 80.1231 168.892 63.5645 167.147 63.2188 173.431 63.0046 180.351 61.591 188.257 60.778 195.385 60.6146 202.698 61.2195 209.701 61.8753 215.622 16 31 102 157 102.018 156.994 102.011 157.042 101.898 157.533 101.073 159.731 101.676 163.838 97.5619 167.407 81.7536 172.261 66.4006 172.242 63.2112 176.175 62.5744 182.902 61.1159 190.731 60.4669 198.031 60.2456 205.334 60.8299 212.468 61.372 218.726 16 31 102 167 102.064 167.104 101.967 167.082 101.967 167.082 102.077 167.077 102.035 167.318 98.3267 170.2 86.3635 176.902 72.1061 180.374 63.2976 177.395 62.5099 183.964 61.0514 191.705 60.2881 199.077 60.0725 206.408 60.6327 213.54 61.1579 219.944 16 31 102 172 102.18 171.923 101.977 171.941 101.977 171.948 102.234 171.906 102.413 171.763 100.147 172.537 90.5396 181.575 75.0326 183.91 63.0667 178.549 61.9022 184.3 60.4216 191.982 59.8139 199.501 59.6075 206.904 60.1598 213.999 60.6334 220.603 16 31 107 107 106.439 108.896 106.655 109.58 107.124 112.121 104.12 114.775 104.051 118.672 104.095 122.041 104.55 125.794 105.94 129.659 107.202 135.244 104.939 140.23 107.832 145.718 110.037 149.918 111.794 152.697 112.56 154.691 112.781 156.994 16 31 107 147 106.037 148.566 104.999 150.021 104.509 152.118 103.487 155.716 103.983 160.843 99.6529 165.018 83.1889 169.872 67.5247 169.631 63.9167 173.171 63.6211 180.094 62.2261 188.284 61.4132 195.411 61.2498 202.724 61.8547 209.727 62.496 215.648 16 31 107 152 106.235 153.669 105.417 155.068 104.615 157.089 103.646 159.954 104.091 164.649 98.5307 168.157 84.8584 173.619 71.3822 176.898 64.171 175.793 63.4902 182.441 62.0093 190.311 61.1964 197.528 60.983 204.861 61.5673 211.951 62.1266 218.195 16 31 107 157 106.891 157.128 106.701 157.494 105.706 159.182 105.209 160.975 105.492 165.47 98.7508 169.112 86.3105 175.249 72.8343 179.074 63.4228 175.671 62.7859 182.398 61.3051 190.268 60.6936 197.531 60.4802 204.864 61.0645 211.998 61.6066 218.256 16 31 107 162 107.037 162.024 107.011 162.026 107.064 162.075 107.224 162.279 106.556 166.798 99.8768 170.86 89.5468 178.351 73.9283 182.845 62.5714 177.845 61.608 184.128 60.1274 191.811 59.5198 199.33 59.3042 206.66 59.8565 213.805 60.3301 220.41 16 31 112 87 112.012 91.9287 112.303 98.5206 114.414 105.625 111.846 109.341 110.326 112.981 110.385 116.554 111.256 120.309 110.358 125.258 111.881 130.3 112.322 135.712 112.589 140.935 113.556 145.375 114.844 148.629 115.404 150.962 115.486 153.17 16 31 112 92 111.969 94.5064 112.185 99.5495 114.09 106.598 111.229 110.405 108.39 114.527 108.463 118.008 108.741 122.043 109.512 126.409 110.911 131.605 111.611 136.613 111.878 141.79 113.408 146.09 114.964 149.258 115.525 151.591 115.634 153.669 16 31 112 97 111.549 98.9848 111.446 102.403 113.351 109.531 110.192 113.139 107.09 117.393 107.155 120.834 107.085 124.772 108.261 128.815 109.367 134.494 109.305 139.444 112.382 144.134 113.912 148.303 114.995 151.471 115.555 153.686 115.796 155.718 16 31 112 102 111.225 104.639 110.854 107.917 110.597 112.672 110.445 113.968 107.504 118.148 107.602 121.333 107.419 125.272 108.594 129.314 109.644 134.892 109.706 139.862 112.429 144.422 113.959 148.59 114.964 151.658 115.486 153.772 115.731 155.926 16 31 112 137 114.722 141.604 117.157 145.817 117.111 148.581 126.806 159.104 122.573 169.899 115.898 185.424 107.217 198.196 98.242 203.36 89.8672 204.738 83.9549 211.751 82.354 212.876 78.4588 220.792 77.5124 226.649 81.0382 227.398 83.5568 230.329 16 31 112 147 111.141 148.784 110.056 150.47 109.522 153.689 108.653 157.734 107.048 162.781 99.7975 166.753 86.0168 172.214 72.4675 174.199 66.0147 174.065 65.4407 180.731 64.051 189.144 63.0814 196.38 62.9349 203.713 63.5482 210.743 64.2105 216.897 16 31 112 152 111.254 153.613 110.52 155.446 109.003 157.68 108.422 160.769 107.15 165.82 100.342 169.806 89.3615 176.88 74.3139 181.74 63.2236 177.068 62.4359 183.637 60.9553 191.319 60.3063 198.619 60.0907 205.95 60.6509 213.082 61.1761 219.4 16 31 112 157 111.843 157.336 111.42 158.666 110.352 160.036 110.728 162.046 108.528 166.844 101.772 171.431 92.053 181.26 76.1882 183.674 63.9508 178.015 62.9077 184.049 61.4492 191.791 60.6859 199.162 60.4825 206.565 61.0348 213.66 61.56 220.063 16 31 112 162 111.972 162.026 111.913 162.003 111.934 162.193 112.372 163.162 108.83 166.606 102.075 171.193 92.3557 181.022 76.4908 183.437 64.262 177.7 63.2188 183.734 61.7604 191.476 60.9475 198.73 60.7441 206.133 61.3155 213.215 61.8407 219.532 16 31 112 167 112.007 167.009 111.947 166.979 111.948 166.977 112.005 167.082 107.491 168.244 101.263 172.588 91.7101 182.797 76.6794 185.133 64.442 179.48 63.1769 185.082 61.6964 192.783 60.7976 200.05 60.6054 207.545 61.1656 214.475 61.6907 220.879 16 31 112 172 112.231 172.057 112.018 172.037 111.987 172.035 111.769 171.948 108.556 175.39 101.896 187.675 91.4052 204.587 82.4175 211.654 73.1196 216.626 67.6783 213.993 64.0253 214.725 63.5004 214.058 63.6943 218.675 65.576 219.498 65.7318 222.74 16 31 117 82 116.707 88.3833 116.014 96.1711 121.945 103.602 122.689 106.625 119.458 110.31 122.905 113.739 124.662 117.683 124.802 122.566 124.878 127.505 127.24 132.818 126.12 136.689 126.758 139.892 127.331 142.705 127.757 144.935 128.114 146.863 16 31 117 87 116.495 92.5856 116.1 99.2771 120.733 106.638 121.034 109.636 118.584 112.951 122.286 116.351 124.5 120.309 124.776 125.096 124.895 129.963 127.085 135.195 125.755 139.098 126.449 142.301 127.226 145.069 127.363 147.32 127.628 149.224 16 31 117 97 115.922 100.983 115.521 104.89 114.157 110.91 111.05 113.864 108.109 117.783 108.207 120.968 108.024 124.907 108.905 129.024 109.954 134.602 110.016 139.572 112.74 144.131 113.842 148.345 114.924 151.513 115.446 153.627 115.691 155.782 16 31 117 167 117.142 166.875 116.933 166.71 116.934 166.709 116.265 167.346 107.642 167.637 100.988 172.441 91.306 181.574 75.799 183.91 63.5616 178.251 62.5185 184.285 61.06 192.026 60.2966 199.397 60.0811 206.728 60.6334 213.823 61.1586 220.227 16 31 122 82 121.867 88.5456 121.363 96.589 128.98 104.222 128.313 107.859 128.071 111.608 127.853 115.215 128.533 119.316 129.187 124.246 130.186 129.176 132.291 134.448 132.353 138.938 131.509 142.435 131.05 145.252 131.301 147.597 131.658 149.506 16 31 122 87 121.544 92.979 121.283 100.885 127.842 108.071 126.972 111.548 126.562 114.784 126.099 118.208 126.847 122.338 127.42 127.126 128.35 131.956 130.337 137.07 130.426 141.356 129.612 144.795 129.201 147.596 129.277 149.847 129.616 151.744 16 31 122 97 121.366 98.7301 121.248 102.567 124.125 108.812 123.326 111.307 120.704 114.67 124.247 117.831 125.358 121.832 125.673 126.516 126.463 131.278 128.633 136.452 128.469 140.678 127.751 143.871 128.048 146.594 128.042 148.846 128.359 150.749 16 31 127 82 126.816 88.6364 132.491 99.2071 132.216 103.109 131.872 106.849 131.283 110.338 131.082 113.968 131.561 118.047 132.479 123.082 133.675 128.151 135.046 133.476 135.071 138.075 134.182 141.349 133.656 144.113 133.869 146.46 134.377 148.535 16 31 127 87 126.567 93.4511 132.518 104.226 131.811 107.972 131.587 111.693 130.952 115.092 130.553 118.67 130.942 122.779 131.714 127.682 132.808 132.683 133.816 137.734 133.855 142.146 132.892 145.488 132.456 148.322 132.738 150.651 133.079 152.574 16 31 127 92 125.677 95.6703 129.126 103.576 129.686 107.711 129.472 111.382 129.134 115.003 128.746 118.58 129.208 122.726 129.887 127.561 130.928 132.501 132.052 137.569 132.154 141.935 131.282 145.37 130.855 148.182 131.11 150.486 131.447 152.391 16 31 127 97 126.139 97.6637 127.934 103.972 129.408 108.353 128.976 111.915 128.557 115.482 128.17 119.06 128.7 123.21 129.379 128.045 130.417 132.925 131.674 138.018 131.775 142.384 130.904 145.819 130.487 148.636 130.628 150.896 130.963 152.797 16 31 127 132 123.769 136.489 121.038 142.328 119.741 146.38 128.889 155.301 126.897 159.41 121.64 163.431 111.541 172.48 103.547 175.295 102.959 185.389 95.2728 194.945 92.7306 198.965 94.8123 200.871 95.3418 203.057 95.3487 203.064 95.3489 203.065 16 31 127 142 126.408 146.542 124.628 150.612 123.494 153.371 133.997 161.781 132.517 165.561 127.763 168.197 123.657 175.344 123.201 179.498 123.597 183.689 122.453 191.455 124.063 196.584 123.543 200.986 125.297 205.486 127.019 207.676 128.019 209.88 16 31 127 147 126.495 149.589 125.305 152.957 123.665 155.644 134.887 164.292 133.372 168.003 128.941 169.914 124.343 175.54 123.951 179.64 124.217 183.691 123.072 191.456 126.999 195.093 126.639 199.43 129.06 203.037 131.234 205 135.759 205.509 16 31 132 87 131.776 93.6164 134.701 100.862 133.709 104.423 133.324 108.122 132.732 111.539 132.376 115.103 132.848 119.196 133.766 124.254 134.979 129.542 136.079 134.903 136.08 139.518 135.166 142.583 134.682 145.321 134.938 147.648 135.435 149.765 16 31 132 92 131.524 98.4655 132.903 104.023 132.196 107.769 131.972 111.491 131.41 114.968 131.012 118.546 131.4 122.654 132.172 127.557 133.266 132.559 134.275 137.61 134.313 142.022 133.351 145.364 132.915 148.197 133.202 150.527 133.543 152.45 16 31 132 97 127.848 97.1507 130.891 104.354 130.561 108.272 130.347 111.993 129.843 115.542 129.383 119.067 129.844 123.213 130.601 128.076 131.671 133.054 132.724 138.109 132.804 142.521 131.855 145.891 131.427 148.731 131.683 151.016 132.02 152.923 16 31 132 122 129.213 129.292 126.255 138.012 122.052 150.022 131.34 159.388 129.55 163.506 124.234 167.086 114.917 177.058 104.017 180.99 98.2713 194.392 89.7465 205.93 86.6764 210.26 81.5126 220.378 80.0213 226.881 79.5798 231.582 85.0592 230.678 16 31 132 127 129.06 134.179 126.195 143.095 121.652 152.989 132.155 161.486 130.438 165.604 125.341 168.346 116.137 178.284 104.938 182.435 99.2087 195.361 90.2918 207.117 87.2847 211.357 82.1209 221.374 80.8805 227.474 80.2949 232.174 85.7742 231.27 16 31 132 132 128.921 138.231 125.402 143.564 123.194 147.237 132.336 155.489 130.558 159.589 125.563 163.298 121.17 170.889 120.74 175.12 120.489 179.477 120.201 186.794 119.913 193.007 119.084 197.551 121.582 201.27 123.23 203.497 123.929 204.76 16 31 132 137 129.196 142.243 124.452 146.87 123.261 149.707 132.549 158.449 130.964 162.316 125.823 166.077 119.167 175.955 116.789 180.653 117.147 184.605 112.96 192.632 106.789 200.275 103.236 204.351 105.291 208.718 109.701 211.224 111.698 213.593 16 31 132 142 130.533 146.188 125.927 150.188 124.287 152.889 134.79 161.299 133.315 165.068 128.485 167.99 124.379 175.137 123.923 179.291 123.515 184.065 122.371 191.831 123.601 197.294 123.081 201.696 124.835 206.196 126.498 208.324 127.627 210.429 16 31 137 92 132.642 93.3667 135.551 100.612 134.463 104.05 134.079 107.749 133.445 111.093 133.094 114.68 133.582 118.759 134.529 123.832 135.741 129.195 136.865 134.757 136.815 139.372 135.949 142.349 135.417 144.903 135.673 147.23 136.213 149.414 16 31 137 97 131.968 97.6221 133.347 103.18 132.651 106.964 132.379 110.677 131.818 114.154 131.428 117.732 131.878 121.846 132.71 126.808 133.846 131.865 134.854 136.944 134.873 141.412 133.914 144.655 133.478 147.488 133.762 149.825 134.171 151.783 16 31 137 122 134.3 129.269 131.074 138.438 127.236 149.899 135.733 158.563 134.279 162.344 130.081 165.779 129.819 170.908 129.373 175.741 130.003 181.249 135.81 188.287 138.35 192.658 138.482 196.561 140.137 199.824 141.306 201.79 141.762 204.295 16 31 137 127 134.296 134.189 131.154 143.902 126.381 154.985 136.915 163.76 135.46 167.223 131.391 169.695 130.104 174.656 129.64 179.487 130.336 184.933 135.75 191.368 137.986 195.676 138.854 199.076 140.114 202.479 140.723 204.411 140.554 206.662 16 31 137 132 133.909 137.986 129.938 144.049 124.952 155.16 135.863 163.807 134.39 167.333 129.959 169.646 127.493 174.669 126.537 179.475 126.129 184.959 127.424 191.768 130.252 194.864 131.149 198.714 134.682 201.808 136.405 203.691 136.591 206.002 16 31 137 137 134.369 142.464 130.299 147.966 126.522 153.502 136.192 162.468 134.717 166.117 130.648 168.835 129.995 173.858 129.503 178.709 130.199 184.156 135.915 190.924 138.151 195.232 138.975 198.753 140.234 202.157 140.844 204.089 140.674 206.339 16 31 137 142 135.333 145.831 132.078 148.036 128.817 153.936 136 162.913 134.511 166.488 130.442 169.207 129.108 174.252 128.334 179.108 128.448 184.588 132.313 191.053 132.423 194.921 133.481 198.636 137.356 201.816 138.536 203.686 138.021 205.939 16 31 142 122 138.938 129.492 135.889 138.529 136.379 150.516 137.179 156.333 135.932 160.125 133.229 163.698 133.231 168.695 133.141 173.603 134.093 179.057 139.421 185.821 141.835 190.364 141.864 197.208 143.426 200.665 144.345 203.199 145.343 206.348 16 31 142 127 139.048 134.43 136.142 144.109 136.397 150.291 137.407 156.108 136.16 159.9 133.457 163.473 133.514 168.451 133.606 173.36 134.474 178.557 139.802 185.321 142.012 189.965 142.041 196.809 143.602 200.265 144.935 202.922 145.949 206.112 16 31 142 132 139.15 138.18 136.211 143.265 136.946 151.349 137.207 156.79 135.96 160.581 133.158 164.154 133.159 169.151 133.07 174.06 134.022 179.513 139.5 186.278 141.826 190.777 142.122 196.985 143.684 200.441 145.016 203.098 146.03 206.288 16 31 142 137 138.733 141.992 136.343 147.302 135.522 152.224 136.09 158.041 134.637 161.832 130.912 165.275 130.792 170.291 130.57 175.145 131.522 180.643 137.303 187.596 139.844 191.983 139.975 197.379 141.508 200.734 142.323 203.028 142.851 205.753 16 31 147 107 146.277 112.9 145.774 118.895 145.264 125.078 144.866 131.751 144.435 138.293 144.458 144.895 144.74 151.559 145.463 158.061 145.509 166.176 143.758 176.06 141.062 182.06 139.712 192.364 141.198 195.619 142.854 197.746 143.882 200.903 16 31 147 132 145.564 136.776 144.49 139.73 143.98 142.351 144 142.579 143.528 144.972 143.42 150.047 143.61 156.437 144.179 162.939 144.247 170.834 142.119 179.962 139.176 186.29 138.514 194.37 140.019 197.523 141.565 199.59 142.555 202.667 16 31 152 112 152.644 116.113 153.073 120.322 152.799 126.498 152.572 132.723 152.316 138.752 152.293 145.324 152.718 152.082 153.638 158.536 154.395 166.121 155.595 174.699 154.924 181.741 155.355 188.253 158.452 190.26 160.366 192.27 161.283 195.822 16 31 152 117 151.392 119.118 150.919 124.455 150.429 130.75 149.743 137.001 149.317 143.356 149.32 149.912 149.669 156.64 150.31 163.237 150.542 171.15 149.539 180.646 150.053 186.437 147.389 195.978 148.852 199.453 149.642 202.308 150.711 205.495 16 31 152 127 151.923 127.315 151.887 127.469 151.283 131.022 150.628 136.897 150.205 143.041 150.231 149.535 150.499 156.234 151.222 162.867 151.474 170.688 150.471 180.184 150.858 186.159 148.482 195.346 149.919 198.828 150.693 201.686 151.791 204.869 16 31 182 137 180.194 142.806 180.691 149.521 181.248 155.855 183.126 161.593 190.64 167.523 211.989 181.312 220.619 194.644 218.887 200.754 218.193 201.573 217.784 200.481 215.919 198.829 214.28 198.869 211.955 199.395 212.349 200.542 210.573 200.959 16 31 192 137 192.45 137.22 196.749 138.17 200.378 144.938 197.104 156.201 193.84 164.685 212.086 177.825 219.706 190.541 219.56 197.901 218.842 198.707 218.27 197.495 217.479 196.297 214.414 195.942 212.646 196.55 212.28 197.607 211.13 198.051 16 31 212 127 211.069 139.464 206.221 153.253 203.591 163.418 207.101 167.851 212.501 169.766 225.579 182.491 224.29 193.95 222.28 200.094 222.943 201.255 223.226 200.184 221.937 198.545 218.756 197.705 217.627 198.419 215.759 199.288 213.527 199.56 16 31 217 117 221.806 124.682 225.681 136.068 225.808 152.793 224.727 169.373 222.87 182.093 225.701 184.915 224.322 196.295 222.273 202.39 223.181 203.551 223.818 202.516 222.959 201.107 220.891 200.397 219.706 201.125 218.379 201.93 217.222 202.286 16 31 24.0416 108.894 22.9057 112.06 21.89 115.462 20.6842 119.655 19.3916 124.273 18.4522 128.624 17.962 133.378 17.9712 138.346 18.1746 144.621 19.7564 149.575 21.6441 155.781 23.0807 163.058 24.0267 170.974 25.2492 178.042 26.129 184.243 26.8444 189.86 16 31 24.0416 115.966 22.4833 119.078 21.2711 122.417 19.9395 126.773 18.6316 130.915 17.6552 135.047 17.1762 139.022 17.1353 143.901 17.4746 149.226 18.7954 154.588 20.7421 159.708 22.1922 166.882 23.6447 174.378 24.6685 181.792 25.0614 188.593 25.1011 195.996 16 31 24.0416 158.392 23.9745 158.441 24.0317 158.478 23.9594 158.486 24.102 158.486 23.7851 159.508 23.5648 164.06 23.7577 169.683 24.2268 175.056 24.9844 179.873 26.9369 185.786 28.2725 192.31 29.3876 199.01 30.2012 206.096 29.9607 213.536 30.0023 222.532 16 31 24.0416 165.463 23.9389 165.675 23.9674 165.689 23.8921 165.716 23.9817 165.555 23.922 165.288 24.0034 165.714 24.1962 171.26 24.5169 176.175 25.2419 179.433 27.1944 185.345 28.53 191.869 29.6451 198.569 30.3681 205.748 30.5161 213.591 30.5756 222.65 16 31 31.1127 94.7523 30.2072 98.4792 30.399 100.947 31.3632 103.822 32.092 107.089 31.8915 111.084 31.973 115.746 32.071 120.735 32.3029 126.168 33.0448 131.364 33.8022 136.205 35.1923 137.991 35.3841 138.238 35.4825 138.364 35.8058 138.278 35.8318 138.304 16 31 31.1127 101.823 29.6674 105.988 29.3494 109.001 28.483 113.293 27.222 117.464 26.7486 121.647 26.4132 126.622 26.2644 132.103 26.8045 137.914 27.7273 144.01 29.6481 150.569 31.3794 157.458 32.7236 164.975 33.5098 172.519 33.7065 180.363 33.7506 187.662 16 31 31.1127 108.894 29.8051 113.008 28.6215 117.481 27.3466 121.795 26.0113 126.347 25.0012 131.204 24.548 136.326 24.5211 141.496 24.8533 147.781 26.3531 153.234 28.1686 159.724 29.628 166.945 30.9956 174.279 31.9446 181.812 32.1166 189.791 32.0964 197.556 16 31 31.1127 115.966 29.6015 119.997 28.3078 124.124 26.9343 128.934 25.6405 133.102 24.6067 137.667 24.2308 142.885 24.1693 148.444 24.5733 153.587 25.8545 160.316 27.8294 166.442 29.3351 172.628 30.7701 179.57 31.6197 187.565 31.7507 195.613 31.7283 203.494 16 31 31.1127 123.037 29.583 127.085 28.2636 131.413 26.7641 135.785 25.6546 140.625 24.5812 145.147 23.9053 149.951 23.8432 155.672 24.3567 161.158 25.5413 167.354 27.468 173.676 29.1298 179.97 30.5467 187.313 31.1915 194.571 31.2701 202.355 31.3869 211.652 16 31 31.1127 130.108 29.6256 134.389 28.3148 138.789 26.8022 143.045 25.4922 147.569 24.5249 152.418 23.8664 157.207 23.6423 162.573 24.2363 168.161 25.2737 174.215 27.0444 180.389 29.0179 187.329 29.9794 193.937 30.5952 201.21 30.7985 209.159 30.858 218.24 16 31 31.1127 137.179 29.3091 140.387 27.825 144.427 25.6757 148.026 23.573 151.68 22.5432 156.365 22.1085 161.143 22.1809 166.674 22.8259 172.242 23.5613 177.852 25.1892 183.712 26.8428 190.339 27.3852 196.542 28.0944 203.834 28.2424 211.692 28.2034 220.514 16 31 31.1127 144.25 31.1634 144.319 29.7555 145.893 27.6629 149.315 26.0066 153.516 24.8621 157.808 24.4492 162.519 24.626 168.14 25.1599 173.666 26.1866 179.608 28.3657 185.927 29.7064 192.544 31.0734 199.579 31.7964 206.671 31.6202 214.217 31.6797 223.276 16 31 31.1127 151.321 31.4281 151.416 31.2795 151.37 31.0586 152.033 30.2318 155.723 28.8686 160.331 28.4557 164.952 28.4544 170.415 28.7953 175.886 30.014 182.244 32.1431 188.562 33.3178 195.245 35.0077 202.533 35.6684 209.463 34.9989 216.865 34.5683 225.646 16 31 31.1127 165.463 31.0855 165.141 31.1185 165.153 31.1409 165.269 31.1695 165.117 31.0955 165.284 30.9993 166.693 30.6656 171.958 31.0065 177.238 32.2716 183.558 33.2011 189.758 34.1442 196.288 35.834 203.108 36.4538 209.657 36.1324 217.205 35.27 225.961 16 31 38.1838 94.7523 37.0524 98.8811 35.8875 103.021 34.6098 107.542 33.6076 111.952 32.5277 116.524 32.2766 121.499 32.2161 126.714 32.664 132.739 33.4675 138.915 35.7018 145.18 37.9004 151.912 40.3335 159.161 40.7779 166.878 40.8041 175.011 40.7904 182.709 16 31 38.1838 101.823 36.9983 105.975 35.7928 110.093 34.5076 114.563 33.1728 118.928 32.4269 123.39 32.031 128.393 32.0165 133.702 32.4637 139.717 33.4104 145.907 35.3593 152.287 37.1857 159.038 38.4458 166.497 39.0905 174.2 39.1588 182.317 38.9566 190.013 16 31 38.1838 123.037 36.8056 127.063 35.6733 131.356 34.2501 135.507 33.0458 139.957 32.1716 144.477 31.7874 149.037 31.6885 154.177 32.2708 159.491 33.259 165.126 34.8874 170.364 35.7829 172.77 36.5802 176.9 37.4654 182.704 37.6496 190.082 37.8167 196.864 16 31 38.1838 130.108 36.434 134.024 35.3934 138.55 34.0721 142.946 32.7318 147.392 31.5636 152.125 30.913 156.783 30.6056 162.108 31.183 167.765 32.4105 174.151 34.4823 180.583 36.2507 187.559 37.7297 194.721 38.7246 201.868 38.8103 209.749 38.418 218.687 16 31 38.1838 137.179 36.8528 140.922 35.1767 145.488 33.689 149.6 32.4937 154.035 30.9408 158.67 30.3442 163.305 30.0105 168.704 30.5001 174.348 31.812 180.87 33.9774 187.353 35.5127 194.357 37.1301 201.694 37.7909 208.635 37.4695 216.259 36.6949 225.032 16 31 38.1838 144.25 38.1592 144.522 36.4762 148.649 35.0963 152.015 33.9461 156.459 33.0183 160.903 32.598 165.362 31.9321 170.74 32.3427 176.38 33.6314 182.7 35.3697 189.184 36.5414 196.046 37.4544 202.891 38.1152 209.832 37.7938 217.456 36.6993 226.213 16 31 38.1838 151.321 38.1754 151.576 38.1662 151.426 37.7072 152.557 37.0601 156.985 36.9288 161.333 36.5933 165.797 35.9645 171.197 36.2649 176.836 37.541 182.977 39.3595 189.513 40.444 196.508 38.6777 203.343 39.0324 210.093 37.7702 217.694 36.6757 226.451 16 31 38.1838 158.392 38.133 158.486 38.1362 158.392 38.0438 158.473 37.9386 158.728 38.1382 162.061 37.9804 166.075 37.5978 171.388 38.0514 176.878 39.4731 182.96 41.3387 189.49 42.2919 196.235 40.4202 203.069 40.7748 209.938 40.1258 217.664 39.0252 226.437 16 31 38.1838 165.463 38.1638 165.42 38.1787 165.362 38.1702 165.457 38.1564 165.366 38.158 165.381 38.0972 166.541 37.5681 171.593 38.0217 177.083 39.4434 183.165 40.9902 189.447 41.9434 196.192 40.0717 203.027 40.4263 209.835 39.7773 217.561 38.6767 226.334 16 31 45.2548 94.7523 43.8732 98.8211 42.5994 103.001 41.2352 107.563 40.1691 112.067 39.1905 116.692 38.672 121.709 38.5227 127.054 39.1942 133.102 40.0244 139.401 42.3345 145.603 44.8218 152.109 46.9291 159.258 48.095 166.756 48.8108 174.559 49.8425 182.169 16 31 45.2548 101.823 43.9876 105.949 42.7722 110.107 41.5008 114.597 40.3116 118.961 39.4703 123.538 38.8387 128.568 38.8375 133.858 39.2996 139.74 40.1172 145.969 41.9931 152.163 44.3693 158.637 45.9767 165.707 46.8719 173.207 47.4597 181.161 47.4778 188.762 16 31 45.2548 123.037 43.9573 127.062 42.6284 131.378 41.2813 135.789 40.0462 140.2 39.192 144.869 38.7613 149.8 38.9564 154.701 39.4871 160.586 40.1961 166.771 41.7778 173.191 42.904 180.132 43.8737 186.749 44.5143 193.781 44.9769 200.388 44.9412 206.079 16 31 45.2548 130.108 43.983 134.056 42.4837 138.571 41.058 143.032 39.7679 147.551 38.9711 152.132 38.6082 156.779 38.5524 162.037 39.1495 167.737 39.8654 174.141 42.0505 180.573 43.8502 187.625 44.7607 194.74 46.3674 201.952 46.9076 209.911 46.493 218.784 16 31 45.2548 137.179 44.3118 141.052 43.2073 145.584 42.2966 149.555 40.898 154.051 40.2339 158.744 39.9367 163.333 39.8334 168.64 40.4288 174.296 41.328 180.737 43.469 187.267 45.4529 194.194 45.8767 201.115 46.8458 208.389 46.8326 216.277 46.6924 224.927 16 31 45.2548 144.25 45.275 144.481 45.7019 148.007 45.4686 150.709 44.0145 154.691 43.1626 159.397 43.1994 163.495 43.4642 168.751 44.3733 174.138 45.664 180.42 47.9013 186.646 49.7044 193.257 48.9316 200.473 48.8828 208.014 49.9404 215.804 50.1157 224.227 16 31 45.2548 151.321 45.2438 151.61 45.3155 151.547 45.2646 152.315 44.2287 155.608 43.5033 160.23 43.5401 164.327 43.805 169.583 45.187 174.698 46.7629 180.622 49.0125 186.481 50.4252 193.19 49.6524 200.644 49.6035 208.184 50.6611 215.974 50.8364 224.397 16 31 45.2548 158.392 45.1976 158.53 45.1568 158.589 45.0054 158.1 44.8742 158.235 44.3019 160.337 44.3387 164.434 44.6036 169.69 46.01 174.674 47.8603 180.341 50.1435 185.809 51.5562 192.619 51.8905 200.176 51.7537 207.624 52.8113 215.186 52.9462 223.729 16 31 45.2548 165.463 45.2631 165.508 45.2904 165.549 45.1408 165.544 45.1079 165.627 45.11 165.724 45.3981 167.584 46.2576 171.977 48.4272 176.001 50.4002 181.591 52.6834 186.738 53.5541 193.549 52.7813 201.307 52.6445 208.755 53.2959 216.024 53.9592 224.448 16 31 52.3259 94.7523 51.0399 98.8942 49.7385 103.106 48.2963 107.625 47.3271 112.183 46.6833 116.835 46.2415 121.96 46.0651 127.225 46.9229 133.24 48.5156 139.779 49.803 146.083 51.669 152.204 53.7801 159.147 54.6928 166.499 53.7388 174.157 52.3678 181.469 16 31 52.3259 101.823 51.1755 106.034 50.075 110.24 48.7057 114.75 47.7262 119.248 46.8248 123.743 46.3152 128.761 46.3112 133.975 46.9871 139.646 48.2414 146.048 49.6059 152.296 51.3589 158.593 53.036 165.595 53.4767 172.624 52.5814 180.149 52.916 187.418 16 31 52.3259 115.966 50.9811 120.367 49.7003 124.825 48.5547 129.566 47.3529 134.144 46.3928 138.868 46.0271 144.464 46.3079 149.746 46.8769 155.435 47.9672 161.736 49.5802 168.72 50.6044 176.831 51.7185 184.741 52.2188 193.07 52.8415 201.404 53.1362 210.119 16 31 52.3259 123.037 51.0215 127.236 49.8055 131.482 48.4546 136.009 47.2024 140.411 46.1551 145.453 45.6654 150.537 45.9231 155.614 46.5327 161.567 47.5564 167.801 49.1183 174.689 50.1122 182.053 51.1191 189.427 51.6772 197.48 52.4495 205.726 52.8835 214.452 16 31 52.3259 130.108 51.0379 134.281 49.7563 138.605 48.1004 143.395 46.9356 147.699 45.6593 152.954 44.9138 158.078 44.7911 163.308 45.1492 169.264 46.1046 175.713 47.8558 182.682 49.2685 189.809 49.9294 197.319 50.4594 205.016 51.3648 213.025 51.3747 221.835 16 31 52.3259 137.179 51.1331 140.735 50.7996 144.968 49.7404 148.32 48.3087 152.394 46.7677 157.612 46.3172 162.141 46.3478 167.496 46.8071 172.943 48.1634 179.082 50.4006 185.288 51.653 192.177 51.7505 199.936 51.7016 207.556 52.7592 215.118 52.8941 223.661 16 31 52.3259 144.25 52.1786 144.076 52.8412 146.351 51.7386 149.829 50.3526 153.379 48.8116 158.395 48.2889 162.331 48.3849 167.785 49.017 172.999 50.4393 178.888 52.41 185.309 53.5085 192.349 53.3686 200.186 53.2521 207.634 53.9035 215.16 54.0981 223.624 16 31 52.3259 151.321 52.2056 151.303 52.3374 151.398 51.7884 152.371 51.1825 154.602 49.7224 159.367 49.3871 163.024 49.5721 168.542 50.031 173.857 51.7471 179.524 53.8897 185.536 54.7006 192.677 54.4177 200.632 54.3012 208.079 54.9527 215.606 55.6159 223.837 16 31 52.3259 158.392 52.1742 158.404 52.0351 158.504 51.856 158.264 51.8313 158.425 51.8063 160.401 51.4083 163.98 51.8943 169.324 52.3829 174.252 53.8052 179.905 55.6412 185.978 55.8518 193.119 55.079 200.941 54.9677 208.389 55.5088 215.676 56.4072 223.906 16 31 52.3259 165.463 52.3423 165.544 52.3506 165.776 52.2885 165.357 52.2749 165.558 52.2927 165.586 52.6984 165.932 53.1844 170.065 53.4529 175.11 54.8752 180.657 56.4593 186.669 55.6979 194.11 54.9251 201.932 54.8138 209.383 55.2861 216.301 56.1845 224.531 16 31 59.397 87.6812 58.6182 91.8309 57.5335 96.2026 56.7133 100.681 56.5535 105.05 56.4363 109.581 56.6547 114.651 56.1697 118.693 57.0926 121.913 59.5564 124.285 61.0672 127.815 61.1752 128.515 61.1664 128.538 61.1921 128.595 61.1777 128.605 61.1874 128.594 16 31 59.397 94.7523 58.4642 98.9522 57.4235 103.439 56.4756 107.991 56.0997 112.573 55.7383 117.132 55.6106 122.361 54.7682 127.587 54.8158 133.096 56.0842 139.633 58.3225 146.131 60.2846 152.246 61.6221 158.803 61.745 165.609 60.9766 172.901 60.2658 179.817 16 31 59.397 101.823 58.3655 106.03 57.3161 110.42 56.1952 114.918 55.5293 119.473 54.7537 123.925 54.5347 128.92 53.9885 134.097 54.4721 139.791 55.8072 146.257 57.9021 152.756 59.7913 159.085 60.9665 165.824 61.2441 172.59 60.5177 179.816 59.4817 186.659 16 31 59.397 108.894 58.1094 113.322 56.9835 117.823 55.8574 122.463 54.7583 127.299 53.9436 132.016 53.8943 137.526 53.8577 142.732 54.3529 148.446 55.4497 154.633 56.9679 161.53 58.4715 169.2 59.1421 177.412 60.0322 185.245 60.383 193.77 59.9963 201.879 16 31 59.397 115.966 57.8668 120.488 56.6467 125.112 55.6012 129.869 54.2381 134.866 53.438 139.716 53.0467 145.373 52.9472 150.415 53.4015 156.036 54.1458 162.191 55.461 168.918 56.4304 177.005 57.3204 185.2 57.8667 193.714 58.5997 202.258 59.2677 211.068 16 31 59.397 123.037 58.0482 127.531 56.8224 131.975 55.3585 136.568 53.8096 141.02 52.4572 145.894 51.721 151.316 51.3669 156.469 51.7255 162.264 52.2489 168.482 53.4949 175.277 54.3165 182.725 55.2002 190.302 55.796 198.202 56.717 206.446 57.7313 214.489 16 31 59.397 130.108 58.4777 133.847 57.1871 137.58 55.5502 142.157 54.0386 146.867 52.392 151.995 51.5448 157.231 51.3875 162.634 51.4558 168.656 52.3471 175.077 53.8948 182.034 54.6477 189.466 55.0369 197.378 54.9523 204.829 55.7642 212.667 56.3975 221.006 16 31 59.397 137.179 58.8041 140.328 58.048 143.345 57.2023 146.742 55.9488 151.451 54.6698 156.025 54.0228 160.597 53.6376 166.221 53.5864 172.014 54.5365 178.208 55.8677 184.969 55.2337 192.467 54.9509 200.421 54.8396 207.868 55.3807 215.156 56.044 223.386 16 31 59.397 151.321 59.3068 151.226 59.3048 151.179 59.4825 151.535 59.1158 153.318 57.9349 157.568 57.3782 161.92 56.7667 167.104 56.5764 172.674 57.117 179.023 58.3771 185.703 57.195 193.249 56.413 201.071 56.301 208.522 56.8308 215.7 57.9369 223.922 16 31 59.397 165.463 59.4036 165.619 59.3554 165.913 59.3699 165.21 59.3577 165.172 59.3417 165.122 59.6927 165.255 59.146 168.53 58.6311 173.574 59.1557 179.71 58.2785 186.273 56.9794 193.82 56.1974 201.642 56.0869 209.187 56.5766 215.923 57.6828 224.145 16 31 66.468 87.6812 66.1503 91.7696 64.8695 96.1088 64.211 100.468 64.2458 104.785 64.1317 109.793 65.7026 114.658 69.2841 117.481 72.494 119.928 76.0121 122.569 81.9324 126.169 81.8786 131.48 81.8244 136.442 82.3301 141.07 82.3518 144.464 82.145 147.286 16 31 66.468 94.7523 65.827 98.8689 64.6749 103.407 63.913 107.917 63.8996 112.194 63.7614 116.996 64.6917 121.738 66.2626 125.003 68.6182 127.676 73.3821 130.044 80.4961 133.363 82.0324 138.115 82.753 142.755 83.2432 146.955 83.6906 150.276 84.02 152.865 16 31 66.468 101.823 65.5425 105.938 64.4348 110.377 63.5539 114.848 63.003 119.17 62.3401 123.741 62.2709 128.735 61.9483 133.485 62.5744 138.436 63.6071 144.609 65.6886 150.825 67.436 157.435 68.4075 164.399 68.6966 171.215 68.5349 178.075 67.9186 184.409 16 31 66.468 108.894 64.9758 113.563 63.4622 118.794 61.7099 124.338 60.3324 129.827 59.0528 135.518 57.8158 140.959 57.557 146.227 57.8543 151.895 58.4188 158.023 59.6631 164.868 60.8437 172.873 61.4717 181.048 62.3229 189.349 62.7027 197.793 62.618 206.338 16 31 66.468 115.966 64.847 120.92 63.2336 126.296 61.2813 132.063 59.6727 137.66 57.8738 143.611 56.7572 149.178 55.8942 154.441 55.5291 159.793 55.7404 165.943 56.8717 172.653 57.7305 180.507 58.5678 188.351 59.1142 196.526 60.0163 204.406 61.0125 211.933 16 31 66.468 123.037 64.8544 127.74 63.189 132.714 61.2643 137.109 59.6092 141.709 57.8567 146.764 56.8167 151.658 55.9537 156.998 55.6255 162.313 55.6671 168.494 56.6932 175.281 57.4743 183.001 58.2663 190.749 58.8421 198.418 59.7907 206.109 60.8554 213.034 16 31 66.468 130.108 64.9246 133.989 63.4333 137.867 61.6341 141.474 60.0967 145.668 58.4172 149.937 57.4665 154.224 56.5774 159.342 56.269 164.625 56.1687 170.827 56.9741 177.673 57.6618 185.295 58.2524 193 58.719 200.512 59.6334 207.717 60.6477 214.34 16 31 66.468 137.179 66.7816 140.807 66.1403 144.39 64.5791 147.62 63.4471 151.705 61.5574 155.978 60.506 160.288 59.446 165.05 59.0794 170.405 58.9259 176.645 58.4499 183.355 57.4039 190.869 56.9115 198.691 56.8017 206.106 57.3466 213.493 58.1646 221.074 16 31 66.468 144.25 66.3287 144.608 65.6115 147.418 64.3739 149.91 63.056 153.833 61.5003 157.865 61.4274 162.197 60.6019 166.775 59.5917 171.818 59.4814 178.117 58.8081 184.781 57.3784 192.429 56.6868 200.251 56.5748 207.702 57.1045 214.88 57.8988 222.978 16 31 66.468 151.321 66.41 151.311 66.3902 151.412 65.8786 151.858 64.231 155.069 63.04 158.924 62.8652 163.221 61.6571 167.787 59.4466 172.313 59.3398 178.483 58.6664 185.147 57.4843 192.693 56.7927 200.515 56.6807 207.966 57.2104 215.144 58.0047 223.242 16 31 66.468 158.392 66.4473 158.378 66.397 158.363 66.4623 158.396 66.1458 158.726 65.2823 161.818 65.8227 165.555 64.1598 169.485 61.0802 173.706 60.1083 179.265 58.9865 185.628 57.5568 193.276 56.7128 201.035 56.6008 208.486 57.1305 215.664 57.9248 223.762 16 31 73.5391 87.6812 73.0538 91.7092 72.4856 96.7326 72.1056 100.995 70.5434 104.727 69.0926 108.653 68.1905 112.171 71.2127 114.657 73.7283 116.897 76.0439 119.447 76.9884 123.673 77.1196 128.025 77.1351 132.316 77.7613 135.174 77.8221 135.956 77.9203 136.151 16 31 73.5391 94.7523 72.7487 99.0778 72.1772 104.092 71.3679 108.566 69.8048 112.526 67.9095 116.403 67.7321 120.035 71.153 122.725 73.7233 125.351 77.6232 127.87 86.4557 130.831 87.6178 135.377 87.7851 140.107 88.0043 144.308 87.7382 147.779 87.611 150.292 16 31 73.5391 115.966 72.0314 121.008 70.3586 126.452 68.5095 132.753 67.0276 138.106 65.1376 144.138 63.6304 149.315 62.3439 154.387 61.4946 159.818 61.0328 165.943 61.5653 172.616 62.215 180.455 62.6464 188.059 63.2124 195.764 63.9678 203.484 64.8176 211.144 16 31 73.5391 123.037 71.9809 127.358 70.3163 131.766 68.4115 135.607 66.8948 139.777 64.9994 144.928 63.4922 150.105 62.2056 155.176 61.3215 160.532 60.7564 166.671 61.1468 173.495 61.8328 181.35 62.2642 188.954 62.8049 196.389 63.6632 203.801 64.5131 211.461 16 31 73.5391 137.179 74.1853 140.859 73.5082 144.625 72.2804 148.408 70.737 152.275 68.1803 156.166 66.7302 160.112 64.9687 164.66 60.6647 168.981 60.1119 175.164 59.5563 181.869 58.2708 189.45 57.988 197.203 57.8783 204.554 58.5228 211.785 59.3514 218.86 16 31 73.5391 158.392 73.5377 158.439 73.5899 158.473 73.5929 158.458 73.625 158.973 72.9329 162.383 71.7821 165.691 68.338 169.572 63.3253 173.25 60.5262 177.878 59.6359 184.489 58.2715 192.069 57.5799 199.892 57.4634 207.312 58.0068 214.269 58.8354 222.23 16 31 73.5391 165.463 73.5492 165.477 73.579 165.537 73.5831 165.55 73.5698 165.586 73.5283 165.707 72.9916 167.203 70.4417 171.145 64.2666 174.199 61.4675 178.828 60.5773 185.438 59.07 193.127 58.1267 200.78 58.0128 208.289 58.5562 215.246 59.3848 223.206 16 31 80.6102 87.6812 80.4447 91.945 80.3002 97.3883 80.7208 102.478 80.7691 106.145 80.3697 109.558 80.1318 112.839 80.1776 115.719 80.4125 118.534 80.7663 121.056 80.8521 125.197 80.7349 130.508 80.6757 135.333 81.3019 139.928 81.3746 143.222 81.1774 145.957 16 31 80.6102 94.7523 80.3242 99.2007 80.5866 104.472 81.9112 109.226 82.0846 113.022 82.1584 116.709 82.3339 120.217 82.1119 123.131 82.822 126.006 82.4737 128.447 86.6249 131.498 87.7869 136.044 87.9542 140.773 88.2373 145.017 87.9674 148.548 87.9681 151.087 16 31 80.6102 108.894 80.0304 110.831 79.6131 113.55 79.2802 116.017 78.5129 119.084 78.5538 122.322 78.7293 125.733 79.8875 128.584 80.2753 130.826 79.8291 132.865 85.143 135.888 87.401 140.35 87.5858 144.823 87.8697 149.032 87.5537 152.581 87.6925 155.093 16 31 80.6102 137.179 81.413 140.73 80.5954 144.653 79.5136 148.938 78.9123 153.06 77.403 157.808 76.0666 162.534 71.9828 167.257 64.5052 169.53 62.0091 174.7 61.3146 181.364 59.9184 189.053 59.3917 196.651 59.2662 203.987 59.9716 211.124 60.8339 217.832 16 31 80.6102 151.321 80.5407 151.444 80.4779 151.813 80.1834 155.421 80.3473 159.702 79.9459 164.605 79.8088 170.168 77.0539 175.186 65.7198 175.737 62.0039 179.588 60.6465 185.605 59.1393 193.295 58.196 200.947 58.082 208.456 58.6255 215.413 59.4541 223.374 16 31 80.6102 158.392 80.5584 158.934 80.5858 158.829 80.4053 159.269 80.4098 160.861 80.0475 165.193 79.9402 170.567 76.6084 175.408 65.2743 175.959 61.5584 179.81 60.201 185.827 58.6938 193.517 57.7505 201.169 57.6365 208.678 58.1684 215.136 58.997 223.097 16 31 80.6102 165.463 80.6726 165.424 80.7112 164.947 80.6489 165.248 80.5933 164.891 80.5692 165.244 80.4619 170.619 77.13 175.459 64.8902 175.818 61.1742 179.669 60.0525 186.032 58.5801 193.613 57.736 201.372 57.6221 208.881 58.154 215.339 58.9826 223.3 16 31 80.6102 172.534 80.6716 172.525 80.6842 172.117 80.6458 172.649 80.5788 172.249 80.551 171.783 80.652 173.101 77.8971 177.631 68.213 180.11 61.5421 180.392 59.8905 185.644 58.4608 193.292 57.6168 201.051 57.5028 208.56 58.0463 215.517 58.8749 223.478 16 31 80.6102 179.605 80.6273 179.593 80.5778 179.867 80.6187 179.892 80.5411 179.74 80.528 179.605 80.5805 179.672 78.9866 172.049 63.7308 171.485 61.5991 176.939 60.7088 183.524 59.2158 191.196 58.5242 198.848 58.4077 206.269 58.9587 213.447 59.8209 220.523 16 31 87.6812 80.6102 87.1838 85.8018 86.7517 92.2326 86.6136 98.0687 86.4994 101.807 86.2574 105.148 86.1235 108.462 86.0571 111.272 86.0467 114.103 85.8923 116.274 86.0653 118.318 85.9013 122.573 85.9396 123.318 85.8771 123.442 85.9016 123.281 85.8931 123.476 16 31 87.6812 87.6812 87.1189 92.9781 86.7539 99.2475 86.5775 105.147 86.3854 109.012 86.1539 112.631 86.0897 116.256 85.6885 119.395 85.8003 122.324 85.452 124.802 85.6564 128.055 86.1587 132.762 86.2796 137.699 86.467 142.132 86.2176 145.537 85.9902 148.129 16 31 87.6812 94.7523 86.6671 100.055 86.5328 105.573 86.4847 111.045 86.5312 114.929 86.3619 118.84 86.1262 122.52 85.0824 125.637 85.626 128.464 84.5007 130.983 86.6076 133.985 88.8656 138.495 89.2561 142.97 89.5399 147.121 89.2066 150.67 89.2073 153.16 16 31 87.6812 108.894 87.5073 109.347 87.32 110.419 87.1286 112.593 87.1684 116.478 87.039 120.467 86.7486 124.147 85.7047 127.152 86.2148 129.835 84.9641 132.262 86.9466 135.251 89.3055 139.739 89.696 144.191 89.9791 148.336 89.1742 151.844 89.175 154.335 16 31 87.6812 137.179 87.4932 139.465 87.0383 142.839 86.3756 147.506 86.2934 152.173 86.0079 157.674 85.8632 163.835 76.8518 167.839 62.4834 166.751 61.8165 172.907 61.4041 179.723 60.2536 187.485 59.6921 194.923 59.6212 202.255 60.4299 209.393 61.3325 215.844 16 31 87.6812 144.25 87.6162 145.815 87.4367 148.585 86.9724 153.583 86.8493 158.305 86.5356 163.853 86.6861 169.326 76.9984 174.359 64.5015 174.328 61.1395 178.677 60.2492 185.288 58.742 192.977 57.96 200.703 57.846 208.212 58.3895 215.169 59.2181 223.13 16 31 87.6812 151.321 87.7268 151.439 87.685 151.348 87.2732 156.125 87.1171 160.846 86.9144 164.939 87.0649 170.293 76.9044 175.365 65.5704 175.916 61.8544 179.767 60.4971 185.785 58.9898 193.474 58.0465 201.127 57.9326 208.635 58.4645 215.094 59.2931 223.054 16 31 87.6812 158.392 87.7519 158.573 87.7251 158.502 87.6057 158.408 87.485 161.284 87.2823 165.377 87.4832 170.619 77.3227 175.838 65.0829 176.197 61.2493 179.618 60.1275 185.981 58.6078 193.636 57.7638 201.396 57.6498 208.904 58.1817 215.362 59.0103 223.323 16 31 87.6812 165.463 87.7947 165.73 87.8093 165.18 87.7343 165.851 87.7485 165.347 87.8134 166.684 88.0014 171.787 77.8409 177.005 66.5233 178.189 61.5576 180.391 59.906 185.643 58.4763 193.291 57.6322 201.05 57.5183 208.559 58.0617 215.516 58.8904 223.477 16 31 87.6812 172.534 87.7276 172.362 87.7294 171.853 87.7125 172.399 87.7192 171.94 87.6952 171.238 87.8198 173.662 78.1321 178.667 68.448 181.146 61.6417 180.381 59.9901 185.632 58.5604 193.28 57.7164 201.04 57.6024 208.548 58.1459 215.505 58.9745 223.466 16 31 87.6812 179.605 87.6874 179.593 87.6357 180.103 87.6485 180.06 87.6121 180.053 87.6176 179.974 87.6502 179.829 81.3223 165.092 65.8711 164.805 63.6651 170.203 63.4124 177.068 62.4464 184.983 62.184 192.421 62.3629 199.774 63.1809 206.827 64.1519 213.478 16 31 87.6812 186.676 87.6901 186.58 87.6355 187.09 87.6399 187.088 87.6247 187.176 87.6554 187.039 87.701 186.716 83.6718 182.975 72.3924 183.809 61.7765 179.667 60.4192 185.685 58.9119 193.374 57.9686 201.027 57.8546 208.535 58.3981 215.492 59.2267 223.453 16 31 94.7523 80.6102 94.7687 86.2906 94.568 93.0005 94.3135 99.4028 94.0301 103.636 93.8639 107.071 93.8596 110.668 93.7948 114.265 93.8347 117.211 93.5435 120.607 93.8036 122.902 93.7136 126.173 93.7961 130.217 93.7702 133.751 93.6149 136.748 93.473 138.523 16 31 94.7523 87.6812 94.7902 93.4057 94.9648 100.143 94.6446 106.694 93.7925 111.036 93.8738 114.996 93.7784 118.824 93.8024 122.477 95.1984 125.76 97.7073 130.806 104.168 136.77 107.149 142.11 109.599 146.251 111.418 149.125 112.135 151.24 112.063 153.533 16 31 94.7523 101.823 94.7065 101.953 95.2202 104.843 95.5394 111.139 94.3681 115.142 94.2933 119.164 93.9079 122.897 93.9836 126.435 95.6531 129.713 97.9665 134.655 103.339 140.651 106.264 146.063 108.614 150.247 110.498 153.097 111.251 155.148 111.327 157.477 16 31 94.7523 137.179 94.672 139.093 95.803 141.899 97.5571 146.457 97.1562 150.987 98.115 156.314 98.1706 161.775 80.3617 165.803 64.4489 165.226 63.0973 171.144 62.685 177.988 61.5553 185.882 61.2929 193.32 61.3006 200.664 62.114 207.74 63.085 214.192 16 31 94.7523 144.25 94.7944 145.964 95.0076 148.166 95.3266 152.49 94.9951 157.406 94.4977 162.601 95.2429 167.659 80.1203 172.358 64.8995 172.211 61.696 176.818 60.8058 183.403 59.3128 191.075 58.6211 198.728 58.5046 206.148 59.0556 213.326 59.9179 220.402 16 31 94.7523 151.321 94.7135 152.245 94.5522 153.334 94.1239 156.242 93.7702 160.906 93.5197 166.003 93.9873 170.959 80.466 176.507 67.723 177.784 61.7602 178.829 60.6679 185.23 59.1607 192.919 58.3787 200.645 58.2647 208.154 58.8082 215.111 59.7066 222.337 16 31 94.7523 172.534 94.7771 172.136 94.7315 172.245 94.7052 172.313 94.7313 172.143 94.7475 171.612 94.7862 173.516 83.2637 181.324 71.245 182.157 62.0363 179.106 60.679 185.124 59.1717 192.813 58.3897 200.54 58.2758 208.048 58.8192 215.005 59.7176 222.231 16 31 94.7523 179.605 94.7639 179.491 94.7147 179.48 94.6827 179.482 94.6515 179.483 94.7006 179.386 94.4499 180.267 85.113 189.422 77.3315 192.238 65.9828 190.62 60.4617 187.128 58.942 194.783 57.85 202.372 57.7403 210.014 58.27 215.586 59.0986 223.547 16 31 101.823 87.6812 101.819 93.4223 102.817 100.303 111.098 107.05 107.993 111.146 105.506 115.399 105.667 118.916 105.868 122.906 107.585 126.929 108.786 132.37 108.376 137.524 110.472 142.701 112.683 146.845 114.604 149.838 115.385 151.983 115.126 154.213 16 31 101.823 94.7523 101.978 96.506 103.352 103.183 111.136 109.908 108.214 113.819 104.947 118.248 104.996 121.765 105.105 125.671 106.634 129.544 107.817 135.008 106.659 140.113 109.287 145.452 111.497 149.59 113.436 152.433 114.274 154.496 114.22 156.73 16 31 101.823 108.894 101.006 112.143 101.274 113.058 101.953 114.119 103.567 115.41 100.525 119.455 100.415 123.016 100.459 126.694 100.713 130.247 102.058 135.229 102.923 140.533 105.847 145.945 108.197 150.13 110.081 152.979 110.834 155.03 110.946 157.383 16 31 101.823 144.25 101.106 145.831 100.127 147.26 99.3583 149.544 97.56 153.635 97.8156 158.815 98.1258 164.192 80.3169 168.513 64.0258 167.514 62.9735 173.606 62.2793 180.316 60.8069 188.077 60.2455 195.516 60.1209 202.852 60.8821 209.921 61.7847 216.373 16 31 101.823 158.392 101.816 158.421 101.76 158.596 101.48 159.504 100.69 161.814 100.799 166.472 97.0885 169.979 84.6651 176.269 70.6911 178.247 62.5098 176.727 61.6195 183.312 60.0985 191.006 59.4068 198.658 59.2799 206.01 59.852 213.053 60.7142 220.128 16 31 101.823 165.463 101.797 165.592 101.704 165.566 101.725 165.569 101.86 165.612 101.694 166.725 96.8889 170.439 85.4928 178.174 71.6138 180.053 62.1786 177.002 61.2884 183.587 59.7954 191.259 59.1037 198.912 58.9791 206.363 59.5512 213.405 60.4135 220.481 16 31 108.894 80.6102 105.599 87.1057 103.588 94.6621 111.826 100.967 109.291 104.594 107.398 108.413 107.842 111.895 108.368 115.762 109.527 120.237 110.329 125.204 110.129 130.289 111.158 135.169 112.936 139.477 114.464 142.698 114.959 144.828 114.619 146.874 16 31 108.894 87.6812 107.583 93.3086 106.639 100.496 112.219 107.387 109.115 111.483 106.284 115.755 106.445 119.272 106.646 123.263 108.363 127.286 109.564 132.677 109.154 137.83 111.251 143.008 113.203 147.162 115.125 150.156 115.905 152.301 115.676 154.53 16 31 108.894 144.25 108.093 145.874 107.216 147.581 106.753 150.202 105.646 153.939 104.596 159.179 97.5082 163.541 79.6993 167.862 63.4082 166.619 62.3558 172.711 61.9435 179.527 60.5473 187.371 60.2285 194.872 60.1039 202.209 60.9126 209.346 61.8152 215.798 16 31 108.894 151.321 108.036 153.084 107.085 154.596 106.047 157.032 105.091 160.523 104.525 165.65 97.5083 169.533 85.0849 175.823 70.951 177.801 62.7696 176.281 61.8794 182.866 60.3695 190.575 59.5875 198.076 59.4606 205.428 60.1051 212.503 60.9925 219.27 16 31 108.894 158.392 108.792 158.611 108.422 159.316 108.019 160.746 108.34 163.204 105.962 168.176 98.9831 172.574 88.8556 183.239 74.7281 185.89 62.4715 180.901 60.8199 186.152 59.3002 193.808 58.3569 201.46 58.2429 208.969 58.7748 215.427 59.6732 222.653 16 31 108.894 165.463 108.888 165.524 108.802 165.528 108.839 165.538 109.35 165.717 106.822 169.116 99.8324 174.389 90.0953 186.263 78.0162 192.764 66.6675 191.146 61.1465 187.654 59.6061 195.243 58.5142 202.832 58.4044 210.474 58.9342 216.046 60.0403 223.63 16 31 115.966 80.6102 115.523 87.0973 114.824 94.9858 121.085 102.575 119.809 106.186 117.247 109.842 119.747 113.393 121.829 117.29 122.506 122.126 122.434 127.114 124.705 132.437 124.577 136.772 125.325 140.379 125.701 143.387 125.924 145.731 126.303 147.708 16 31 115.966 87.6812 114.861 94.026 114.186 101.551 114.826 109.354 111.722 113.266 107.651 117.734 107.815 121.237 107.834 125.194 108.988 129.184 110.107 134.664 109.602 140.009 111.094 145.111 112.753 149.247 114.674 152.116 115.512 154.187 115.316 156.413 16 31 115.966 101.823 114.361 107.706 113.158 114.898 111.431 123.727 109.007 134.186 107.084 144.838 105.657 149.607 105.565 149.683 105.437 149.685 105.437 149.708 105.437 149.769 105.645 150.187 107.673 154.059 109.254 156.804 109.809 158.793 109.966 161.142 16 31 115.966 108.894 115.022 113.142 113.967 119.275 112.137 127.633 109.512 137.503 108.015 146.28 106.262 151.049 106.175 151.142 105.862 151.122 105.861 151.158 105.861 151.2 105.952 151.373 107.494 154.476 109.075 157.222 109.631 159.211 109.787 161.456 16 31 115.966 137.179 119.281 142.539 119.067 146.738 118.732 149.613 128.486 159.773 125.206 166.451 119.99 170.154 110.41 180.233 98.9557 184.014 89.837 202.564 83.6449 210.414 81.0131 213.479 77.7946 220.249 77.1238 225.491 79.8719 227.167 83.8987 228.708 16 31 115.966 144.25 115.157 145.382 115.02 147.239 114.704 149.923 124.197 159.796 119.065 167.865 112.258 180.025 103.405 193.662 93.9604 203.504 85.9804 205.862 81.6736 209.214 78.9165 212.905 75.9782 219.112 75.4115 224.354 80.8223 223.401 84.021 225.52 16 31 123.037 80.6102 122.874 87.205 123.083 95.3573 130.143 102.821 129.758 106.563 129.205 110.13 129.023 113.885 129.569 117.992 130.471 122.964 131.525 127.948 133.092 133.289 133.086 137.819 132.112 141.32 131.653 144.156 131.891 146.525 132.311 148.512 16 31 123.037 87.6812 122.564 94.2426 125.172 102.465 128.959 108.568 128.552 112.334 128.105 115.92 127.8 119.603 128.207 123.728 128.962 128.628 129.985 133.633 131.304 138.891 131.32 143.252 130.345 146.791 129.974 149.633 130.195 151.941 130.527 153.905 16 31 123.037 94.7523 121.344 99.3417 120.764 106.265 119.277 112.943 116.316 115.697 113.352 118.526 113.692 121.45 114.477 125.068 114.179 129.862 115.278 135.021 114.11 140.712 114.495 145.851 115.408 150.005 116.265 152.984 117.061 155.064 116.833 157.254 16 31 123.037 101.823 121.19 106.706 119.722 113.54 118.033 122.213 116.262 131.977 114.657 137.152 114.72 137.566 114.706 137.733 114.611 137.793 115.058 139.319 114.693 140.734 115.05 145.725 115.963 149.88 116.821 152.859 117.011 154.964 116.782 157.154 16 31 123.037 130.108 120.549 134.838 118.383 141.336 117.946 144.274 127.399 153.551 124.483 157.87 119.279 161.756 110.347 170.123 102.451 172.705 101.579 182.538 95.788 191.113 90.64 196.248 91.7993 198.431 91.7979 200.039 91.7989 200.072 91.7986 200.084 16 31 123.037 137.179 122.8 141.709 120.561 146.284 120.205 149.16 129.698 159.038 127.866 162.978 122.542 166.697 113.26 176.649 102.257 180.405 95.5459 194.53 88.2254 204.652 85.3457 209.049 81.1231 218.065 80.5666 223.37 82.8673 226.699 84.979 230.076 16 31 123.037 144.25 122.155 146.913 121.258 150.726 120.546 153.725 131.123 163.329 129.046 168.617 124.242 171.685 114.745 181.707 103.409 185.09 94.2802 201.882 89.2359 207.814 86.4586 211.789 82.8011 219.683 82.7936 223.936 84.1167 227.722 85.811 231.292 16 31 130.108 87.6812 129.756 94.2784 133.4 102.422 132.769 106.333 132.432 110.087 131.807 113.532 131.516 117.212 131.89 121.296 132.701 126.273 133.839 131.443 134.83 136.716 134.739 141.105 133.772 144.455 133.355 147.202 133.622 149.541 134.028 151.528 16 31 130.108 94.7523 127.658 98.8191 129.593 105.25 129.058 109.391 128.651 113.156 128.204 116.687 127.869 120.333 128.276 124.458 129.029 129.342 130.051 134.347 131.277 139.523 131.257 143.821 130.27 147.374 129.899 150.216 130.121 152.524 130.446 154.494 16 31 130.108 123.037 127.381 130.133 124.644 139.086 120.957 149.204 130.53 158.433 128.699 162.373 123.585 165.942 114.473 175.794 104.567 179.55 101.553 191.225 92.3912 202.652 89.8842 206.716 90.5724 210.919 87.9006 218.134 89.5143 222.103 91.6731 225.888 16 31 130.108 130.108 127.285 136.471 124.355 143.477 122.237 147.421 131.666 156.413 130.049 160.1 125.014 163.639 118.111 172.757 117.088 177.048 116.53 181.631 112.934 189.639 109.569 197.334 107.401 201.469 108.673 205.975 114.169 208.062 117.401 209.881 16 31 130.108 137.179 127.269 142.266 122.733 146.703 122.025 149.457 131.598 158.686 129.991 162.356 124.927 165.773 117.432 175.362 111.023 180.05 110.946 186.249 103.471 195.632 92.1488 204.912 94.0573 208.162 93.1289 214.262 94.7085 218.269 98.0547 220.709 16 31 137.179 87.6812 136.051 92.4747 136.426 97.3776 135.416 100.97 134.879 104.602 134.084 107.921 133.622 111.468 134.104 115.447 135.083 120.484 136.191 125.683 137.401 131.213 137.385 135.811 136.614 139.085 136.185 141.503 136.465 143.82 136.936 145.934 16 31 137.179 94.7523 132.064 95.5239 134.274 102.136 133.473 105.903 133.136 109.657 132.511 113.102 132.19 116.718 132.564 120.802 133.429 125.839 134.568 131.009 135.559 136.282 135.468 140.775 134.532 143.97 134.116 146.717 134.392 149.057 134.833 151.133 16 31 137.179 123.037 134.477 130.358 131.502 139.692 127.228 151.631 135.766 160.843 134.338 164.427 130.153 167.389 129.323 172.587 128.471 177.512 128.74 182.879 134.337 189.738 136.097 193.965 136.694 197.61 138.196 201.095 139.322 203.197 139.486 205.499 16 31 137.179 130.108 134.264 136.579 130.725 143.252 126.013 154.278 135.813 163.408 134.414 167.092 130.178 169.43 127.672 174.508 126.763 179.418 126.592 184.684 130.893 191.417 131.591 195.472 132.429 199.18 135.487 202.609 136.82 204.637 136.716 206.846 16 31 137.179 137.179 134.175 142.64 130.479 147.646 126.702 153.363 135.66 162.766 134.233 166.38 129.996 168.718 128.571 173.915 127.697 178.863 127.323 184.199 132.542 191.035 133.487 195.125 134.363 198.749 136.814 202.235 137.851 204.336 137.964 206.551 16 31 137.179 144.25 134.977 148.307 131.478 151.087 127.944 156.826 137.027 165.39 135.729 169.003 132.474 171.146 131.26 175.807 130.779 180.797 131.128 185.996 135.397 191.784 137.194 195.952 138.018 199.35 139.422 202.839 140.084 205 139.932 207.281 16 31 144.25 108.894 143.203 114.5 142.489 120.137 141.874 126.314 141.465 132.853 141.004 139.303 141.015 145.866 141.343 152.48 141.994 159.073 142.057 167.28 140.138 177.094 137.315 183.423 136.047 191.78 137.536 195.057 139.126 197.01 139.849 199.386 16 31 144.25 123.037 141.146 130.524 137.904 140.203 139.03 151.463 138.751 156.749 137.618 160.565 135.246 164.087 135.554 169.229 135.958 174.407 137.019 179.779 139.901 185.187 142.106 189.926 142.025 197.371 143.516 200.814 144.636 203.362 145.618 206.565 16 31 144.25 130.108 141.196 136.577 138.277 141.589 139.403 151.602 138.763 156.649 137.63 160.466 135.355 164.063 135.663 169.205 136.067 174.383 137.128 179.755 140.01 185.163 142.215 189.902 142.4 197.347 143.926 200.798 145.045 203.447 146.087 206.692 16 31 144.25 137.179 141.887 142.355 139.414 147.3 138.759 152.819 138.293 158.074 137.084 161.811 134.711 165.334 134.88 170.476 134.91 175.585 135.954 180.979 139.677 186.752 142.014 191.435 142.407 198.208 143.932 201.659 144.752 204.363 145.724 207.486 16 31 151.321 108.894 150.068 113.288 149.381 118.289 148.909 124.366 148.58 130.776 148.223 136.96 148.247 143.488 148.66 150.086 149.525 156.612 150.087 164.492 149.755 173.84 148.678 180.359 146.983 189.401 148.74 192.654 150.086 195.072 151.022 198.365 16 31 151.321 115.966 150.044 120.528 149.45 125.953 148.967 132.206 148.397 138.665 147.915 145.129 147.95 151.579 148.297 158.271 148.952 164.889 149.289 172.862 147.923 182.632 148.106 188.547 149.372 196.337 150.886 199.72 151.702 202.568 152.829 205.778 16 31 151.321 123.037 149.566 127.708 149.275 129.545 149.072 131.629 148.503 138.088 148.021 144.552 148.046 151.158 148.393 157.851 149.048 164.469 149.385 172.489 148.019 182.259 148.202 188.174 149.468 195.964 150.982 199.347 151.798 202.195 152.925 205.405 16 31 151.321 130.108 149.658 135.29 148.516 139.143 148.091 140.754 147.971 141.348 147.354 145.365 147.34 151.953 147.639 158.669 148.259 165.287 148.596 173.26 147.202 182.961 146.977 188.876 149.286 196.187 150.8 199.569 151.617 202.417 152.743 205.627 16 31 179.605 137.179 177.344 144.569 177.354 151.436 177.298 158.171 177.758 163.993 181.971 169.428 197.73 178.841 195.314 183.232 194.197 186.794 186.655 187.45 183.355 188.538 185.09 192.667 186.321 195.701 186.726 197.121 186.208 198.091 186.013 199.416 16 31 193.747 151.321 193.609 158.043 201.028 159.692 215.769 162.107 216.899 175.167 207.87 187.54 197.304 192.702 190.188 195.882 188.158 199.647 189.705 204.547 198.316 210.416 201.574 212.987 204.834 214.843 207.063 216.717 209.364 218.614 210.811 220.098 16 31 193.747 165.463 194.078 165.688 197.187 167.296 209.478 168.983 213.968 178.948 189.612 189.232 183.804 192.791 173.586 193.916 175.525 199.069 171.429 202.05 170.722 205.793 171.185 209.771 171.375 212.301 171.54 214.462 171.485 216.491 171.726 218.064 16 31 200.818 137.179 212.348 131.967 215.374 142.757 217.558 155.005 216.965 168.908 223.1 181.375 225.089 193.297 222.82 202.431 220.495 207.071 220.824 208.06 221.546 207.368 221.316 206.387 221.224 206.066 220.798 206.684 221.087 207.284 220.975 207.43 16 31 200.818 144.25 209.623 142.593 212.389 150.816 208.053 165.234 214.944 175.093 201.874 187.029 188.866 190.435 179.771 192.537 180.367 197.316 180.463 201.996 182.84 206.335 183.432 209.552 185.024 211.842 185.234 213.623 185.279 215.257 185.198 216.782 16 31 200.818 151.321 203.589 152.605 206.411 158.526 208.604 167.854 213.094 177.819 188.738 188.387 183.469 191.961 173.252 193.086 175.19 198.239 170.866 201.146 170.665 205.114 170.964 208.977 171.153 211.507 171.319 213.668 171.124 215.697 171.365 217.27 16 31 207.889 130.108 209.203 139.557 206.853 154.209 206.485 165.64 218.15 173.82 213.912 187.44 216.587 200.563 216.686 211.866 215.509 219.187 216.309 221.916 215.967 220.375 216.441 219.367 217.109 219.24 216.074 219.257 216.799 220.018 216.346 220.586 16 31 207.889 137.179 212.251 141.621 210.348 155.184 208.943 167.206 210.518 178.507 186.86 188.639 182.474 192.231 172.478 193.356 174.997 198.636 170.673 201.543 169.965 205.286 170.485 209.281 170.675 211.81 170.841 213.972 170.646 216 170.886 217.573 16 31 207.889 151.321 207.965 151.486 209.996 155.219 208.591 166.709 215.261 176.568 194.825 187.802 181.063 191.047 174.698 193.017 176.579 198.17 172.255 201.078 171.742 204.898 172.04 208.761 172.21 211.265 172.307 213.416 171.874 215.339 172.168 216.937 16 31 214.96 115.966 220.011 122.769 223.045 135.643 222.546 152.745 220.229 169.078 218.428 183.88 223.893 198.036 221.869 207.581 220.004 210.492 219.98 210.642 220.306 210.496 220.202 210.178 220.358 209.996 220.244 210.276 220.521 210.581 220.539 210.667 16 31 214.96 137.179 216.82 139.884 217.85 150.922 216.859 165.11 218.306 177.928 205.514 190.066 190.997 193.41 181.665 195.574 181.922 200.345 183.027 205.316 186.53 209.979 188.171 213.182 189.603 215.05 190.088 216.773 190.524 218.358 190.468 219.834 16 31 214.96 144.25 214.971 144.479 216.606 152.363 215.016 166.62 216.463 179.209 196.572 190.443 182.704 193.522 172.9 194.854 174.509 199.761 170.414 202.742 169.707 206.485 170.361 210.509 170.548 213.028 170.721 215.189 170.666 217.218 170.794 218.747 16 31 24 104 22.8554 107.399 22.1762 110.537 20.9858 114.933 19.7402 119.277 18.9441 123.64 18.6774 128.381 18.7391 133.547 19.256 139.17 20.4746 144.743 22.6467 150.642 24.0687 157.766 25.1345 165.466 26.3592 172.785 27.064 180.261 27.191 187.498 16 31 24 114 22.5436 118.062 21.3394 121.949 19.9814 126.427 18.6977 130.487 17.6554 134.984 17.114 139.823 17.0769 145.038 17.5446 150.485 18.8076 156.313 20.7313 162.244 22.1804 168.657 23.6042 175.736 24.5875 183.665 24.7598 191.628 24.7787 199.218 16 31 24 124 22.3047 127.456 20.838 131.251 19.1994 135.397 17.9084 139.761 16.6237 144.106 15.983 148.485 15.9894 153.993 16.3686 158.768 17.3468 164.461 19.3449 170.62 20.7873 176.616 22.1743 183.412 22.8959 190.472 23.024 198.248 23.0121 206.497 16 31 24 134 22.2382 137.12 20.3758 139.89 18.2287 142.208 16.8665 146.2 15.0789 148.066 14.3194 152.003 13.8995 155.9 13.9229 159.297 14.4983 163.455 16.2165 166.773 17.577 171.817 18.9067 178.224 19.7167 183.397 19.9939 189.473 20.2445 194.409 16 31 24 144 24.2354 143.81 22.8278 145.537 20.8335 147.184 19.526 151.837 18.3594 156.359 17.6259 160.443 17.1021 165.207 17.5183 170.421 18.1857 175.803 19.8688 181.648 21.0674 187.854 21.607 193.821 22.1086 200.486 21.9797 207.786 21.9224 216.59 16 31 24 154 24.0741 153.811 24.0681 153.976 23.9198 154.327 23.7141 158.279 22.9255 162.944 22.5496 167.181 22.6086 172.038 22.8302 177.073 23.2186 181.448 25.069 187.493 26.0226 194.084 26.6828 200.17 26.9421 206.846 26.7707 214.386 26.7134 223.221 16 31 24 164 23.7324 164.091 23.8312 164.168 23.7743 164.295 23.9365 164.144 23.8173 164.359 23.5508 168.417 23.4702 173.869 23.4001 178.387 24.0897 183.698 25.2792 189.235 25.5832 195.131 25.865 200.348 26.1244 207.024 25.777 213.878 25.7197 222.713 16 31 34 94 32.6481 98.2413 31.0756 102.558 30.3185 106.818 29.8594 111.056 29.1205 115.439 28.8907 120.36 29.1863 125.542 29.6876 131.499 30.4909 137.701 32.3562 144.135 34.5501 150.881 36.1982 158.343 36.4813 166.215 36.4732 174.435 36.2098 181.868 16 31 34 104 32.5261 108.306 31.5956 112.075 30.3566 116.584 29.0424 121.023 28.2247 125.616 27.8024 130.71 27.7438 136.115 28.2889 142.052 29.4884 147.917 31.4077 154.27 33.0124 161.304 34.2999 168.732 35.3137 176.177 35.4844 184.28 35.5036 191.806 16 31 34 114 32.5518 118.232 31.4516 122.438 30.1259 126.959 28.869 131.474 27.9396 136.13 27.5434 141.273 27.4813 146.67 28.0237 152.448 29.3254 158.473 31.2059 164.808 32.8134 171.646 34.2066 178.849 35.1871 186.415 35.3465 194.382 35.3056 202.359 16 31 34 124 32.4674 128.041 31.3169 132.436 29.8714 136.767 28.6815 141.382 27.7232 146.167 27.1628 150.895 26.9682 156.314 27.5498 161.878 28.7081 168.05 30.6556 174.454 32.4068 181.182 33.6535 188.107 34.4502 195.344 34.603 203.284 34.6345 212.481 16 31 34 134 32.2507 137.868 31.2252 142.442 29.7995 146.609 28.4367 151.198 27.4615 156.057 26.8715 160.762 26.6159 166.134 27.1529 171.698 28.3019 178.081 30.3663 184.494 31.992 191.416 33.3622 198.509 34.1769 205.649 34.3337 213.389 34.1353 222.474 16 31 34 144 32.9918 146.046 31.7652 149.71 30.4153 153.709 29.7513 158.379 28.7956 163.071 28.2594 167.589 27.8659 173.083 28.0249 178.363 29.0992 184.422 30.9798 190.787 31.8123 197.618 32.8864 204.297 33.0294 210.962 32.6389 218.52 32.3206 227.528 16 31 34 154 33.9766 154.104 33.9754 154.142 33.3758 156.031 32.9009 160.505 31.9452 165.099 31.2441 169.259 30.7373 174.671 31.0399 180.174 32.1834 186.335 33.1493 192.625 33.8022 199.582 33.6388 205.526 32.8333 211.778 31.5465 218.623 31.2283 227.631 16 31 34 164 33.9173 163.949 33.9639 163.905 33.8833 164.05 33.9531 163.93 33.4056 166.159 32.6331 170.319 32.1263 175.731 32.3505 180.729 33.4939 186.89 34.4028 193.284 34.6038 199.588 34.4404 205.533 33.6349 211.785 32.3481 219.012 31.7945 227.931 16 31 44 94 42.568 98.1232 41.2663 102.338 39.9573 106.847 38.9712 111.318 38.0348 115.976 37.6311 121.002 37.8176 126.355 38.5113 132.41 39.3872 138.688 41.3012 144.918 43.6269 151.494 45.6061 158.624 46.6874 166.184 47.1289 174.095 47.758 181.632 16 31 44 104 42.6276 108.246 41.3132 112.474 40.0169 116.999 38.8797 121.405 38.0352 126.059 37.6049 131.136 37.6458 136.557 38.2163 142.521 39.0791 148.803 40.8928 155.05 42.9567 161.615 44.4903 168.826 45.3031 176.445 45.7192 184.449 45.601 192.127 16 31 44 124 42.6081 128.039 41.2797 132.4 40.0224 136.774 38.7445 141.257 37.7901 145.932 37.4368 150.712 37.4239 155.906 38.0782 161.6 39.0638 167.826 41.0475 174.466 42.4683 181.608 43.8412 188.62 44.9843 196.149 45.4208 204.115 45.4083 212.797 16 31 44 134 42.8027 137.888 41.281 142.491 39.9487 146.682 38.6561 151.217 37.4678 155.913 37.1164 160.603 36.9127 165.94 37.5603 171.623 38.5326 178.026 40.6896 184.579 42.4224 191.604 43.6907 198.537 44.9499 205.808 45.0797 213.782 44.4833 222.736 16 31 44 144 43.8005 146.926 43.7188 150.685 42.3406 154.416 40.9587 159.148 40.1655 163.187 39.8141 167.621 39.7593 172.708 41.1744 178.238 42.643 184.146 44.5314 190.43 45.4942 197.405 44.0206 204.387 43.8472 211.492 42.2164 219.404 41.4187 228.432 16 31 44 154 43.9091 154.189 43.8876 154.266 42.8389 155.718 41.5709 160.193 41.001 163.907 40.8493 168.132 40.7945 173.219 42.4445 178.252 43.9132 184.16 45.9958 190.236 47.0653 197.149 45.5918 204.238 45.4183 211.518 43.6918 219.574 43.7199 228.562 16 31 44 164 43.9824 164.021 43.9872 164.06 43.9093 163.989 43.7884 164.195 43.6729 165.089 43.9624 167.896 44.6697 172.417 47.0014 177.569 49.0443 182.956 51.0856 189.009 50.286 195.98 48.8629 203.535 48.7823 210.953 48.0813 219.008 52.3612 227.524 16 31 54 94 52.8904 98.2166 51.7323 102.533 50.4947 107.022 49.9022 111.606 49.2959 116.28 48.9527 121.469 48.2811 126.732 48.4949 132.582 49.7103 139.004 51.8099 145.591 53.6178 151.864 54.7895 158.528 54.8998 165.695 53.7843 173.319 52.6297 180.58 16 31 54 104 52.7219 108.258 51.5129 112.599 50.2849 117.13 49.3136 121.71 48.3685 126.325 47.9391 131.375 47.8363 136.651 48.3813 142.429 49.4318 148.747 51.3355 155.314 53.352 161.786 54.4674 168.864 55.4613 176.042 55.2948 183.592 54.6814 190.791 16 31 54 114 52.7077 118.461 51.4029 123.025 50.3064 127.768 49.0425 132.665 48.0494 137.499 47.6359 143.01 47.9226 148.283 48.4756 154.023 49.5056 160.382 51.0086 167.491 51.9971 175.575 53.021 183.627 53.6411 191.994 54.2914 200.194 54.7442 208.6 16 31 54 124 52.7773 128.359 51.6015 132.817 50.3702 137.503 49.0875 142.03 47.8966 147.202 47.2686 152.262 47.0664 157.385 47.4887 163.272 48.3801 169.435 49.7952 176.358 50.6847 183.745 51.7093 191.017 52.4543 198.809 53.4062 206.794 54.3505 214.97 16 31 54 134 53.2503 137.93 52.4346 142.272 51.1458 146.65 49.706 151.312 48.3751 156.394 47.8859 161.301 47.6336 166.742 47.9617 172.516 49.2859 178.589 51.0594 185.308 51.5709 192.665 50.888 200.529 51.4322 208.174 51.8745 216.063 53.4171 224.49 16 31 54 144 53.8841 145.678 54.6166 147.976 53.916 151.493 52.6403 155.065 51.5653 159.33 51.2189 163.544 50.9665 169.095 51.5723 174.32 52.8108 180.393 54.469 187.096 53.5422 194.634 52.2453 202.598 52.3504 210.215 52.5377 218.021 55.6989 226.42 16 31 54 154 53.8919 153.95 53.8413 153.916 53.5508 155.46 53.0516 157.252 52.2434 160.743 51.897 164.957 51.6447 170.507 51.9096 175.745 53.2339 181.573 53.7243 188.122 52.7975 195.661 51.3601 203.418 51.4432 210.994 51.6306 218.8 54.7917 227.199 16 31 54 164 54.0019 164.069 53.9658 164.212 53.8285 163.809 53.8514 164.103 53.8944 164.739 54.168 167.469 53.9724 171.66 54.2373 176.399 55.1432 182.471 54.5617 189.002 53.2548 196.3 51.8174 204.13 51.9005 211.706 51.6432 219.21 57.205 227.637 16 31 64 84 63.4379 88.1492 62.2022 92.5745 61.2771 96.9385 61.2195 101.24 61.0437 106.024 61.8168 110.535 62.8392 113.438 64.6864 115.927 67.2297 118.114 69.2561 122.21 69.6407 126.087 70.2752 128.197 70.4386 128.388 70.4173 128.379 70.4163 128.411 16 31 64 94 63.2853 98.1654 62.0477 102.666 61.1303 107.179 61.0159 111.612 60.7238 116.238 61.3214 121.186 61.5983 125.237 63.5921 128.446 67.8104 130.98 70.5445 136.06 73.9919 140.192 77.2536 144.309 79.7894 147.84 81.5342 150.649 82.1518 153.196 16 31 64 104 62.86 108.258 61.656 112.803 60.5391 117.423 59.5928 122.175 58.782 126.931 58.551 132.09 57.9621 137.258 58.4178 142.965 59.4886 149.334 61.3448 155.891 63.173 162.39 64.1462 169.66 64.855 176.689 64.7986 184.067 64.2278 191.046 16 31 64 114 62.413 118.707 60.8012 123.768 59.0003 128.893 57.4031 134.362 55.7295 139.634 54.5339 145.086 53.9889 150.259 54.0929 155.823 54.6356 162.03 55.8127 169.008 56.749 177.063 57.6603 185.188 58.3433 193.514 59.2872 201.738 60.3372 209.913 16 31 64 124 62.4221 128.453 60.9041 133.053 59.3968 137.665 57.8931 142.233 56.3277 147.218 55.3404 152.084 54.6793 157.136 54.5938 162.65 54.9845 168.667 56.0876 175.495 56.7194 183.047 57.4314 190.695 58.108 198.417 59.206 206.226 60.4964 213.79 16 31 64 134 63.8154 137.681 62.8602 141.412 61.8539 144.843 60.3881 148.848 58.8699 153.24 58.036 157.628 57.429 162.735 57.2977 168.251 57.5638 174.507 57.6291 181.309 56.5217 188.872 56.4126 196.667 56.7888 204.221 57.7823 211.771 59.0124 219.843 16 31 64 144 63.7026 145.84 63.1639 148.761 62.3828 152.16 60.9105 155.879 59.8515 160.214 59.3847 164.393 58.5011 169.248 58.2061 174.587 58.3461 180.832 57.3953 187.535 55.9055 195.115 54.4824 202.988 54.5778 210.605 55.0201 217.776 56.6853 226.401 16 31 64 154 63.9603 153.925 63.8997 153.838 63.6172 155.138 62.2993 157.965 61.362 162.139 60.9296 166.175 59.9737 170.73 59.0162 175.9 57.341 181.58 56.0068 188.037 54.6424 195.609 53.205 203.44 53.2881 211.024 53.6574 217.648 55.8524 226.121 16 31 64 164 63.999 164.083 63.9184 164.145 63.9535 164.136 63.8153 164.099 63.6686 164.841 63.2743 168.468 61.8869 172.567 60.0167 176.747 58.3414 182.427 57.0073 188.884 55.5174 196.465 54.0943 204.338 54.1774 211.922 54.5467 218.546 56.7417 227.019 16 31 74 84 73.4702 88.2757 72.7496 93.1967 72.1037 97.5803 71.2849 101.341 70.2696 104.954 69.8472 108.159 70.149 110.695 71.0493 112.953 71.8987 115.016 72.7919 119.069 73.0295 122.175 73.0755 122.977 73.0818 123.138 73.077 123.074 73.0811 123.226 16 31 74 94 73.5809 98.3469 73.5367 103.563 73.547 108.218 72.0679 112.233 69.9734 116.089 69.7494 119.617 72.746 122.473 75.4994 125.002 79.6382 127.443 84.2251 130.695 85.8325 135.346 86.1601 139.992 86.6605 144.374 86.7269 147.922 87.0103 150.452 16 31 74 124 72.4498 127.789 70.8067 131.775 68.984 135.283 67.4377 139.377 65.6257 144.031 64.1981 148.782 63.2967 154.106 63.0111 159.866 62.5545 166.063 63.1006 172.983 63.6139 181.03 63.8116 188.942 64.4864 196.868 65.4317 204.502 66.3905 211.963 16 31 74 134 73.637 137.715 72.8013 141.666 71.8103 145.42 70.3251 149.32 68.392 153.617 66.7822 157.674 65.0069 162.241 62.0604 167.02 60.4909 172.944 60.2553 179.746 59.148 187.379 58.4565 195.065 58.6667 202.58 59.7421 210.003 61.0482 217.593 16 31 74 144 74.1069 147.599 73.7251 151.264 73.3024 155.138 72.8689 159.033 72.1536 163.169 71.4172 166.953 67.6873 170.977 60.8847 174.366 59.5642 180.523 58.23 186.98 56.8118 194.552 55.5149 202.516 55.5979 210.115 56.2109 216.739 57.4425 225.137 16 31 74 154 73.9166 154.218 73.9427 154.871 73.6702 158.328 73.3578 161.824 72.9447 165.799 72.5045 169.434 70.2207 173.482 63.3641 176.219 59.5823 180.764 58.2482 187.222 56.7583 194.802 55.4614 202.766 55.5444 210.364 56.1574 216.989 57.389 225.387 16 31 74 164 73.9985 164.035 74.0384 164.064 73.9699 164.431 73.7444 164.516 73.4553 167.166 73.1172 170.61 70.6763 174.873 63.8197 177.609 59.5805 181.145 57.8708 187.319 56.3809 194.899 55.084 202.863 55.1671 210.462 55.78 217.086 57.4453 225.712 16 31 74 224 74.1452 223.76 74.09 223.872 74.0564 223.824 74.0413 223.782 74.0586 223.778 73.9904 223.829 74.0012 223.669 66.1585 230.516 53.4783 243.038 40.0693 254.178 32.8936 255.544 31.4858 251.687 35.2714 244.209 45.4305 232.189 57.5988 227.002 16 31 84 84 83.3425 88.946 82.8321 95.1167 82.9065 100.417 82.7699 104.16 82.4871 107.585 82.3416 111.03 82.1855 113.934 82.2975 116.642 82.3793 119.047 82.6963 122.886 82.7818 128.166 82.6812 132.858 82.9457 137.617 82.9058 141.355 82.9539 143.877 16 31 84 94 83.6885 98.6689 83.8889 104.149 84.7928 109.435 84.6166 113.449 84.6148 117.25 84.4016 120.966 83.5623 124.102 83.9337 126.902 82.9426 129.426 87.2104 132.677 89.9783 137.085 91.2058 141.557 92.216 145.526 92.2824 148.856 92.1936 151.163 16 31 84 134 85.0075 137.306 84.7187 140.905 83.5915 145.162 83.2848 149.697 83.0061 155.1 82.5666 161.068 75.3556 164.46 64.2571 165.291 62.1416 171.027 61.5126 177.739 60.4231 185.606 60.1539 193.292 60.434 200.766 61.5217 208.188 62.836 215.527 16 31 84 144 83.9792 146.344 83.7781 150.054 83.3069 154.705 83.0748 159.373 82.9612 164.697 82.8664 170.259 76.2862 174.666 63.3613 175.411 59.5796 179.957 58.2454 186.414 56.8272 193.986 55.5303 201.95 55.6133 209.548 56.2263 216.172 57.4579 224.571 16 31 84 154 84.0399 154.798 83.9636 155.427 83.6408 158.59 83.4021 163.027 83.3899 167.398 83.3266 172.827 74.9945 177.233 67.5145 179.967 59.1016 179.119 57.7675 185.576 56.3493 193.148 55.0523 201.112 55.1354 208.71 55.8134 215.787 57.045 224.185 16 31 84 164 84.1441 164.245 84.152 163.837 83.9925 164.71 83.9016 164.53 83.8894 168.9 83.8261 174.329 76.2588 178.72 67.2594 181.454 58.662 179.95 57.7113 186.653 56.2931 194.225 54.9961 202.189 55.0915 209.805 55.7045 216.429 56.9361 224.828 16 31 84 174 84.0562 173.906 84.025 173.637 84.0062 174.196 83.9607 173.726 83.9046 173.345 83.9544 175.041 77.6451 178.579 66.8921 181.155 59.6629 181.387 57.9532 187.561 56.4634 195.141 55.0403 203.014 55.1233 210.595 55.7363 217.219 57.4015 225.845 16 31 84 204 84.0192 204.03 83.9899 204.14 84.0801 203.8 84.1561 203.736 84.1238 203.729 84.1573 203.486 82.0107 210.39 69.1472 225.311 54.3641 239.954 41.742 250.675 35.3951 251.12 32.788 248.526 34.54 245.261 44.3066 233.936 56.5142 227.911 16 31 94 84 93.6949 89.6348 93.56 96.4345 93.3802 102.865 92.8805 107.08 92.4404 110.835 92.2821 114.524 92.0341 118.176 92.3839 121.26 92.0709 124.903 93.4404 127.745 95.8992 131.532 98.4147 135.704 100.132 139.1 101.09 141.971 100.61 144.006 16 31 94 114 93.5009 115.318 93.0683 117.323 92.456 119.348 91.5296 122.77 91.2088 126.438 91.5788 129.984 92.0138 132.326 92.9142 134.95 92.9764 138.097 96.7452 142.283 99.9963 146.866 102.631 151.132 104.058 154.525 104.714 156.681 104.61 159.032 16 31 94 134 93.9316 135.793 95.5971 138.709 96.7709 142.834 96.4924 147.68 98.0072 153.556 96.0317 158.919 79.3471 161.797 66.6816 162.662 64.5847 168.215 64.315 175.017 63.5109 183.051 63.2417 190.813 63.8403 198.632 64.7948 206 66.1009 213.309 16 31 94 144 93.9352 145.674 94.1788 148.16 94.5895 152.66 94.291 157.608 94.3743 162.984 94.9303 168.189 79.1095 172.872 64.0431 172.904 60.8725 178.05 59.8236 184.606 58.4051 192.187 57.3196 200.059 57.3943 207.643 58.0723 214.48 59.2987 222.837 16 31 94 154 93.9607 155.541 93.752 157.137 93.4744 159.528 93.2751 163.809 93.6022 169.059 93.6102 173.786 79.9139 179.375 69.2905 182.168 59.2539 179.701 57.9197 186.158 56.5015 193.73 55.2046 201.694 55.2877 209.293 55.9006 215.917 57.1322 224.315 16 31 94 164 94.1222 164.472 94.1135 164.147 94.0856 165.017 94.1459 165.027 94.473 169.869 94.4811 174.596 80.7847 180.185 70.1613 182.979 60.1247 180.512 58.7906 186.969 57.3724 194.541 56.0786 202.414 56.1616 210.012 56.7746 216.636 58.0062 225.035 16 31 94 174 94.0495 173.426 93.997 173.642 93.9547 173.872 93.9691 173.782 94.0387 173.269 93.9489 176.619 80.8327 183.419 73.6352 185.755 61.8776 182.171 59.6758 187.448 58.0399 195.02 56.6532 202.835 56.7362 210.433 57.3492 217.057 58.5793 225.789 16 31 104 84 104.599 89.5527 105.221 96.5849 111.982 103.483 107.632 107.595 105.411 111.674 105.703 115.208 105.9 119.146 108.362 123.243 109.489 128.464 109.387 133.644 112.143 138.498 114.787 142.657 116.47 145.766 117.09 147.917 116.808 150.074 16 31 104 104 102.633 108.548 102.215 112.724 102.141 117.905 104.885 121.654 105.193 126.351 98.8992 130.388 98.4233 133.043 98.6045 135.332 99.6515 139.424 100.513 144.446 103.487 149.425 106.076 153.545 107.614 156.672 108.302 158.709 108.338 161 16 31 104 114 102.992 117.982 102.498 123.11 101.571 128.979 100.491 135.211 99.7155 142.003 97.8844 146.595 97.2924 146.586 97.3263 147.366 97.3532 147.889 97.4257 148.463 99.3766 151.66 101.828 155.567 102.441 158.884 102.522 161.05 102.196 163.537 16 31 104 144 103.283 145.565 102.523 147.026 101.526 149.44 99.6374 153.261 99.9408 158.61 96.3819 163.716 78.8642 167.681 63.6905 167.266 61.1711 172.812 60.542 179.524 59.4347 187.157 58.7432 194.842 59.148 202.338 60.1231 209.575 61.4292 217.165 16 31 104 154 103.026 155.611 102.237 157.244 101.464 159.299 100.773 162.989 100.9 168.199 95.3512 172.434 82.5839 179.234 70.5474 181.495 60.5108 179.028 59.1766 185.486 57.6868 193.057 56.393 200.93 56.4884 208.532 57.1664 215.608 58.3928 224.081 16 31 104 164 103.923 164.054 103.8 164.135 103.759 164.429 103.97 166.213 102.207 170.527 95.0434 174.796 83.6372 182.592 71.0812 184.853 60.2954 181.749 58.5857 187.922 57.0959 195.503 55.7092 203.318 55.7923 210.899 56.4053 217.523 58.0705 226.149 16 31 104 174 103.799 173.837 103.792 173.878 103.688 173.875 103.866 173.969 103.933 173.516 96.5488 182.988 85.5681 196.647 76.1357 197.003 66.14 200.525 59.355 198.158 56.836 196.87 55.4129 204.743 55.496 212.324 56.0378 218.127 57.703 226.753 16 31 114 84 113.21 90.4769 112.068 98.4339 116.303 106.169 112.109 110.256 109.603 114.398 109.692 117.983 110.169 121.845 111.447 126.248 112.596 131.469 112.021 136.986 114.568 141.844 117.075 145.916 118.399 148.916 119.058 151.042 118.818 153.164 16 31 114 104 112.264 110.347 111.005 117.512 108.971 126.359 106.499 136.074 104.832 145.397 101.597 150.815 100.667 151.105 99.9794 151.885 99.9634 152.44 100.053 152.659 100.322 153.219 101.94 155.838 102.553 159.156 102.333 161.234 102.007 163.721 16 31 114 114 113.174 117.502 112.065 123.45 110.273 131.369 107.802 140.418 106.197 147.961 102.389 153.152 97.2462 155.539 97.0148 155.73 96.9891 156.163 97.0692 156.39 97.0836 156.552 97.6387 157.728 97.4232 161.383 96.7372 163.46 96.0196 165.757 16 31 114 134 116.533 140.036 117.639 143.349 117.443 145.723 127.031 155.69 123.975 160.479 118.344 164.28 108.271 174.249 96.8463 177.762 90.7635 190.926 84.2952 199.693 80.7253 204.809 79.2554 210.763 78.4242 216.238 79.6388 220.509 82.286 223.439 16 31 114 144 114.344 145.431 115.015 147.443 114.66 149.969 124.032 159.603 119.176 165.443 112.676 171.748 102.611 184.714 91.1448 194.87 81.736 200.631 74.7032 208.145 70.3383 211.545 68.9352 215.376 69.033 219.847 69.6031 222.82 70.1567 226.49 16 31 124 74 123.975 80.6206 124.294 88.5814 130.396 95.8074 129.909 99.4579 129.067 102.879 128.778 106.76 129.47 110.806 130.158 115.287 131.202 120.146 132.419 125.164 132.379 129.567 131.465 133.288 130.798 135.965 131.006 138.123 131.414 140.07 16 31 124 84 123.721 90.6098 125.833 99.2057 129.848 106.114 129.219 109.897 128.597 113.435 128.347 117.203 128.649 121.303 129.385 126.149 130.507 131.24 131.648 136.59 131.503 141.09 130.496 144.652 130.101 147.569 130.338 149.811 130.702 151.762 16 31 124 94 122.423 99.9546 121.663 107.812 119.561 115.282 115.322 119.79 109.175 123.926 103.311 128.021 102.835 131.39 103.719 134.834 104.798 139.957 104.161 144.92 106.836 149.998 109.404 154.101 111.189 157.126 111.937 159.101 111.983 161.268 16 31 124 104 122.192 109.748 120.328 116.759 118.45 126.339 115.55 137.534 111.942 147.458 106.744 152.969 105.813 153.887 104.212 153.897 104.201 154.268 104.293 154.338 104.365 154.49 106.22 157.612 106.877 160.351 107.225 162.377 107.353 164.63 16 31 124 124 121.636 130.183 119.032 137.643 118.115 142.524 127.338 152.851 123.88 157.368 118.34 161.116 108.377 170.962 98.2771 174.017 94.2169 185.889 87.343 195.563 84.3968 200 84.1286 204.289 82.2008 210.703 82.8627 214.974 83.5849 218.399 16 31 124 134 121.924 139.432 118.554 144.645 118.359 147.019 128.009 157.056 124.943 162.151 119.429 165.962 109.767 175.875 98.1806 179.424 91.5857 193.762 85.0043 202.401 81.6257 207.517 78.8837 214.65 78.169 220.117 80.2121 223.746 83.7577 225.89 16 31 124 144 123.483 148.507 121.873 152.711 121 155.367 131.029 165.594 129.196 172.581 124.881 176.692 115.356 187.058 104.378 190.066 94.4052 205.567 88.8287 211.322 86.5654 213.119 83.4574 219.865 84.0505 223.588 85.9858 227.19 87.8544 230.928 16 31 134 84 133.82 90.7457 138.718 99.9529 137.859 103.758 137.029 107.271 135.948 110.454 135.407 114.04 135.308 117.738 135.751 122.458 136.875 127.67 137.887 133.199 137.72 137.723 137.081 140.904 136.78 143.293 137.035 145.499 137.525 147.75 16 31 134 94 129.181 95.2536 133.36 104.074 132.8 108.346 132.155 112.119 131.451 115.595 130.969 119.206 131.126 123.24 131.861 128.116 132.971 133.24 133.835 138.493 133.567 142.919 132.601 146.308 132.239 149.195 132.464 151.43 132.832 153.344 16 31 134 114 131.534 120.982 129.181 129.952 126.474 141.001 133.617 149.731 131.905 153.469 128.076 156.402 127.627 161.428 126.875 166.301 125.653 171.614 128.878 178.531 128.98 182.715 127.788 186.957 128.782 190.654 129.75 192.668 130.201 194.694 16 31 134 124 131.34 131.243 128.514 141.128 124.409 149.941 133.839 159.693 132.519 163.384 128.128 166.308 125.347 172.479 124.36 177.304 123.219 182.242 126.819 189.265 129.423 193.233 129.751 197.232 132.35 200.973 133.964 202.987 134.842 205.042 16 31 134 134 130.993 139.655 127.328 144.864 123.284 151.436 132.869 161.473 131.218 165.703 126.903 168.428 120.355 176.85 113.432 181.681 110.449 187.535 103.398 196.878 92.0759 206.126 93.2813 209.767 91.6556 216.261 93.1303 220.48 97.2287 222.525 16 31 134 144 131.089 148.562 127.357 152.27 123.745 157.035 133.983 167.273 132.932 173.18 129.15 176.788 121.676 180.755 112.437 186.113 108.477 193.903 98.7172 205.311 96.3482 209.121 97.1009 212.733 96.9385 218 98.9699 221.745 102.514 223.918 16 31 144 124 141.098 131.38 138.468 141.365 138.302 150.71 139.064 156.696 137.954 160.501 135.157 163.57 135.463 168.672 135.673 173.758 136.568 179.127 140.511 185.529 142.051 190.629 142.071 198.314 143.562 201.688 144.585 204.208 145.554 207.317 16 31 144 134 141.18 139.429 138.819 144.347 138.163 151.497 138.926 157.483 137.739 161.198 134.812 164.197 134.789 169.331 134.615 174.329 135.476 179.654 139.419 186.056 140.959 191.157 141.907 197.757 143.398 201.131 144.421 203.651 145.39 206.76 16 31 154 114 151.617 117.535 150.904 121.797 150.278 127.822 149.909 134.292 149.518 140.613 149.576 146.978 150.072 153.487 150.971 159.957 151.514 168.082 151.275 178.122 150.514 184.673 149.398 193.28 151.155 196.517 152.488 199.034 153.639 202.219 16 31 154 124 151.878 128.746 150.476 133.267 149.722 137.857 148.954 141.864 148.116 147.812 148.388 153.776 148.797 159.99 149.488 166.512 150.158 174.494 150.262 184.319 151.358 189.806 154.101 196.279 155.938 199.514 156.874 202.368 158.304 205.411 16 31 154 134 152.238 139.477 150.531 144.588 149.243 149.672 148.481 153.502 148.014 157.507 148.405 161.422 148.747 165.515 149.268 169.935 150.285 176.614 151.795 185.563 153.528 190.763 156.271 196.901 158.201 200.086 159.143 202.918 160.767 205.632 16 31 184 134 184.33 139.727 186.352 144.925 188.902 151.79 196.06 158.314 202.657 166.333 214.882 180.443 220.669 193.021 219.503 199.344 219.489 200.636 219.33 199.594 217.657 198.104 212.37 197.252 208.965 197.915 207.725 199.046 204.2 199.477 16 31 194 134 202.186 133.43 217.573 131.565 219.733 147.177 220.395 162.61 222.429 177.431 223.181 191.75 222.092 202.92 220.011 208.874 219.696 210.063 220.725 209.268 220.443 208.088 220.07 207.776 219.826 208.7 220.631 209.708 220.268 210.096 16 31 194 144 198.12 147.509 211.192 146.372 213.418 158.213 215.366 170.96 212.259 184.682 217.347 198.078 216.393 209.363 216.036 217.189 215.032 218.65 215.164 217.37 215.169 216.217 215.467 216.046 214.739 216.275 215.952 217.196 215.394 217.719 16 31 194 154 193.283 161.307 198.047 165.008 211.893 167.34 213.804 177.843 188.743 189.173 183.615 192.706 173.131 193.803 174.903 198.804 171.673 202.339 169.827 205.528 169.588 209.233 169.238 211.62 169.393 213.761 169.743 215.952 169.833 217.566 16 31 194 164 194.438 165.42 197.706 168.676 210.635 170.947 216.407 179.624 193.751 191.046 182.151 194.416 171.642 195.447 173.016 200.206 170.677 203.854 168.801 207.043 168.723 210.819 168.681 213.32 168.891 215.532 169.353 217.724 169.124 219.175 16 31 204 124 207.291 130.941 220.185 132.521 221.476 149.099 220.298 165.347 219.628 179.999 220.798 194.081 220.14 205.265 217.736 211.611 216.931 212.899 217.615 212.121 217.387 210.969 217.393 210.818 217.16 211.59 218.417 212.621 218.072 213.079 16 31 204 134 206.466 141.528 216.609 143.685 215.163 159.169 215.9 172.333 209.267 185.828 205.06 194.603 198.647 198.398 196.585 202.434 199.099 207.979 204.829 212.567 209.874 215.926 210.134 215.837 211.49 217.226 212.734 218.336 213.048 219.461 16 31 204 144 210.529 145.021 210.643 155.023 205.533 169.385 213.856 176.999 195.223 188.82 180.744 192.042 175.402 194.274 176.776 199.156 174.437 202.804 172.731 206.138 172.259 209.736 171.865 212.088 171.426 214.022 171.155 216.005 171.013 217.469 16 31 214 124 219.055 130.706 221.977 143.047 218.544 160.866 216.288 175.171 200.994 187.431 184.631 190.448 174.677 192.027 177.079 197.264 173.849 200.799 172.619 204.253 172.142 207.801 171.784 210.178 171.133 212.096 170.862 214.109 171.41 215.853 16 31 214 134 219.118 139.077 220.602 150.813 215.131 167.782 213.587 180.076 188.525 191.045 183.839 194.581 173.322 195.611 174.204 200.37 171.865 204.018 169.988 207.171 169.874 210.94 169.524 213.327 169.609 215.459 169.837 217.525 169.608 218.976 16 31 224 134 223.684 135.155 226.346 146.912 223.328 165.182 218.12 181.29 202.826 193.094 185.911 195.644 175.427 197.163 175.678 201.831 174.309 205.479 172.781 208.736 172.428 212.439 171.995 214.765 171.794 216.728 171.602 218.702 171.414 220.104 16 31 224 154 223.997 154.163 224.237 154.834 222.785 171.531 219.046 183.827 211.07 196.638 195.051 200.766 185.909 202.81 184.47 206.854 186.389 212.094 194.137 217.61 198.41 220.672 201.591 222.578 203.63 224.448 205.512 226.083 205.49 227.249 16 31 19.799 118.794 18.1817 122.578 16.8038 126.369 15.2014 130.521 13.7481 134.487 12.6066 138.974 12.1315 143.584 12.1964 148.948 12.6805 154.428 13.7246 160.471 15.536 166.644 16.9348 172.708 18.1985 179.399 19.0748 186.427 19.322 194.481 19.299 202.579 16 31 19.799 132.936 17.8187 135.963 16.1866 139.24 14.3173 143.019 13.074 147.094 12.0119 151.709 11.4521 155.366 11.1907 160.553 11.46 165.633 12.1552 171.028 13.7975 176.81 15.0228 182.908 15.6262 188.267 16.3528 194.935 16.4909 202.792 16.447 211.58 16 31 19.799 147.078 19.8078 147.134 19.2239 149.217 18.1475 152.773 17.5608 156.904 16.4763 161.347 15.8381 165.39 15.5837 170.705 15.7792 175.864 16.281 181.187 17.744 187.048 18.1396 192.582 18.0166 197.184 18.3092 202.763 18.1599 209.111 18.0261 217.343 16 31 33.9411 90.5097 32.36 94.7694 30.8389 98.956 29.7112 103.199 29.068 107.497 28.5138 111.954 28.2328 116.877 28.5541 122.095 29.2893 128.006 30.1992 134.147 31.9219 140.411 33.952 147.149 35.554 154.354 36.177 162.286 36.057 170.622 36.144 178.045 16 31 33.9411 104.652 32.4818 108.956 31.3928 113.02 30.1253 117.523 28.9215 122.012 28.0366 126.596 27.6354 131.582 27.6312 136.908 28.1873 142.905 29.4278 148.936 31.3641 155.233 32.9751 162.167 34.3256 169.531 35.3215 177.126 35.5335 185.281 35.536 192.939 16 31 33.9411 118.794 32.4612 122.739 31.2972 126.922 29.9442 131.261 28.7297 135.783 27.7628 140.52 27.297 145.349 27.2236 150.776 27.8091 156.457 29.043 162.608 30.9654 168.979 32.6415 175.663 33.9898 182.621 34.9629 189.93 35.1639 197.913 35.0632 206.885 16 31 33.9411 132.936 32.3616 136.903 31.1915 141.304 29.8553 145.579 28.5902 150.174 27.5642 154.92 27.0515 159.59 26.7451 164.951 27.2932 170.551 28.4252 176.722 30.4076 183.089 32.0256 190.036 33.2832 196.883 34.2882 204.007 34.4786 211.917 34.2106 221.04 16 31 33.9411 147.078 32.5126 150.325 31.6949 154.538 30.7741 158.972 29.9126 163.673 28.7805 168.287 28.1423 172.928 27.9271 178.298 27.845 183.393 28.3063 189.009 28.6488 194.871 28.5153 200.58 28.7165 206.31 28.3029 212.596 27.6854 220.23 27.2246 229.29 16 31 48.0833 90.5097 46.7616 94.7401 45.5136 98.9284 44.1948 103.321 43.4675 107.758 42.8059 112.387 42.477 117.46 42.6451 122.815 43.3851 128.898 44.313 135.217 46.2446 141.599 48.0568 148.25 49.0259 154.991 49.8533 162.444 48.5436 170.212 48.2949 177.532 16 31 48.0833 118.794 46.7948 122.946 45.5027 127.313 44.3734 131.768 43.0502 136.35 42.0556 141.211 41.7412 146.157 41.8002 151.448 42.49 157.228 43.5788 163.45 45.4182 170.173 46.7667 177.381 48.2116 184.566 49.2257 192.168 49.6996 200.058 50.0184 208.321 16 31 48.0833 132.936 47.0237 137.012 45.6936 141.473 44.4805 145.77 43.077 150.449 42.1374 155.253 41.8387 160.004 41.7239 165.327 42.537 171 43.6812 177.252 45.7669 183.813 47.1574 190.863 47.987 198 48.6834 205.478 48.8125 213.377 49.6771 222.349 16 31 48.0833 147.078 47.5129 150.165 46.9639 153.479 45.4554 157.757 43.9327 162.041 42.9403 166.385 42.7467 170.921 43.2032 176.001 44.658 181.173 45.5368 187.171 44.9962 193.78 43.0277 200.836 42.5042 207.582 42.2235 214.632 39.9921 222.494 42.897 231.691 16 31 48.0833 161.22 48.07 161.255 48.059 161.309 47.4593 162.39 46.6021 165.319 45.7626 168.812 45.9702 172.183 46.7324 176.225 48.1871 181.046 48.8891 187.02 47.0116 193.734 44.8853 201.111 43.7506 207.915 42.6895 214.777 40.458 222.639 43.363 231.836 16 31 62.2254 90.5097 61.3335 94.7799 60.0003 99.2643 58.9311 103.724 58.7653 108.173 58.4834 112.761 59.3828 117.452 58.9039 121.889 59.619 126.082 61.1796 130.97 63.9027 136.156 66.9863 140.563 69.2388 145.235 70.349 149.628 70.7665 153.675 71.0594 156.794 16 31 62.2254 104.652 60.9115 109.043 59.4205 113.848 58.1957 118.541 56.9808 123.389 55.8657 128.279 55.4952 133.563 54.7376 138.735 54.9135 144.521 55.7242 150.921 57.4797 157.597 59.3361 164.311 60.3792 171.584 61.2435 178.92 61.1861 186.512 60.7481 193.768 16 31 62.2254 118.794 60.7618 123.391 59.2974 128.221 57.8672 133.204 56.2495 138.389 54.836 143.685 54.0456 148.955 53.6551 154.401 53.8913 160.368 54.3948 166.892 55.2052 173.883 55.5032 181.711 55.9934 189.398 56.6686 197.052 57.7017 204.677 58.9303 212.704 16 31 62.2254 132.936 61.8392 136.664 60.9402 140.429 60.0906 144.236 58.7039 148.394 57.4167 153.002 56.8074 157.694 56.4812 163.131 56.4999 168.961 56.3101 175.375 55.7098 182.244 54.6993 190.025 54.5627 197.712 54.8929 205.271 55.6905 212.547 57.1475 221.367 16 31 62.2254 161.22 62.1898 161.332 62.1494 161.423 61.8137 162.303 60.7769 165.044 59.8162 168.314 59.4191 171.572 58.9114 175.785 57.6623 180.394 56.3956 186.374 53.6157 192.92 50.674 200.764 49.3748 207.96 49.1287 214.984 47.5013 222.595 55.4291 230.765 16 31 76.3675 90.5097 76.222 94.9046 76.3871 100.199 76.7238 105.01 76.0656 108.961 75.1402 112.83 75.4828 116.429 75.6502 119.443 76.6807 122.121 79.0884 124.725 82.6503 128.275 84.8826 132.844 86.1373 137.1 87.153 141.349 88.0491 144.747 88.411 147.029 16 31 76.3675 118.794 74.7914 122.599 73.2447 127.975 71.7128 132.268 70.1661 136.737 68.8168 141.92 67.7506 146.564 66.8777 151.298 66.6706 157.256 64.9687 162.858 64.9601 170.181 65.1926 178.284 65.553 186.501 66.2744 194.574 67.139 202.188 67.7542 210.005 16 31 76.3675 132.936 76.2324 136.493 75.2376 140.416 74.3237 144.343 72.7393 148.367 71.0841 152.864 69.2374 156.834 67.641 161.435 62.8588 165.493 61.4117 171.495 60.3597 178.169 59.4819 185.957 59.1088 193.811 59.4262 201.413 60.5007 208.73 61.9609 217.214 16 31 76.3675 147.078 76.353 150.585 76.201 154.633 75.576 158.709 75.0582 163.126 74.7446 167.759 74.2166 172.396 71.7712 177.426 64.9779 181.482 57.718 183.051 55.2987 189.996 52.9422 197.881 51.6997 205.508 51.7859 212.799 51.222 220.35 55.1409 229.069 16 31 76.3675 161.22 76.3376 161.453 76.2945 161.781 75.8563 164.854 75.4668 168.459 75.2211 171.336 74.9387 175.844 72.9329 180.874 65.6742 185.573 56.5811 185.839 53.8012 192.385 50.8595 200.229 49.5603 207.425 49.6464 214.716 48.9014 221.627 53.81 230.347 16 31 90.5097 76.3675 90.2001 81.9199 89.6758 88.5926 89.4553 94.9079 89.1464 98.7292 88.8592 102.331 88.6404 105.819 88.3123 108.898 88.2938 111.725 88.0756 114.489 88.2698 117.29 88.3756 121.023 88.5147 124.942 88.6594 128.81 88.8055 132.203 88.847 134.407 16 31 90.5097 90.5097 89.5382 95.9451 88.9396 102.456 88.7161 108.394 88.279 112.54 87.4584 116.781 86.9759 120.749 85.9224 124.276 86.0548 127.128 84.7908 129.851 87.9646 133.386 91.4541 137.721 94.0623 141.922 96.0043 145.884 97.1196 148.849 97.0167 151.014 16 31 90.5097 132.936 91.8783 135.384 93.6682 138.88 94.6105 143.19 94.4625 148.127 95.8053 154.054 94.9018 159.216 77.1629 161.826 65.6727 163.664 63.5587 169.266 62.9584 176.116 62.4306 184.119 62.294 192.021 62.7033 199.746 63.716 207.033 65.2105 214.927 16 31 90.5097 147.078 90.5455 149.128 90.5574 152.103 90.4259 156.326 90.3204 161.161 90.5943 166.484 90.3548 171.539 77.4647 175.904 65.7025 177.86 59.4484 180.726 57.3451 187.272 55.4047 195.115 54.2555 202.918 54.4402 210.388 54.8464 217.365 56.7992 226.413 16 31 90.5097 161.22 90.7765 161.974 90.6471 162.976 90.5349 164.957 90.59 169.141 90.4852 173.76 89.9797 178.579 79.5705 184.61 70.3869 190.006 59.7484 188.598 55.5334 192.957 52.5917 200.796 51.2925 208.202 51.3225 214.818 50.5774 221.729 54.4963 230.448 16 31 90.5097 175.362 90.5857 175.144 90.5295 175.213 90.4761 175.516 90.4981 175.484 90.3982 176.32 89.8927 180.916 80.0445 188.415 72.8916 192.968 61.8095 191.628 55.9198 191.572 52.978 199.411 51.6788 206.817 51.765 214.108 51.0199 221.019 54.9388 229.738 16 31 90.5097 189.505 90.5155 189.577 90.4795 189.523 90.522 189.611 90.4897 189.708 90.4941 189.672 90.562 188.844 80.7641 201.698 68.7107 212.619 60.0343 217.082 53.1458 216.354 49.3722 215.791 48.7378 215.391 48.7195 218.216 47.0921 225.093 57.4461 230.869 16 31 104.652 76.3675 105.091 82.0788 105.064 89.2009 109.205 96.5269 106.451 100.167 104.842 103.864 104.687 107.531 104.542 111.305 106.312 115.326 107.802 120.28 109.812 125.09 113.105 129.376 115.162 133.218 116.239 136.391 116.703 138.555 116.445 140.583 16 31 104.652 104.652 102.845 110.399 102.237 116.809 100.797 123.693 100.217 130.483 100.993 138.26 96.7835 144.895 92.7691 148.777 90.71 151.771 90.7091 153.655 91.1416 155.546 91.2989 157.675 91.4725 161.062 91.0393 166.195 90.2479 170.271 89.3183 173.093 16 31 104.652 118.794 103.888 123.002 103.62 128.237 102.828 134.267 102.34 140.59 102.108 146.666 94.4995 151.521 80.6741 153.145 73.5848 156.026 71.4697 160.719 71.4611 168.103 71.965 176.189 72.4185 184.362 72.951 192.311 73.2981 199.869 73.3467 207.144 16 31 104.652 132.936 106.151 134.66 106.984 136.546 106.682 139.331 106.525 145.653 104.468 151.585 96.5846 155.969 79.089 158.125 67.7861 160.19 65.0008 165.382 64.9922 172.705 64.5449 180.867 64.5318 189.039 65.0756 196.973 66.014 204.483 67.3374 212.376 16 31 104.652 147.078 104.493 148.606 103.983 150.363 102.874 152.756 102.353 157.174 99.2158 162.279 94.6308 166.819 79.5895 170.596 65.2753 171.483 61.2009 176.139 59.7225 182.701 58.0256 190.54 57.2509 198.415 57.4857 205.99 58.3242 212.967 59.7913 222.061 16 31 104.652 161.22 104.114 162.47 103.43 164.175 102.683 166.02 102.439 169.377 100.688 174.164 93.5415 182.777 83.6696 194.652 71.6732 202.215 62.7301 205.778 55.7921 204.48 52.0978 205.076 50.8776 210.355 50.9076 216.971 50.1496 222.947 56.8419 231.052 16 31 118.794 76.3675 118.355 82.9958 118.511 90.8921 124.749 98.3669 123.757 102.046 122.869 105.655 122.716 109.571 123.014 113.601 123.772 118.238 124.938 123.195 125.972 128.641 125.762 133.152 124.676 137.048 124.339 140.214 124.547 142.414 124.808 144.406 16 31 118.794 90.5097 118.128 96.9111 118.38 104.7 120.998 112.447 118.68 116.704 116.932 121.139 113.257 125.865 112.721 129.721 113.268 133.845 114.359 139.138 114.25 144.479 116.68 149.288 117.715 153.372 118.723 156.336 119.402 158.432 119.413 160.475 16 31 118.794 104.652 117.069 111.142 115.624 118.489 113.652 127.687 110.336 136.206 107.117 145.527 98.2501 150.221 84.932 152.831 77.5652 155.665 74.7798 159.819 74.6232 166.885 74.9625 174.84 74.8527 182.368 75.5263 189.877 75.4396 197.435 75.1668 204.424 16 31 118.794 118.794 116.7 125.163 115.138 131.868 114.254 137.033 112.649 145.422 107.416 150.473 98.7725 154.783 81.2768 157.393 69.153 159.749 67.4511 165.35 67.1337 172.673 66.6142 180.963 66.4964 189.206 67.0289 197.25 67.8757 204.731 68.9746 212.476 16 31 118.794 132.936 118.118 138.856 115.648 142.857 115.842 145.255 124.278 155.097 120.245 159.716 114.408 163.365 104.052 173.944 91.8932 177.894 85.0803 189.684 78.7175 197.631 74.8717 202.819 73.1948 208.97 72.791 214.464 72.8705 219.439 74.6004 222.173 16 31 118.794 147.078 120.083 150.026 120.586 152.893 119.827 155.561 129.17 165.463 127.406 172.297 122.744 178.749 113.315 189.227 102.161 194.892 92.3806 206.42 84.7833 216.106 80.6869 219.358 78.7011 223.484 80.2706 225.996 83.297 228.344 87.8548 229.528 16 31 132.936 90.5097 132.581 96.6466 135.332 104.658 134.522 109.062 132.818 112.287 131.685 115.638 130.981 119.235 130.985 123.227 131.723 127.865 132.695 132.956 133.256 138.105 132.7 142.479 131.808 145.894 131.575 148.513 131.761 150.566 132.136 152.511 16 31 132.936 118.794 130.466 125.987 127.849 135.65 124.558 144.976 131.814 154.618 130.695 158.409 126.428 160.954 124.13 166.464 122.901 171.186 120.771 176.58 123.933 183.555 125.954 188.27 125.598 192.749 127.258 196.973 128.49 199.144 129.207 201.531 16 31 132.936 132.936 129.943 138.674 126.729 143.83 123.675 149.515 132.038 159.369 130.934 163.316 126.775 165.846 122.346 171.974 116.42 176.696 114.193 182.74 108.205 191.73 97.437 200.727 98.7153 204.27 100.702 208.782 105.377 211.08 108.94 213.011 16 31 147.078 118.794 144.324 124.047 142.266 131.06 141.152 139.975 140.49 146.236 139.538 150.636 138.18 154.777 138.54 160.285 139.106 165.862 139.47 171.896 139.212 179.38 137.441 185.084 136.598 193.411 137.948 196.733 139.007 198.821 139.714 201.717 16 31 147.078 132.936 144.727 138.39 142.606 143.614 141.1 150.179 142.34 156.441 141.441 160.467 138.86 163.11 138.903 168.044 139.258 173.061 139.776 178.502 143.039 185.801 143.766 191.123 144.451 198.294 146.056 201.577 147.177 204.083 148.138 207.166 16 31 161.22 132.936 158.199 138.442 154.897 143.141 151.501 147.658 149.266 151.426 148.465 155.554 149.256 159.73 151.497 165.243 153 170.735 154.471 177.9 157.068 185.572 158.51 190.216 161.299 195.038 163.433 198.145 164.794 200.863 166.406 203.502 16 31 189.505 132.936 195.502 135.441 214.226 130.829 223.151 142.171 225.709 158.944 226.122 176.106 226.23 191.391 225.353 202.719 224.172 208.732 225.455 209.994 226.745 208.987 226.95 207.93 227.074 207.567 226.988 208.383 226.878 209.136 226.718 209.368 16 31 203.647 118.794 209.757 124.273 213.288 135.854 222.763 147.195 223.529 163.805 222.956 179.72 222.794 194.49 221.69 205.873 219.864 212.354 219.464 213.58 219.883 211.998 219.812 210.908 219.946 210.829 219.82 211.327 220.288 212.093 219.461 212.451 16 31 203.647 132.936 206.767 139.859 207.448 150.223 212.239 159.885 210.185 173.08 203.184 184.876 188.598 189.666 179.091 191.688 179.187 196.193 178.778 201.116 179.66 205.154 179.989 208.462 178.787 210.378 177.325 212.005 176.419 213.77 175.571 215.079 16 31 203.647 147.078 207.533 151.062 209.207 159.682 207.797 171.953 212.161 181.162 189.835 192.228 186.701 195.992 176.22 197.347 175.474 201.507 175.921 206.658 176.136 210.172 174.85 213.441 173.458 215.373 172.729 217.198 172.152 219.083 171.31 220.422 16 31 217.789 118.794 223.02 124.817 225.378 137.554 222.754 155.674 217.439 172.282 209.291 186.067 201.146 194.136 191.611 196.709 189.001 200.416 191.692 206.062 199.634 211.997 205.776 215.822 208.373 217.21 208.723 218.154 210.227 219.47 210.11 220.475 16 31 217.789 147.078 218.441 148.538 217.661 158.646 212.595 173.959 217.24 182.577 204.516 195.091 183.511 196.871 173.676 198.277 173.012 202.488 173.069 207.421 171.246 210.343 169.96 213.622 168.777 215.673 168.462 217.6 168.357 219.589 167.521 220.975 16 31 8 108 6.60296 111.547 5.55732 115.06 4.52426 117.666 3.67681 120.62 2.95928 124.015 2.83829 127.781 2.95262 132.162 3.52759 136.844 4.50669 141.214 5.8989 144.3 6.58307 145.515 7.24064 146.293 7.41219 146.523 7.42044 146.529 7.421 146.529 16 31 8 148 7.96019 148.053 7.76026 148.511 7.15202 150.219 6.61469 153.091 5.80119 157.443 5.50303 160.094 5.46754 163.714 5.66646 167.91 6.18787 172.415 7.92292 178.295 8.88607 184.232 9.64907 189.987 10.4052 196.353 10.6014 204.361 10.6527 213.004 16 31 28 88 26.3449 92.3707 24.7623 96.5578 23.5696 100.831 22.7249 105.094 22.2057 109.575 22.1796 114.595 22.5925 119.841 23.6195 125.595 24.8018 131.69 26.6552 137.789 28.3036 144.131 29.4571 151.327 29.9565 159.28 30.1444 167.812 30.1821 175.456 16 31 28 108 26.479 112.233 25.2514 116.318 23.8975 120.788 22.6713 125.296 21.79 129.879 21.4186 134.879 21.4991 140.325 22.1417 146.222 23.3498 152.389 25.403 158.701 27.0701 165.439 28.4257 172.693 29.4918 180.207 29.7917 188.428 29.8028 196.333 16 31 28 128 26.357 131.851 25.0635 136.137 23.6318 140.545 22.3699 145.198 21.4244 149.975 20.9974 154.812 20.8551 160.231 21.5201 165.82 22.7063 171.765 24.7805 177.986 26.4327 184.505 27.8057 191.24 28.8512 198.36 29.0704 206.446 28.9681 215.546 16 31 28 148 26.4489 151.567 25.3175 155.826 24.1056 160.513 23.0705 165.217 21.8981 169.941 21.2686 174.559 20.8821 179.506 20.7346 184.726 21.0596 190.398 21.7403 195.769 22.1491 201.447 23.0807 207.798 23.6269 214.77 23.3502 222.497 22.9973 231.618 16 31 28 168 27.5724 168.825 27.4394 169.525 26.8917 171.664 26.4434 173.44 25.3589 177.584 24.6202 181.1 24.2281 185.36 23.6329 190.063 23.0744 193.817 24.0215 199.667 23.779 205.01 24.2009 210.901 23.8118 216.997 23.5351 224.724 23.1822 233.845 16 31 48 88 46.5805 92.3426 45.2847 96.5997 43.8849 101.044 43.0718 105.497 42.503 110.113 42.2281 115.208 41.9219 120.437 42.798 126.426 43.5258 132.769 45.3454 139.137 47.2394 145.742 48.072 152.665 48.023 159.96 46.9482 167.954 46.0479 175.313 16 31 48 108 46.6162 112.326 45.342 116.656 44.0505 121.209 42.9846 125.762 42.1137 130.423 41.7207 135.616 41.8118 140.98 42.4977 146.904 43.5448 153.253 45.5334 159.765 47.354 166.495 48.8156 173.762 49.7866 181.25 50.0835 189.185 50.1974 196.707 16 31 48 128 46.9107 132.007 45.6666 136.307 44.4833 140.744 43.2256 145.387 42.2259 150.189 41.8671 155.115 41.7779 160.537 42.4967 166.265 43.4502 172.486 45.262 179.045 46.7454 185.912 47.8843 193.028 48.8587 200.363 49.0701 208.169 49.7613 217.461 16 31 48 148 46.818 151.701 45.6858 155.525 44.3373 160.026 43.1097 164.527 42.0338 169.153 41.5815 174.102 41.5125 179.464 41.6286 185.288 41.608 191.586 40.3074 198.088 38.1014 204.565 37.5285 211.157 36.5066 217.913 35.6152 225.821 37.6255 235.131 16 31 48 168 47.8403 168.289 47.7735 168.781 47.0901 170.088 46.887 171.481 46.2004 174.516 45.7324 178.228 45.7042 182.519 45.5472 188.024 43.9537 194.381 40.9698 200.883 38.7637 207.36 37.4781 213.711 36.4562 220.467 35.5648 228.106 41.1063 237.256 16 31 68 68 68.0612 71.7276 67.636 76.1504 66.9894 80.5854 66.6594 84.2531 66.1146 87.4686 66.0355 90.4057 65.9659 92.5255 66.144 93.3817 66.2913 93.7561 66.2654 93.9364 66.2842 94.208 66.3258 94.7914 66.3548 94.1943 66.2862 94.2345 66.4114 94.2995 16 31 68 88 67.1718 92.4032 65.9856 97.1951 64.8468 101.825 64.825 106.164 64.5165 110.529 65.8416 114.779 66.5136 118.532 68.1573 121.554 71.7479 124.506 75.8894 128.276 78.9136 132.681 81.2906 136.775 83.346 140.759 84.9028 143.92 85.7568 146.267 16 31 68 108 66.6949 112.442 65.5013 117.336 64.0668 122.282 63.2199 127.123 62.0586 132.052 61.3332 137.273 60.4744 142.546 59.9054 148.501 59.9501 155.169 60.6123 162.506 61.4325 169.85 62.3568 177.423 63.1783 184.867 63.6537 192.593 63.8359 200.235 16 31 68 128 67.3026 131.467 66.4484 135.336 65.446 139.292 64.4646 143.664 63.4624 148.46 63.1577 153.306 62.9798 158.808 60.9933 164.554 59.9349 171.222 58.698 178.465 57.3047 186.374 57.2256 193.88 57.8697 201.393 58.3703 209.035 60.2036 217.703 16 31 68 148 67.7136 151.528 66.9881 155.165 66.3938 159.011 65.4212 163.192 64.5035 167.606 63.9149 172.227 62.6565 177.598 57.678 182.702 54.2958 189.582 50.0422 196.952 46.6003 204.089 45.6027 210.71 44.8071 217.415 43.6282 225.194 50.2149 234.506 16 31 68 168 67.8848 168.337 67.7206 169.029 67.3529 170.828 66.9048 172.635 66.4868 176.439 66.0742 180.922 64.752 185.961 59.9824 191.243 52.5137 196.302 46.9928 202.804 42.9454 208.658 41.6599 215.01 39.9203 221.426 38.7413 229.125 49.0481 237.979 16 31 68 208 68.1074 207.657 68.2418 207.769 68.102 207.82 68.0804 207.813 68.0516 207.7 68.0021 207.88 67.7617 208.142 59.5894 221.996 46.0047 235.686 33.6624 244.512 27.7087 244.194 25.5244 241.95 27.409 239.784 37.1935 232.901 47.9784 240.083 16 31 88 68 88.0223 73.1025 87.8019 79.435 87.3875 85.6295 87.1693 89.3454 86.9751 92.8284 86.9158 96.1094 86.7478 99.0014 86.7249 101.651 86.6515 104.271 86.8214 107 86.9246 110.163 86.9355 113.51 86.988 115.807 86.9923 117.541 87.0241 118.039 16 31 88 88 87.438 93.3394 87.0673 99.7448 87.1677 105.645 86.7481 109.526 85.8678 113.57 85.3057 117.409 84.0645 120.81 83.9114 123.784 83.478 126.9 86.9273 130.542 90.4076 134.789 93.0605 138.813 95.408 142.678 96.9245 145.64 97.206 147.963 16 31 88 128 89.4863 130.731 90.6689 134.521 90.1188 138.747 90.3874 143.867 93.0588 149.772 90.4151 153.957 74.3477 155.754 65.5209 159.61 64.2524 166.176 63.0902 173.632 62.4773 181.615 62.491 189.752 63.0697 197.499 63.7604 205.186 65.5298 213.287 16 31 88 148 88.5159 150.595 88.5772 153.82 88.2017 157.806 88.1313 162.5 87.9271 167.548 86.2989 172.372 73.5885 176.702 63.1966 179.748 56.9183 184.25 53.8037 191.457 51.4037 198.87 50.439 205.738 50.7521 212.735 50.2114 220.22 52.7442 230.072 16 31 88 168 88.0781 169.059 87.8031 170.658 87.5962 172.533 87.4857 175.842 87.1467 179.884 85.4774 184.577 74.7768 189.86 64.5036 195.676 54.7937 199.603 48.2022 204.057 44.1548 209.911 42.8693 216.263 41.2544 222.327 40.0755 228.527 50.3823 237.381 16 31 88 188 87.978 187.999 87.9701 188.048 87.976 188.386 87.9858 188.315 87.8787 188.445 86.53 191.121 75.5336 205.556 63.0334 222.565 47.5217 239.275 34.8325 249.589 27.6546 251.203 26.1498 246.886 30.8106 237.781 40.0352 231.852 53.2752 237.42 16 31 108 88 107.556 94.133 107.901 101.424 112.02 108.609 108.991 112.398 106.776 117.071 105.307 121.601 104.977 125.361 108.221 129.088 110.003 134.21 111.855 139.489 114.731 144.176 115.788 148.052 116.741 151.026 117.15 153.167 117.044 155.237 16 31 108 108 106.34 114.134 105.595 120.324 104.992 126.596 103.318 131.863 101.687 137.508 92.6684 142.354 78.2654 145.719 73.3044 150.004 71.8172 156.367 71.9277 164.044 72.5567 171.998 73.0841 180.006 73.8783 187.621 74.4173 195.307 74.6231 202.563 16 31 108 128 109.547 130.612 110.095 133.141 109.911 137.692 108.526 143.126 102.455 148.343 94.6098 151.668 78.5424 154.934 67.9576 158.148 65.6625 164.348 65.047 171.846 64.4342 179.828 64.4479 187.966 65.0266 195.712 65.7173 203.4 67.4867 211.5 16 31 108 148 110.734 149.95 111.657 152.273 112.202 154.931 118.753 161.88 114.769 166.867 108.548 170.944 98.4299 181.36 86.3483 186.037 77.6047 190.953 71.3509 195.068 67.5017 200.601 65.5799 207.356 65.3921 213.647 64.4621 220.638 66.3209 229.258 16 31 128 88 128.11 94.4715 130.373 102.402 130.613 108.236 128.941 111.947 127.4 115.663 126.631 119.644 126.493 123.311 127.227 127.676 128.033 132.802 128.15 138.108 127.219 142.717 126.485 146.426 126.398 149.258 126.579 151.306 126.858 153.251 16 31 128 108 126.14 113.561 124.641 119.358 123.158 126.292 120.29 132.02 115.167 136.804 108.266 142.01 96.4339 147.488 85.4374 151.738 82.0954 157.224 81.5851 162.939 81.5351 169.616 81.9394 176.526 82.3911 183.525 82.2306 190.22 82.1837 195.943 16 31 128 128 125.231 133.536 122.69 136.944 119.973 142.854 125.36 151.11 122.386 155.831 117.186 157.936 107.472 165.887 95.0367 171.551 90.3064 181.21 84.4483 190.063 80.2577 195.915 78.5621 201.685 77.8759 207.691 77.5886 213.884 79.37 218.406 16 31 128 148 125.58 153.138 123.629 156.004 120.798 159.913 127.349 169.248 126.827 175.787 122.848 180.133 115.095 185.774 103.859 190.599 94.996 201.588 86.5999 212.253 81.7144 217.949 79.7737 223.025 81.2852 226.22 84.4319 228.336 89.7736 228.898 16 31 148 128 145.508 133.208 142.963 137.859 140.926 144.786 142.491 152.477 141.997 157.064 139.701 159.915 139.812 164.898 140.532 170.269 140.855 176.25 142.148 184.278 141.535 190.084 141.636 197.539 143.287 200.843 144.403 203.32 145.241 206.356 16 31 188 128 192.609 131.214 209.34 126.761 221.954 129.606 224.755 139.105 225.129 152.98 225.982 167.951 227.507 179.55 226.569 184.957 227.151 186.032 227.573 185.208 227.026 184.502 226.851 184.35 227.024 184.902 227.21 185.511 226.911 185.623 16 31 208 128 214.216 132.541 214.554 146.513 209.408 164.599 212.816 175.532 201.632 187.808 191.69 192.597 180.706 193.706 177.321 196.935 176.714 201.963 177.135 205.804 177.153 209.2 175.85 211.098 174.741 212.74 174.167 214.432 173.318 215.865 16 31 208 148 211.593 152.912 212.039 161.954 210.074 175.203 208.792 185.54 192.799 195.625 182.674 198.63 171.69 199.507 167.716 202.385 166.629 207.412 165.037 210.443 164.567 213.919 163.355 216.132 163.253 218.481 162.984 220.718 163.023 222.55 16 31 11.3137 96.1665 9.84962 100.272 8.59065 104.242 7.26739 108.513 6.23441 112.497 5.51567 116.646 5.32888 121.293 5.57193 126.357 6.1961 131.95 7.29822 138.006 9.0693 143.922 10.5777 150.303 11.7269 157.402 12.4642 164.879 12.8038 173.064 12.9669 180.829 16 31 11.3137 152.735 10.6352 154.991 9.7586 158.658 8.67623 162.678 7.78692 166.265 7.16491 169.525 6.81859 173.022 6.71609 176.146 7.01981 181.014 7.59586 186.07 8.98111 191.07 10.0903 197.235 11.0756 203.347 11.5706 209.938 12.0379 218.058 11.9415 226.155 16 31 39.598 67.8822 38.2604 71.9377 36.7143 75.8998 35.0329 79.8349 33.9309 83.5766 33.3698 87.3139 33.3613 91.183 34.1472 95.075 36.0129 99.2247 37.8819 104.214 40.62 107.496 43.5789 110.43 45.7397 114.338 47.3926 115.664 47.4445 116.017 47.7075 116.086 16 31 39.598 96.1665 38.0327 100.423 36.6339 104.587 35.1893 109.003 34.0293 113.37 33.216 117.85 32.9639 122.851 33.2559 128.066 34.0978 133.887 35.2981 140.222 37.3304 146.669 39.1592 153.346 40.4767 160.626 41.3762 168.143 41.6619 176.194 41.7703 183.774 16 31 39.598 124.451 38.2017 128.455 36.9106 132.602 35.6483 137.055 34.4097 141.529 33.4823 146.164 33.2027 151.103 33.2328 156.526 33.9462 162.298 35.1155 168.545 37.2806 175.004 38.9319 181.786 40.3799 188.861 41.3915 196.224 41.6417 204.327 41.5901 213.148 16 31 39.598 152.735 38.1851 156.573 37.0322 160.661 35.7604 165.163 34.5385 169.649 33.4745 174.281 33.0275 179.22 32.9781 184.5 32.7804 190.203 31.2882 196.34 29.7157 202.564 28.5611 208.295 26.6422 214.152 24.9402 220.806 27.9335 229.324 31.7599 238.008 16 31 39.598 181.019 39.5832 181.279 39.43 181.96 39.3801 182.34 39.1406 184.335 39.174 185.961 38.7123 189.464 38.7507 194.237 38.1212 199.745 34.8288 205.783 31.803 211.283 27.1963 216.33 23.8187 221.731 22.6336 227.427 26.2213 235.232 33.7275 242.557 16 31 67.8822 67.8822 67.6079 72.2153 67.4841 77.0255 66.749 81.9075 66.2468 85.9127 65.7967 89.5937 65.7072 93.1779 65.686 95.9618 66.3424 98.6553 67.4132 101.53 69.8213 105.212 72.2915 109.271 73.6511 112.802 75.3495 116.089 76.175 118.883 76.9085 120.497 16 31 67.8822 96.1665 66.5105 100.507 65.9732 105.013 65.2687 109.72 64.2606 114.043 63.1104 118.458 62.1835 123.283 60.1157 128.209 58.9619 133.704 59.1484 140.133 60.5032 146.777 61.9036 153.481 62.5302 160.765 62.5272 168.097 61.6082 175.898 60.2814 183.336 16 31 67.8822 124.451 67.287 128.165 66.7415 132.051 66.2484 136.586 65.6788 141.049 65.4354 145.611 65.7154 150.312 64.1203 155.5 61.4825 161.515 58.1611 168.337 56.7249 175.712 56.1738 183.191 56.3055 190.847 56.5723 198.419 56.425 206.568 58.9711 215.514 16 31 67.8822 152.735 67.5631 156.031 67.0282 159.507 66.7622 163.579 66.8966 167.906 66.8279 172.408 66.5993 177.18 62.2584 182.252 54.7043 188.573 47.3407 195.2 41.503 201.484 37.1381 207.346 34.8293 213.422 32.9447 220.082 34.2509 228.563 38.0773 237.854 16 31 96.1665 67.8822 95.0608 73.7928 94.6539 80.9863 94.3839 88.165 93.8273 91.5056 93.1736 94.9892 92.7924 98.5733 92.4141 101.884 92.5279 105.059 92.5494 108.623 94.7643 112.237 97.1661 116.092 99.0732 119.655 100.234 122.639 101.154 125.034 101.394 127.105 16 31 96.1665 96.1665 95.3674 101.692 96.1845 107.91 97.9639 114.117 97.4431 117.429 95.2513 121.382 92.1609 126.089 87.681 130.451 85.6668 134.187 85.826 138.563 89.4631 142.754 92.8845 147.003 95.9244 150.935 98.6391 154.365 100.275 157.2 100.501 159.44 16 31 96.1665 124.451 97.109 127.274 97.6328 130.984 98.1211 135.497 98.6074 140.381 94.9225 144.359 87.8584 146.839 74.5515 151.179 68.308 156.674 64.9866 163.445 63.5505 170.991 62.9993 178.514 62.5693 186.368 62.8362 193.941 62.6771 201.858 64.9207 209.806 16 31 96.1665 152.735 96.8733 154.958 96.9795 157.444 96.1208 160.642 94.9411 165.178 88.7769 169.664 80.4909 172.911 69.0023 177.279 60.2274 182.673 53.4068 188.954 49.4146 195.819 45.5485 202.192 44.1331 208.55 43.283 215.285 43.1357 223.707 49.6999 233.029 16 31 124.451 67.8822 124.069 74.4696 125.967 82.2708 130.192 89.5019 128.837 92.6662 127.216 95.7561 125.961 99.5536 125.142 103.39 125.048 107.766 125.478 112.507 125.587 117.249 125.412 121.564 125.382 125.114 125.264 128.035 125.162 130.051 125.13 132.069 16 31 124.451 96.1665 124.073 102.111 124.988 108.384 127.07 114.712 125.158 118.163 122.812 122.214 121.093 126.855 120.349 130.778 121.007 134.916 121.674 139.834 120.855 144.844 119.381 149.071 118.645 152.564 118.719 155.329 118.827 157.354 119.011 159.412 16 31 124.451 124.451 122.257 128.317 119.744 131.996 118.627 137.393 116.628 142.938 112.804 147.661 106.616 149.86 95.7107 155.251 83.9931 159.971 79.5344 166.377 76.5313 174.086 73.2605 181.691 71.5482 189.545 70.6882 197.112 69.8488 205.029 72.12 212.478 16 31 152.735 124.451 149.62 128.218 146.284 132.12 142.526 136.291 140.574 140.971 139.606 145.478 139.8 148.603 140.302 152.964 141.041 158.048 141.987 164.368 143.387 172.497 142.58 178.2 142.722 185.402 144.233 188.983 145.245 191.364 146.139 194.451 16 31 209.304 124.451 215.256 128.111 217.35 139.413 221.032 154.453 221.254 169.757 220.053 184.603 219.787 198.381 218.709 209.035 217.238 215.076 216.507 216.187 215.963 214.975 214.88 214.179 214.425 214.204 214.195 214.953 214.101 215.678 213.091 216.271 16 31 209.304 152.735 212.77 158.142 215.73 165.017 213.312 178.148 207.089 193.034 196.961 203.231 185.258 206.662 171.877 207.321 167.857 210.142 168.932 215.529 171.77 220.096 172.703 223.752 171.781 225.917 171.669 227.924 171.147 229.939 170.954 231.693 16 31 16 96 14.6715 100.12 13.4944 104.168 12.4041 108.528 11.6239 112.779 11.5235 117.068 12.1219 121.786 12.9684 126.656 14.3454 132.104 15.91 138.246 18.2971 144.55 20.2644 151.008 21.7172 158.231 22.6757 165.731 23.099 173.801 23.0356 181.491 16 31 16 136 14.6495 139.878 13.4549 144.069 12.3474 148.587 11.4544 153.142 11.1447 157.785 11.3975 162.765 11.9204 168.041 12.905 173.809 14.1904 179.874 16.4399 186.061 18.0865 192.466 19.5361 199.1 20.4514 206.128 20.7816 215.02 20.1938 222.97 16 31 16 176 15.9702 176.409 15.7352 177.484 15.4171 179.891 14.7753 183.388 14.5192 186.685 14.5155 191.026 14.6991 194.838 15.0945 200.024 15.057 205.019 13.9265 209.643 11.7904 214.293 11.0458 218.448 11.0364 224.185 11.9991 231.492 11.7904 237.402 16 31 56 56 56.2799 59.9437 56.5912 64.1067 56.0839 68.5829 55.1349 72.6134 54.3301 76.3882 54.1274 80.1214 53.8627 83.3926 54.3331 86.2374 55.3192 89.8383 57.2444 93.6976 60.0606 97.6348 61.9267 101.028 64.3061 104.028 65.4925 106.153 66.4192 107.436 16 31 56 96 54.4772 100.194 53.1472 104.529 51.6417 109.103 50.5066 113.413 49.6159 117.79 49.2926 122.575 49.7047 127.319 50.5424 132.667 51.5333 138.828 53.9445 145.228 55.8858 151.719 56.9983 158.952 57.678 166.296 57.1603 174.12 56.4208 181.495 16 31 56 136 55.0037 139.866 54.0499 144.105 53.0913 148.578 52.6258 153.065 52.1834 157.77 52.1535 162.884 49.7291 168.439 46.8891 174.84 42.3737 181.198 40.114 187.332 40.439 193.945 40.9862 200.883 41.2635 208.143 41.2863 217.871 41.4398 225.876 16 31 56 176 55.9549 178.991 55.9099 182.529 55.5395 186.217 55.6222 190.479 55.2993 194.938 54.4027 200.052 47.8757 205.889 37.5402 212.777 26.0194 218.705 18.7494 223.466 13.7897 225.354 11.9938 228.268 11.9843 230.251 13.3122 233.944 13.2301 240.959 16 31 96 56 94.899 61.3801 94.3929 68.0598 94.0704 74.7259 94.0253 78.057 93.3168 81.3934 92.6863 84.8885 91.5255 88.514 91.1315 92.0351 90.7375 95.8597 91.9312 99.5869 93.0433 103.215 93.7353 106.464 94.4892 109.187 95.1476 111.371 95.3263 113.265 16 31 96 96 95.4285 100.948 96.329 106.318 98.6819 112.228 98.4374 115.847 95.8335 119.466 91.8032 123.815 86.3969 128.606 81.019 133.098 79.1356 138.337 82.2694 143.096 84.7953 147.919 87.5967 151.93 89.7636 155.715 91.3584 158.664 91.9683 161.258 16 31 96 136 96.3699 139.224 96.4141 143.048 95.7906 147.441 96.0714 152.077 91.3255 155.761 85.2339 159.254 74.1594 164.34 65.3053 170.474 59.8134 177.148 56.3001 183.995 53.8456 190.979 52.1019 198.249 50.6773 205.508 50.2455 215.236 51.6238 223.02 16 31 96 176 95.3094 178.489 94.5069 181.149 92.632 184.466 91.8097 188.93 89.5819 193.647 82.3407 198.941 72.1676 208.133 60.4773 219.113 48.4718 228.586 36.9618 236.442 30.0892 238.43 28.7751 235.331 27.6906 238.048 30.8025 239.829 37.0828 246.238 16 31 136 56 135.347 61.3701 134.857 67.7646 134.123 72.5815 132.53 74.8973 130.577 77.0147 127.666 80.1826 125.442 83.5021 123.435 87.2653 122.44 91.5081 122.058 95.6038 121.91 99.45 121.92 102.427 122.135 105.266 122.499 107.038 122.308 108.666 16 31 136 96 133.331 99.8697 130.956 103.937 130.709 109.44 128.179 112.777 124.774 116.104 121.688 120.241 118.998 124.481 116.81 128.814 116.644 133.303 115.846 137.687 114.548 141.631 114.053 144.973 114.333 147.925 114.829 150.094 115.108 152.177 16 31 136 136 133.004 140.015 129.483 143.911 125.303 148.673 123.362 154.468 122.943 160.03 118.98 162.78 111.318 166.935 102.04 171.122 98.2094 177.028 94.3877 184.057 91.1921 190.238 89.1369 195.51 88.6618 201.065 88.315 206.083 90.8828 209.602 17 32 22 147 21.9865 147.09 21.9278 147.122 21.4874 147.571 20.0198 149.154 19.5325 154.065 19.2384 159.313 19.586 164.359 20.697 169.978 22.4978 176.272 23.4661 180.383 24.8583 186.861 25.334 193.549 25.3459 200.838 25.3216 209.253 25.6161 215.401 17 32 27 112 25.6786 116.743 24.3649 120.989 22.9892 125.671 21.988 130.238 21.4336 135.401 21.4304 140.348 21.6598 146.964 23.106 152.464 25.0522 158.625 26.4622 166.173 27.9406 173.759 28.9354 180.892 29.1274 188.726 28.9954 196.649 29.2689 203.929 17 32 32 102 31.6784 105.431 31.3026 109.423 31.0978 112.6 30.6834 116.775 30.4606 121.679 30.8941 126.407 31.3446 132.398 32.1438 138.427 34.2479 144.81 36.3433 151.496 37.9021 158.897 38.2384 166.516 37.9215 174.62 36.9267 182.264 37.343 189.586 17 32 37 97 35.9537 101.018 34.6717 105.399 34.0768 109.551 32.9891 114.146 33.005 118.943 32.5672 124.044 32.864 130.036 33.5043 136.089 35.6359 142.034 37.9693 148.46 39.2549 155.451 39.2877 160.509 39.0513 161.943 39.0877 162.235 39.0948 162.272 17 32 37 127 36.8246 127.505 36.8201 127.511 36.8194 127.513 36.8133 127.52 36.6441 128.211 36.3953 133.15 36.8964 138.929 38.052 144.832 39.9781 150.832 42.0849 157.246 44.4479 164.24 45.337 171.831 46.1194 179.748 45.5515 187.331 46.2112 194.763 17 32 37 137 36.0773 141.558 34.9666 145.996 33.9727 150.302 32.271 154.892 31.5443 159.517 31.0688 164.793 31.6498 170.482 33.1022 176.923 35.1383 183.394 36.9492 190.464 38.6411 197.772 39.7866 204.872 39.8678 212.579 39.1624 221.403 40.9076 230.482 17 32 72 102 71.5062 106.611 70.5684 110.966 69.2762 114.846 67.8102 118.456 67.187 122.065 69.0896 124.963 70.8827 127.696 75.8007 129.788 83.7227 132.815 85.0163 137.497 84.8306 142.445 85.0291 146.597 85.4214 149.479 85.2326 152.17 85.6468 154.012 17 32 92 167 91.9986 166.546 92.0089 167.011 92.0209 166.56 92.037 167.487 92.199 170.869 81.5193 177.016 70.2961 179.622 62.8629 178.202 61.8994 184.485 60.4189 192.168 59.8112 199.687 59.6048 207.089 60.1571 214.184 60.6307 220.788 64.4221 226.998 17 32 92 172 91.9955 171.72 92.0014 172.092 92.0093 171.715 92.0894 171.722 92.1356 172.755 83.1736 180.866 73.0947 181.731 62.9141 177.471 62.1264 184.039 60.6459 191.722 59.8825 199.093 59.6669 206.424 60.2271 213.556 60.7007 220.161 64.0533 226.37 17 32 102 152 100.899 153.213 100.127 155.152 98.7653 158.259 99.4661 162.631 97.1327 166.765 80.9615 171.505 65.5049 171.012 62.4477 175.115 62.0561 181.944 60.5752 189.57 59.9637 196.833 59.7425 204.137 60.3435 211.252 61.0113 217.528 63.681 223.712 17 32 107 92 105.765 98.9142 111.276 105.859 107.746 109.969 105.058 114.191 105.278 117.726 105.721 121.736 107.378 125.783 108.553 131.34 109.299 136.175 112.034 141.055 113.53 145.385 114.818 148.639 115.378 150.971 115.46 153.18 115.917 155.38 17 32 107 167 106.957 166.993 106.957 166.993 107.074 167.013 106.568 167.932 100.339 172.277 90.3477 180.407 74.4584 183.495 63.1015 178.495 62.1381 184.779 60.6575 192.432 59.8942 199.803 59.6877 207.206 60.2401 214.3 60.7137 220.905 64.5051 227.114 17 32 117 97 116.882 101.504 117.208 108.329 115.164 111.719 113.898 115.114 114.835 118.517 115.796 122.248 116.404 127.029 116.692 131.769 116.476 137.266 116.634 142.472 116.093 146.974 116.781 150.249 116.578 152.604 116.728 154.56 117.313 156.473 17 32 127 157 127.307 156.899 125.348 157.957 134.54 166.493 133.084 170.334 129.205 171.774 122.834 176.88 122.413 180.973 122.876 184.726 121.652 192.499 122.883 197.963 122.363 202.359 124.301 206.757 125.799 208.91 126.797 211.014 128.563 215.417 17 32 147 117 146.494 122.97 145.908 129.205 145.307 135.711 144.906 142.273 144.876 148.868 145.133 155.573 145.755 162.088 145.875 170.169 143.856 179.898 141.021 186.08 140.51 194.822 142.061 198.218 143.717 200.54 144.735 203.728 145.577 206.456 17 32 147 132 145.678 135.281 145.672 135.66 145.096 138.954 144.69 144.008 144.621 150.399 144.845 157.104 145.441 163.618 145.63 171.324 143.436 180.962 140.538 187.144 140.201 195.317 141.733 198.58 143.315 200.957 144.313 204.098 145.118 206.799 17 32 152 122 151.781 124.305 151.324 130.454 150.772 136.78 150.349 142.924 150.375 149.418 150.758 156.153 151.482 162.775 151.734 170.596 151.13 179.86 151.517 185.783 149.368 194.805 150.748 198.294 151.525 201.162 152.691 204.318 154.04 206.938 17 32 177 142 177.491 148.917 177.643 155.743 178.132 161.564 183.185 166.907 209.239 180.515 216.974 192.968 218.488 200.712 217.228 201.299 216.577 200.165 214.712 198.55 214.736 199.038 212.411 199.563 212.797 200.66 210.007 200.951 208.554 201.081 17 32 187 137 194.644 138.515 195.508 147.461 191.869 158.403 192.178 166.329 214.654 180.452 221.428 193.444 220.011 199.798 219.641 200.714 219.381 199.568 217.684 197.882 213.325 197.005 211.543 197.564 212.035 198.739 210.703 199.171 209.871 199.233 17 32 24.0416 123.037 22.7113 126.711 21.2425 131.157 19.8976 135.06 18.8076 139.557 18.2473 145.053 18.182 150.508 18.5647 154.931 19.6485 160.255 21.6293 165.995 23.0621 172.321 24.6491 179.206 25.5025 187.334 25.6199 195.314 25.6217 203.01 25.9799 210.059 17 32 31.1127 130.108 29.812 134.489 28.3034 138.848 27.1567 143.573 26.157 148.301 25.4324 153.021 25.2221 158.403 25.8024 163.967 26.9389 170.179 28.81 176.47 30.7636 183.396 31.891 190.126 32.5068 197.375 32.6335 205.282 32.7259 214.416 33.531 222.787 17 32 52.3259 115.966 51.0068 120.431 49.9183 125.102 48.7597 129.75 47.8256 134.326 47.4738 139.952 47.8116 145.217 48.3501 150.893 49.4499 157.194 51.0363 164.196 52.0701 172.284 53.1148 180.492 53.699 189.004 54.2112 197.518 54.5391 206.128 55.8109 212.449 17 32 59.397 108.894 58.2913 113.285 57.1563 117.807 56.2525 122.357 55.3821 127.03 55.2382 132.032 54.817 137.15 55.4565 142.796 56.7579 149.174 58.7132 155.627 60.5046 161.956 61.6542 169.022 62.2512 175.865 62.3811 183.051 61.5254 189.779 60.5937 196.507 17 32 73.5391 94.7523 73.091 99.7757 72.5294 104.195 70.9226 108.076 68.9951 111.885 67.3026 115.306 71.954 117.848 74.8766 120.376 77.8465 123.126 82.5872 127.026 82.6684 132.329 82.6142 137.325 83.1982 141.928 83.1117 145.323 82.8906 148.097 82.9861 150.198 17 32 73.5391 115.966 72.1228 121.503 70.5608 127.21 69.5403 132.057 68.7079 136.944 68.5116 138.753 68.3635 140.975 68.5027 142.587 69.2728 146.082 70.8476 150.344 71.0251 155.289 71.3417 160.397 71.3641 166.856 70.9502 173.064 70.3736 179.201 69.7261 184.136 17 32 94.7523 158.392 94.7376 158.325 94.7144 158.382 94.3706 161.916 93.9801 166.792 94.4095 171.424 80.7283 177.263 67.9853 178.54 62.0225 179.585 60.6652 185.602 59.158 193.291 58.2146 200.944 58.1007 208.453 58.6441 215.41 59.4727 223.37 63.6055 229.239 17 32 94.7523 165.463 94.7629 165.163 94.7762 165.503 94.7994 165.166 94.8785 166.892 95.1023 171.213 81.4211 177.052 68.1943 178.665 62.2315 179.71 60.8742 185.728 59.3669 193.417 58.4236 201.07 58.3096 208.578 58.8531 215.536 59.7515 222.762 63.8843 228.63 17 32 101.823 151.321 100.772 152.707 99.8969 154.952 98.3711 158.591 98.4349 163.652 95.1342 167.675 80.0116 172.374 64.7909 172.227 61.5874 176.835 60.6972 183.42 59.2041 191.092 58.5125 198.744 58.396 206.165 58.947 213.343 59.8092 220.418 63.0448 226.553 17 32 108.894 101.823 108.009 107.448 108.798 113.22 108.867 116.058 104.549 120.443 104.538 123.957 104.752 127.688 106.084 131.498 107.174 136.945 105.22 142.037 107.902 147.401 110.138 151.514 112.026 154.32 112.812 156.372 112.912 158.644 113.253 160.804 17 32 115.966 101.823 114.983 109.286 113.593 117.968 112.2 123.672 112.276 123.783 112.188 123.985 112.032 126.402 111.458 130.943 112.51 136.173 111.339 141.598 112.085 146.639 113.576 150.747 115.364 153.552 115.82 155.605 115.783 157.828 116.082 159.935 17 32 115.966 137.179 117.008 143.077 116.671 145.83 126.096 155.571 122.082 160.645 116.611 164.944 107.574 174.095 97.6431 178.119 91.3922 192.166 85.5923 200.331 82.5832 204.96 80.3227 211.673 79.6568 216.6 81.043 219.964 83.9449 222.036 86.2087 224.543 17 32 130.108 87.6812 135.867 97.9429 134.857 101.535 134.45 105.272 133.656 108.591 133.304 112.16 133.787 116.138 134.766 121.176 135.905 126.392 137.115 131.923 137.099 136.52 136.314 139.611 135.848 142.142 136.127 144.49 136.598 146.604 137.063 148.779 17 32 144.25 130.108 141.046 139.506 140.252 148.358 139.835 153.497 138.899 157.404 137.769 161.057 138.056 165.939 138.465 170.479 139.306 175.368 139.84 180.415 137.288 185.646 136.298 193.382 137.812 196.672 139.418 198.63 140.279 201.429 140.821 203.872 17 32 179.605 137.179 180.423 142.276 181.38 149.29 183.524 155.764 191.462 162.807 218.206 179.247 221.436 191.497 219.904 198.074 219.135 198.966 218.674 197.788 216.746 196.124 212.146 195.364 209.597 195.932 209.08 197.023 208.301 197.669 205.876 197.847 17 32 186.676 137.179 195.081 137.367 198.373 146.778 192.615 159.054 190.715 167.929 210.997 180.988 219.433 194.359 218.528 200.902 218.171 201.902 217.828 200.829 215.641 199.236 210.642 198.512 207.496 199.055 207.45 200.207 206.051 200.851 203.617 201.061 17 32 214.96 123.037 217.639 136.064 218.209 150.429 224.068 158.847 224.144 173.511 228.435 187.473 226.873 198.448 224.801 204.544 227.064 205.683 229.468 204.62 230.681 203.304 231.509 202.886 230.652 203.496 230.179 204.079 229.944 204.229 229.864 204.351 17 32 222.032 115.966 224.148 129.824 225.725 145.945 227.88 161.291 227.192 175.537 227.692 189.37 226.017 200.044 223.94 205.989 225.345 207.118 227.516 206.105 228.419 204.829 228.746 204.403 228.53 205.102 228.056 205.684 227.426 205.871 226.593 206.025 17 32 74 104 73.4586 109.148 72.874 113.766 71.4513 117.799 69.9398 121.732 70.102 125.6 71.1788 129.046 72.3907 131.793 75.713 134.398 80.7246 137.963 82.9414 142.454 83.4393 147.034 83.8503 151.562 84.0397 155.169 84.4733 157.963 85.3238 159.563 17 32 84 104 83.7627 109.411 83.674 114.465 83.2677 118.463 83.1382 122.262 82.8551 126.103 83.928 129.402 84.939 132.104 83.2672 134.55 85.9915 137.834 88.9134 142.296 89.3919 146.747 89.9767 151.136 89.4285 154.743 89.3818 157.4 89.51 159.203 17 32 114 84 112.836 91.9565 118.676 99.5073 116.409 103.098 115.15 106.688 115.772 110.388 116.389 114.191 118.011 118.956 119.168 123.895 119.229 129.312 119.667 133.938 119.858 137.801 120.266 140.931 120.545 143.111 120.31 145.188 120.484 147.328 17 32 114 104 112.799 111.801 110.972 120.736 108.648 129.295 106.492 140.492 105.055 147.034 104.294 147.386 103.807 147.499 103.819 147.826 103.87 148.144 105.829 152.455 108.264 156.385 109.786 159.41 110.435 161.332 110.563 163.494 110.898 165.564 17 32 114 134 113.879 138.088 113.479 140.397 117.238 150.137 110.352 153.369 102.843 157.763 88.1738 161.702 77.8846 163.865 74.138 166.029 73.7923 169.942 73.8747 175.049 74.1942 179.84 74.5105 185.882 74.5181 193.283 74.2843 199.398 74.1121 205.209 17 32 124 114 122.006 121.012 120.126 129.956 116.468 138.71 112.86 148.634 107.662 154.145 105.7 155.541 103.976 155.488 103.96 155.839 104.059 155.9 104.086 155.967 104.832 157.229 105.106 160.299 105.453 162.326 105.49 164.673 105.683 166.856 17 32 144 114 142.984 119.143 142.296 125.313 141.893 131.789 141.536 138.279 141.619 144.827 142.006 151.429 142.75 157.979 143.036 166.29 141.603 176.398 139.253 182.814 138.222 191.854 139.655 195.092 140.928 197.118 141.769 199.909 142.459 202.567 17 32 144 124 141.472 132.366 139.996 142.545 139.654 147.503 138.476 151.368 136.929 154.756 136.975 159.692 137.239 164.818 137.372 171.435 136.117 178.099 133.491 183.242 131.85 189.584 133.188 193.28 134.427 195.277 135.04 197.361 135.538 199.661 17 32 19.799 104.652 18.9468 108.037 17.9521 112.262 16.6425 116.619 15.7841 120.891 15.5105 125.554 15.7803 130.449 16.3523 136.146 17.4761 141.889 19.4693 147.634 20.9663 154.533 21.9497 161.943 22.8873 169.585 23.3829 177.702 23.5678 185.118 24.1809 192.128 17 32 19.799 161.22 19.9158 161.373 19.6944 161.991 19.7009 163.154 18.9715 166.084 18.3959 169.237 18.485 174.36 18.2869 178.975 18.422 183.593 19.8223 189.455 20.1722 194.988 20.1816 200.308 20.5469 206.326 20.3721 213.67 19.95 221.902 20.1249 229.727 17 32 48.0833 104.652 46.7691 108.94 45.5023 113.415 44.6189 117.875 43.7995 122.497 43.4197 127.509 43.5991 132.803 44.2603 138.769 45.1005 145.099 47.0032 151.504 49.0637 158.077 50.5129 165.17 51.4668 172.53 51.626 180.39 51.5959 187.841 51.3229 195.238 17 32 76.3675 104.652 76.1337 109.943 76.3166 114.817 75.4304 118.966 73.7482 123.098 73.6977 127.215 73.8974 130.845 75.0205 133.671 78.0594 136.276 82.026 139.832 85.2049 144.167 87.1896 148.312 88.5286 152.6 89.1682 156.055 89.1078 158.552 89.0122 160.426 17 32 132.936 147.078 129.574 150.752 126.551 156.425 134.313 166.354 133.542 171.575 130.117 174.175 124.276 178.861 117.188 183.803 114.126 189.93 107.406 199.261 97.2253 208.043 97.868 211.817 98.6411 216.629 102.113 219.501 105.564 221.44 108.655 223.592 17 32 217.789 132.936 219.057 146.079 212.096 164.79 210.042 177.877 189.852 188.758 186.718 192.59 176.237 193.996 176.333 198.501 176.438 203.653 175.855 207.176 174.283 210.294 172.891 212.265 171.777 213.986 170.994 215.831 170.323 217.311 170.477 218.849 17 32 88 108 88.0918 113.59 87.8863 119.206 89.5742 123.193 89.0773 127.929 85.5103 132.683 81.2886 136.66 79.7431 140.276 79.5906 144.024 82.5535 148.013 85.7385 152.183 88.2034 156.118 89.5241 160.062 89.7577 163.549 89.5441 166.364 89.2306 168.626 17 32 11.3137 124.451 9.98313 128.459 8.52915 132.79 7.2176 137.17 6.32246 141.672 6.02107 146.528 6.06135 151.888 6.62339 157.553 7.80195 163.539 9.81378 169.768 11.3175 176.391 12.5347 183.195 13.4893 190.363 13.9012 198.407 14.0245 206.667 14.6103 214.411 17 32 67.8822 181.019 68.0011 181.784 67.871 183.263 67.899 185.539 67.7936 188.349 67.174 192.5 62.8184 197.572 53.3058 204.638 43.7523 210.979 35.7763 214.449 30.6028 218.262 27.2252 222.527 25.5232 228.859 31.0701 237.372 38.5763 244.696 39.7147 251.431 18 33 22 117 20.8884 120.664 19.699 124.698 18.8724 128.455 18.4307 133.041 18.7041 136.914 19.4455 142.027 20.875 145.696 22.8513 150.922 24.519 153.733 24.9059 154.005 24.9063 154.133 25.0257 154.014 24.9282 154.055 24.8495 154.181 24.72 154.098 18 33 27 127 25.5531 131.666 24.2493 136.022 23.1918 140.102 22.8201 145.864 22.7354 151.541 23.1651 156.533 24.3691 163.08 26.4434 169.681 27.8251 175.405 29.283 182.36 30.1591 190.4 30.2117 198.526 30.242 207.283 30.6593 214.113 31.1482 219.26 18 33 32 132 30.5403 136.054 29.4753 140.45 28.5232 144.751 28.0112 148.873 27.9117 153.578 28.3345 158.134 29.3203 163.353 30.6256 167.395 31.6486 169.85 33.0255 175.408 34.0584 182.361 34.2117 189.925 34.248 197.469 34.5648 204.581 35.4584 209.673 18 33 37 132 36.9363 132.232 36.932 132.241 36.9321 132.248 36.9232 132.266 37.1485 134.035 37.6361 138.799 38.8777 144.531 40.4461 150.371 42.7482 156.884 44.5982 163.823 45.5653 171.362 46.4929 179.276 46.216 186.859 46.8756 194.29 46.5991 200.904 18 33 37 172 37.0059 172.009 37.0173 171.975 37.0217 171.976 37.0142 172.016 36.9277 172.163 36.433 175.74 37.5625 181.259 39.6218 187.593 41.4111 194.187 41.4861 201.289 42.9126 208.202 42.4755 215.972 41.4168 224.663 43.4111 234.044 45.0904 242.275 18 33 42 102 40.6074 106.529 39.5267 110.992 38.5104 115.544 38.0603 120.466 37.2228 125.846 37.7043 131.82 38.3798 138.039 40.8846 144.127 43.2148 150.65 45.7603 157.695 46.7903 165.032 47.6227 172.548 48.9343 180.202 49.2728 187.746 49.4544 194.329 18 33 42 132 40.9263 135.634 39.9917 138.965 39.8437 140.267 39.8366 140.337 39.8361 140.352 39.8831 140.653 41.2875 144.752 42.5204 150.505 44.8181 156.78 45.519 163.629 46.5704 171.117 47.4981 178.953 47.4851 186.48 47.986 194.007 47.7166 200.551 18 33 47 112 46.4353 114.282 45.4958 118.551 44.6844 122.851 44.1377 127.769 44.2287 132.937 44.798 138.639 45.2752 144.688 45.9762 150.511 48.322 156.934 50.1184 164.042 50.8646 171.271 50.4492 178.909 51.1249 186.322 50.6833 193.317 50.0592 199.778 18 33 47 127 45.8808 131.019 44.9159 135.108 44.0347 139.378 43.7292 143.532 43.8908 147.648 44.1724 150.69 44.1897 150.854 44.1178 153.163 44.7262 157.829 44.923 164.376 45.8901 171.914 46.6773 179.753 46.1116 187.281 46.7712 194.712 46.5019 201.207 18 33 52 157 51.922 156.883 51.8716 156.94 52.3037 158.713 51.4698 162.395 51.7004 167.673 51.9335 173.408 53.2291 178.988 55.0536 184.136 55.8166 191.158 56.5682 198.868 56.3293 205.959 56.8978 213.493 57.276 221.009 59.321 226.978 60.5555 232.424 18 33 57 102 55.9642 106.577 55.5475 111.176 55.4813 115.578 55.3518 120.809 54.1505 126.219 54.2638 131.584 55.7029 138.039 57.979 144.56 59.9279 150.746 61.4617 157.009 61.5606 163.87 60.9483 171.188 60.3999 178.023 59.7245 184.257 59.1346 189.31 18 33 57 107 55.7869 111.49 55.2139 116.092 54.581 120.517 54.2528 125.603 53.7168 130.851 54.0592 136.567 55.527 143.054 57.8026 149.526 59.7578 155.819 60.9448 162.461 61.2546 169.083 60.4376 176.292 59.4163 183.191 57.1841 190.293 56.2037 196.469 18 33 57 142 55.7173 145.307 54.716 149.04 53.1703 154.028 52.254 159.07 52.2046 164.333 52.3395 170.441 53.2364 176.867 54.7943 183.303 55.5573 190.737 56.3089 198.447 56.0634 205.529 56.6319 213.064 57.01 220.579 59.0551 226.548 60.2896 231.995 18 33 57 162 57.0165 161.896 57.0114 161.924 57.0079 161.931 57.0489 162.168 56.1297 167.283 56.1324 172.779 56.7089 178.905 58.1209 184.976 58.1699 192.308 58.108 199.997 57.876 207.244 58.4362 214.48 58.8306 221.514 62.0538 227.178 64.0602 231.696 18 33 62 97 61.4737 101.445 61.6654 105.716 61.4002 110.866 61.9158 116.167 62.3759 119.974 63.3027 123.042 67.6438 125.184 67.8938 130.86 70.195 135.193 73.3938 138.882 74.5662 141.804 74.7294 142.922 74.7891 142.965 74.7276 142.935 74.7356 142.942 18 33 62 112 61.0744 116.091 60.3003 120.126 59.7343 124.28 59.5207 128.833 59.2739 133.227 59.8366 138.518 61.0302 144.45 63.1949 150.737 65.1784 156.9 66.1167 163.661 66.4847 170.256 65.8705 177.328 65.4181 183.433 64.3102 189.47 63.5391 194.574 18 33 62 162 62.0777 161.954 62.0403 161.877 61.9891 161.864 62.3745 162.776 61.5792 166.587 61.614 171.099 61.1718 177.155 60.5727 183.798 59.0922 191.389 58.8464 198.957 58.6231 206.242 59.1946 213.449 59.683 220.097 62.5411 226.066 64.5458 230.539 18 33 67 117 66.1385 121.365 65.9408 122.244 65.7301 122.733 65.4977 124.634 66.115 128.113 66.4682 130.65 68.6733 133.441 69.7556 138.947 72.5573 143.048 75.8155 146.926 78.1072 149.857 79.8329 152.274 79.7553 155.119 79.9341 157.03 80.3604 158.454 18 33 72 102 71.5939 106.614 69.8845 110.495 67.9667 114.238 68.2024 118.052 73.3764 120.601 76.273 123.151 79.1685 125.887 87.9288 129.066 88.33 133.678 88.4769 138.704 88.5013 142.784 88.2386 146.123 87.8404 148.718 88.0253 150.44 88.0727 152.414 18 33 72 137 70.3287 140.457 68.8905 144.198 67.4315 148.392 66.4609 152.663 65.5861 157.282 65.7854 161.448 65.8666 162.612 65.855 163.016 66.5037 166.363 67.0112 169.712 67.4662 174.974 67.6178 181.011 67.1988 184.782 66.437 190.016 66.0371 194.635 18 33 72 152 71.7592 152.633 71.3191 154.711 69.1048 157.899 67.8538 161.443 65.5445 165.623 61.087 169.57 60.9479 175.82 60.4904 182.547 59.0919 190.007 59.0098 197.491 58.7865 204.776 59.3878 212.002 59.9717 218.65 62.5281 224.698 64.5328 229.205 18 33 97 107 97.5672 111.089 96.6223 114.013 96.2437 117.66 96.1816 121.193 96.0209 124.804 98.2587 128.202 100.024 133.031 104.372 139.053 107.352 144.673 109.706 148.938 111.463 151.727 112.222 153.744 112.388 156.066 112.831 158.296 113.266 160.131 18 33 97 112 96.8606 112.157 95.9157 114.422 96.0351 118.069 95.9729 121.603 95.716 125.042 97.9539 128.44 99.7188 133.269 104.067 139.291 107.047 144.91 109.401 149.176 111.088 151.965 111.847 153.982 112.013 156.304 112.456 158.534 112.891 160.338 18 33 102 177 101.961 177.003 101.989 177.023 102.013 176.935 100.099 177.81 92.2786 185.904 80.5455 191.859 72.8784 191.025 66.0152 183.563 64.6255 191.61 63.6447 198.722 63.4939 206.423 64.1304 213.495 64.8231 219.981 70.7989 228.398 74.03 232.957 18 33 102 212 102.034 212.014 102.076 212.002 102.077 212.052 102.123 212.397 99.58 212.465 83.856 231.935 68.6674 248.266 58.9875 254.626 53.307 256.008 52.754 252.131 64.0804 238.749 79.3686 227.216 82.9937 229.043 84.385 232.761 85.7002 236.32 18 33 107 102 111.878 109.192 108.685 113.084 105.702 117.338 105.76 120.812 105.95 124.711 107.34 128.696 108.479 134.375 107.23 139.285 110.305 144.43 112.443 148.599 114.2 151.438 114.944 153.61 115.189 155.765 115.65 157.942 116.128 159.739 18 33 107 152 106.361 154.619 105.355 157.981 105.366 163.029 98.1864 166.825 82.3781 171.679 67.0251 171.66 63.4775 175.325 62.98 182.162 61.4991 190.032 60.8876 197.295 60.6742 204.618 61.2585 211.708 61.8006 217.967 65.136 224.613 67.1428 229.036 18 33 127 97 133.001 103.719 132.68 107.433 132.056 110.817 131.783 114.425 132.262 118.49 133.18 123.526 134.388 128.697 135.677 134.022 135.677 138.669 134.785 141.697 134.259 144.362 134.472 146.709 134.95 148.765 135.358 150.744 135.552 152.321 18 33 127 147 123.945 149.102 133.171 157.621 131.586 161.488 126.84 165.068 123.608 172.659 122.849 177.128 121.789 182.343 120.645 190.211 120.219 196.764 119.821 201.087 122.173 205.143 123.784 207.399 126.305 208.661 128.072 212.979 130.032 215.09 18 33 142 127 141.719 127.995 141.305 133.944 140.994 139.764 140.935 146.054 141.173 152.578 141.626 158.745 141.496 166.61 139.616 175.149 136.885 181.355 135.358 190.526 136.862 193.926 138.334 195.726 139.119 197.538 139.584 199.669 139.801 201.844 18 33 147 127 146.509 131.337 145.861 137.654 145.447 143.908 145.378 150.299 145.601 157.003 146.24 163.528 146.428 171.52 144.198 180.785 141.374 186.913 141.036 195.085 142.588 198.481 144.277 200.897 145.295 204.084 146.137 206.813 146.374 209.118 18 33 152 117 151.648 121.879 151.558 127.714 151.416 133.168 151.562 138.861 152.1 144.775 153.287 150.595 154.145 158.44 155.322 166.791 154.949 173.132 155.726 179.639 158.248 182.886 159.819 186.138 160.936 188.734 161.904 192.024 162.095 194.664 18 33 24.0416 108.894 22.9174 113.271 21.8444 117.177 21.4531 121.051 21.3344 125.721 21.3507 130.931 22.2819 136.057 23.009 142.025 25.0668 147.875 26.7549 154.725 28.1869 162.193 28.9514 169.659 29.7766 176.374 29.7639 182.966 31.0004 189.335 31.0918 195.78 18 33 31.1127 130.108 29.6084 134.384 28.4976 139.065 27.4474 143.561 26.8553 148.299 26.7957 153.918 27.3444 159.413 28.5129 165.615 30.4352 171.906 32.1385 178.319 33.4227 185.282 34.1435 192.532 34.2189 200.22 34.3306 209.443 34.88 216.982 35.4807 222.552 18 33 38.1838 144.25 36.923 148.539 35.6744 152.998 34.1665 157.638 33.6788 162.289 33.119 167.659 33.6424 173.366 34.8878 179.792 36.9786 186.297 38.6165 193.404 39.79 200.535 40.8973 207.579 40.746 215.309 39.8942 224.142 41.5461 233.376 43.1926 241.622 18 33 59.397 158.392 59.4288 158.751 59.4595 158.654 59.3411 160.165 59.6096 163.774 58.9423 168.31 58.4274 173.354 58.9519 179.49 58.0748 186.053 56.7757 193.6 55.9937 201.422 55.8817 208.873 56.3714 215.609 57.1657 223.707 59.9796 229.567 62.0806 234.652 18 33 66.468 115.966 65.1226 121.415 63.8725 126.428 63.1056 131.444 62.739 135.907 62.5554 140.151 63.0418 144.718 63.7674 149.413 65.4211 155.091 66.9716 161.793 67.9689 169.056 68.566 176.065 68.6726 183.311 68.3476 189.567 67.8485 195.94 67.5051 201.248 18 33 66.468 137.179 64.5694 140.673 63.0291 144.66 61.1702 148.936 60.0924 153.121 59.1476 158.24 58.4835 163.484 58.1226 169.645 58.7354 176.484 59.1639 184.264 59.7455 191.976 60.1208 199.389 61.0353 206.526 62.0904 213.15 64.2149 219.234 66.0705 224.093 18 33 66.468 165.463 66.4659 165.426 66.3777 165.383 66.3407 165.487 66.656 165.975 64.993 169.904 61.1809 173.896 60.2089 179.455 59.0872 185.818 57.6575 193.466 56.8135 201.225 56.7015 208.676 57.1912 215.413 57.9855 223.51 61.6112 229.37 63.7993 233.867 18 33 73.5391 108.894 72.7274 113.244 71.361 117.179 69.75 121.087 69.4023 124.7 70.2594 127.773 71.6052 130.396 74.6058 132.93 79.483 136.25 81.3099 140.845 83.4574 144.556 84.2567 148.264 84.3419 151.586 84.6306 154.156 85.243 155.806 85.2163 157.697 18 33 94.7523 108.894 94.9036 110.394 93.5923 114.789 93.5767 118.831 93.1913 122.632 93.267 126.17 94.9366 129.448 97.8379 134.223 103.211 140.219 106.135 145.631 108.485 149.816 110.369 152.665 111.123 154.717 111.149 157.031 111.51 159.2 111.931 160.955 18 33 108.894 94.7523 111.676 102.164 108.572 106.079 106.405 110.061 106.77 113.578 107.232 117.565 108.581 121.953 109.651 127.021 109.533 132.127 111.514 137.15 113.56 141.478 115.28 144.649 115.741 146.87 115.4 149.073 115.56 151.291 115.824 153.262 18 33 144.25 123.037 143.634 129.24 143.091 135.738 142.611 142.208 142.592 148.784 142.884 155.423 143.497 162.032 143.601 170.17 141.765 180.027 139.061 186.123 138.419 193.86 139.915 197.148 141.487 199.282 142.404 202.292 143.141 204.997 143.366 207.218 18 33 144.25 137.179 142.826 143.882 142.555 145.883 142.125 147.494 141.823 150.38 142.066 156.95 142.629 163.511 142.721 171.487 140.728 180.973 138.176 186.646 137.534 194.383 139.029 197.671 140.602 199.707 141.361 202.506 142.019 205.122 142.244 207.318 18 33 24 104 23.9798 107.439 23.6825 111.229 23.3393 115.056 23.3922 119.6 23.9037 124.605 24.6707 130.121 25.4493 136.126 26.9143 142.53 28.5647 149.109 29.9324 156.621 30.1316 164.515 30.4165 172.49 30.2622 179.505 31.3449 186.161 32.0935 192.344 18 33 74 124 72.3053 129.545 70.8967 134.936 69.4427 140.971 68.4024 145.459 67.7974 150.224 67.9051 155.973 67.7767 161.325 68.0604 168.16 68.686 176.072 69.354 183.952 69.8231 191.775 70.1523 199.369 70.0851 206.83 70.3543 212.462 70.8879 217.092 18 33 124 134 121.102 144.223 130.045 153.834 128.003 157.548 122.911 160.731 115.619 168.398 111.616 171.672 110.863 177.874 104.927 186.72 95.8211 195.148 96.2169 198.437 97.1758 203.095 101.245 205.596 104.132 207.769 106.6 210.128 108.03 212.263 18 33 134 134 131.293 145.696 136.637 153.668 135.305 157.383 132.023 160.404 131.948 165.48 131.523 170.422 131.998 175.735 136.443 182.678 135.524 187.627 134.97 194.005 136.419 197.541 137.69 199.592 138.4 202.098 138.986 204.486 139.351 206.872 18 33 144 124 143.344 130.286 142.859 136.803 142.397 143.296 142.419 149.882 142.695 156.578 143.348 163.157 143.692 171.303 142.455 180.778 140.513 186.547 139.875 194.856 141.355 198.14 142.631 200.358 143.547 203.332 144.38 206.077 144.673 208.436 18 33 194 134 208.811 136.199 218.892 147.494 228.482 162.962 229.931 178.277 228.585 189.703 226.743 195.873 229.026 197.268 230.755 196.292 231.302 195.081 231.799 194.671 231.717 195.217 231.616 195.77 231.442 195.863 231.457 195.955 231.47 196.183 18 33 204 134 220.244 136.893 225.411 151.952 227.46 169.445 228.312 184.807 227.111 196.259 225.252 202.36 227.75 203.577 229.712 202.628 230.608 201.297 231.178 200.755 230.66 201.387 229.612 202.159 229.425 202.318 229.403 202.41 229.16 202.718 18 33 48.0833 118.794 46.886 123.426 45.6326 128.105 44.6485 132.882 44.3066 138.146 44.4829 143.49 45.0614 149.417 45.9775 155.826 47.8778 162.485 49.7773 169.358 51.1062 176.858 52.1203 184.514 52.4819 192.515 52.6423 200.542 52.5575 207.757 52.6213 214.14 18 33 62.2254 90.5097 61.181 94.9396 61.0612 99.1833 60.777 103.455 61.8293 107.701 62.6722 110.749 64.4068 113.319 67.8774 115.919 69.979 120.188 70.4574 125.001 70.9793 129.093 71.9381 132.806 72.3378 135.735 72.745 137.691 72.934 138.936 73.0335 139.698 18 33 76.3675 90.5097 76.2824 95.4005 75.699 99.0907 75.0486 102.556 74.9245 105.923 74.9411 108.8 75.2364 111.407 75.6691 114.005 75.9886 117.99 76.4864 123.06 76.7635 127.702 77.0364 132.054 77.2129 135.844 77.5012 138.25 77.6499 139.616 77.6926 140.82 18 33 104.652 104.652 108.072 111.872 104.947 115.944 103.041 120.938 99.9405 125.515 99.508 129.249 101.593 132.973 102.986 138.026 105.092 143.018 108.267 147.808 111.012 151.991 112.824 154.978 113.545 157.069 113.424 159.159 113.649 161.217 113.981 162.84 18 33 147.078 118.794 145.915 122.646 145.094 126.862 144.64 132.274 144.751 137.837 145.261 143.79 146.264 149.793 147.039 157.175 146.857 166.746 145.494 172.972 144.811 181.687 146.48 184.52 147.682 186.504 148.553 189.405 149.274 192.213 149.532 194.556 18 33 128 88 132.037 95.1129 131.016 98.6518 129.819 102.114 129.346 106.128 129.26 110.152 129.882 114.747 130.924 119.652 131.477 124.534 131.192 128.951 130.459 132.838 130.01 135.839 130.152 137.955 130.429 139.924 130.795 141.799 130.766 143.3 19 34 22 152 21.994 151.929 21.3712 152.851 21.132 157.69 20.7994 162.908 21.3376 168.362 22.0905 173.566 23.494 179.111 25.186 185.123 25.915 191.314 26.2967 198.114 26.3052 205.451 26.2109 213.906 26.5584 220.79 27.1407 225.986 27.5963 226.878 19 34 27 137 25.8756 141.995 24.7939 146.489 24.1488 151.362 24.0278 156.945 24.5463 162.454 25.6219 168.511 27.5856 174.99 29.2053 181.159 30.6499 188.623 31.1833 195.896 31.2042 203.628 31.3518 212.951 31.9933 220.957 33.1007 228.503 34.4539 233.671 19 34 32 117 30.7256 121.235 30.459 125.219 30.1726 130.007 30.2602 135.236 30.7354 141.09 31.8833 146.902 33.761 153.247 35.7087 159.699 36.912 166.964 37.4384 174.642 37.5368 182.623 37.6709 189.951 38.0493 197.513 38.3274 204.451 39.7668 209.91 19 34 32 172 31.9613 171.937 31.9166 171.986 31.9609 171.95 31.9639 172.105 32.2876 175.904 33.7977 181.635 35.1029 187.878 36.6445 194.428 38.4504 201.868 39.7486 208.724 39.6949 216.324 38.4453 224.958 40.1099 234.132 41.9125 242.328 44.1819 248.407 19 34 37 142 35.983 146.494 34.9172 151.008 34.4729 155.648 34.2175 160.883 34.774 166.568 35.4193 172.874 37.4327 179.267 39.0359 186.294 40.2322 193.44 41.416 200.536 41.5025 208.393 40.9052 217.244 42.5993 226.049 43.5916 234.12 45.5095 240.276 19 34 42 147 40.5494 151.427 39.7992 156.035 39.5042 160.646 39.446 165.871 39.9249 171.61 40.5708 178.065 42.8119 184.51 44.7886 191.548 45.5673 198.197 47.149 205.452 47.6169 213.194 47.6636 221.827 44.6408 231.98 45.9523 240.251 48.1834 246.547 19 34 57 122 55.9703 126.434 55.1776 130.363 55.3026 134.078 55.0278 137.668 55.8258 142.473 57.1275 148.53 59.0251 154.54 60.9035 160.728 61.9506 167.473 62.4511 174.021 62.2927 180.89 61.3517 187.323 59.9153 193.979 59.4359 199.37 59.2974 204.78 19 34 62 112 61.6523 116.281 61.4364 120.867 61.3035 125.921 60.8874 130.457 61.2162 134.185 62.4811 138.627 64.733 144.129 66.7634 148.503 68.3028 153.511 68.949 158.25 68.7293 162.67 68.15 166.585 67.9454 170.156 68.0214 173.101 68.0131 175.956 19 34 67 107 66.7417 111.297 66.8283 116.107 68.0672 120.527 71.2914 123.222 74.3563 125.757 77.8463 128.148 86.733 131.191 87.4155 135.855 87.5033 140.721 87.5983 144.873 87.3337 148.347 87.0409 150.998 87.4173 152.712 87.5235 154.628 87.7971 156.389 19 34 67 112 66.3558 115.157 66.4685 119.581 66.4226 123.374 67.4634 126.59 68.0093 129.389 72.9839 131.568 74.5318 135.736 76.502 139.849 80.367 143.176 83.7022 145.605 84.4357 148.519 84.2469 151.317 84.5302 153.299 84.4961 155.322 84.6887 157.21 19 34 72 147 70.5853 150.84 67.5265 154.384 66.1561 158.281 64.8708 162.97 61.8595 167.113 61.7994 173.432 61.6646 180.337 60.2697 188.185 60.1042 195.44 59.9321 202.774 60.5433 209.828 61.1991 215.749 63.3506 221.791 65.3343 226.433 67.5203 230.637 19 34 77 147 75.5873 150.812 73.9835 154.432 71.6291 157.73 68.1061 161.642 63.1874 165.251 63.2086 171.517 63.1072 178.453 62.0959 186.337 61.6452 193.402 61.7822 200.632 62.3934 207.536 63.0491 213.267 64.6998 219.102 66.6177 223.902 68.8359 228.106 19 34 87 147 86.9611 151.763 86.802 157.128 86.6302 163.241 77.2503 167.073 65.496 166.315 64.9099 172.439 64.6397 179.397 63.47 187.638 62.6005 194.783 62.5182 202.065 63.1231 208.969 63.7592 214.769 66.6714 221.011 68.8176 225.73 70.8332 229.406 19 34 97 107 94.3821 111.166 94.6092 115.021 94.1957 118.789 94.2684 122.427 95.9746 125.675 98.6303 130.744 104.164 136.79 107.263 142.474 109.738 146.687 111.504 149.512 112.239 151.647 112.332 153.958 112.762 156.197 113.181 158.057 113.654 159.532 19 34 142 137 141.899 137.453 141.864 142.144 141.777 148.435 142.001 155.005 142.467 161.36 142.335 169.017 140.37 177.995 137.501 184.424 136.092 193.021 137.674 196.291 139.22 198.166 140.04 200.309 140.571 202.512 140.784 204.726 140.88 206.578 19 34 147 122 146.708 128.385 146.337 134.243 146.39 140.627 146.763 147.321 147.65 153.72 148.145 161.764 147.03 171.918 144.552 178.499 144.105 187.038 145.166 191.008 146.374 193.548 147.241 196.831 147.947 199.803 148.158 202.258 148.162 204.278 19 34 59.397 108.894 59.1661 113.319 58.8692 117.935 58.9436 123.051 58.1342 127.866 58.3208 132.721 59.5354 139.077 61.8269 145.507 63.9203 151.588 65.3052 158.142 65.3734 164.955 64.6323 172.076 63.9879 178.708 63.2647 184.882 62.451 190.106 62.3145 194.76 19 34 87.6812 193.747 87.6607 193.879 87.6816 193.761 87.7365 193.469 84.5814 209.132 73.7193 220.978 63.741 226.558 57.3105 223.418 54.3103 218.811 53.3741 218.965 55.2147 218.075 56.6213 219.121 59.499 227.584 64.1248 232.277 65.9736 236.126 68.495 239.237 19 34 87.6812 200.818 87.6874 200.858 87.7148 200.693 87.7945 199.922 83.9992 210.533 74.5813 222.379 63.4494 230.283 55.0441 231.183 51.7797 226.491 50.1036 226.227 50.0755 226.764 52.0233 226.398 62.1061 230.589 66.9323 234.479 67.4781 235.767 70.5227 239.1 19 34 94.7523 186.676 94.7503 186.782 94.7912 186.746 94.1832 187.669 84.2569 208.123 74.8389 215.994 66.2085 219.378 60.1198 214.638 56.4583 215.161 55.7615 214.903 57.5842 214.756 58.7285 217.003 59.8346 224.587 64.4604 230.223 66.6282 234.506 69.2369 238.059 19 34 101.823 108.894 100.316 113.135 99.1617 117.126 99.0357 120.733 98.5804 124.459 99.7047 128.048 100.843 133.092 104.511 138.93 107.431 144.321 109.78 148.487 111.72 151.348 112.505 153.413 112.451 155.694 112.791 157.851 113.212 159.655 113.773 161.14 19 34 101.823 172.534 102.152 172.503 102.387 172.189 99.1517 174.703 89.4705 186.577 77.6181 191.222 66.1829 189.269 61.235 186.639 59.7153 194.294 58.7719 201.947 58.6622 209.666 59.1941 216.124 60.3003 223.707 64.978 229.878 67.1458 234.161 69.7545 237.715 19 34 24 124 22.7241 128.331 21.7525 132.987 21.2738 137.987 21.2148 143.298 21.6785 149.334 22.9416 155.417 24.847 161.683 26.3138 168.645 27.7088 175.861 28.6968 183.678 28.8404 191.749 28.8347 199.542 29.2317 206.908 29.7914 213.349 31.2211 218.358 19 34 44 104 43.0491 108.491 42.4322 113.185 42.0265 118.228 42.2399 123.645 43.1207 129.744 44.3531 136.096 45.8956 142.354 47.6188 148.781 48.9931 155.542 50.0933 163.068 50.2518 170.813 50.609 178.136 50.4294 185.679 49.3753 192.166 49.1816 197.657 19 34 44 144 42.6958 148.53 41.7647 153.284 41.4533 158.01 41.2841 163.314 41.9838 168.995 42.8936 175.31 45.1451 181.859 46.6688 188.923 47.8355 195.984 48.9443 203.514 49.5146 211.502 49.8265 220.261 49.4175 229.237 49.3907 237.153 50.8852 243.397 19 34 64 124 62.5063 129.54 60.9964 135.157 59.7737 140.808 59.2369 146.183 59.3509 151.943 59.6686 158.234 60.7652 165.264 61.9322 173.263 62.6789 181.563 63.4824 189.973 63.9461 198.256 64.0096 206.702 64.5537 213.044 65.7929 218.238 67.01 222.693 19 34 104 104 99.776 108.172 99.0218 112.088 99.0971 115.625 99.1721 119.463 101.371 123.231 102.74 128.391 105.964 133.64 109.138 138.507 111.804 142.66 113.511 145.76 114.152 147.91 113.867 150.094 114.055 152.289 114.291 154.17 114.746 155.831 19 34 184 144 185.962 151.12 193.721 158.458 217.853 174.993 220.597 186.5 219.768 192.857 219.161 194.526 218.376 193.397 215.499 191.653 212.067 191.135 209.933 191.873 208.582 192.841 206.98 193.424 204.412 193.648 203.669 194.303 203.457 194.747 19 34 194 144 190.351 156.619 204.058 163.908 216.253 179.02 220.566 191.259 219.61 197.706 219.418 198.997 219.058 197.941 217.237 196.4 211.975 195.541 208.925 196.216 207.628 197.316 204.339 197.747 201.705 197.966 200.803 198.714 199.601 199.166 19 34 19.799 118.794 18.456 123.309 17.5183 127.822 17.1183 132.53 17.1287 137.79 17.5822 143.727 18.8234 149.584 20.7479 155.849 22.1586 162.903 23.4554 170.234 24.552 177.872 24.8163 186.072 24.8567 193.693 25.2613 201.212 25.7142 207.894 27.0094 213.195 19 34 118.794 118.794 114.175 123.493 110.09 129.858 101.154 135.855 97.0718 140.68 96.6544 144.039 97.0505 146.756 98.3312 150.214 101 154.526 103.645 158.509 105.079 161.586 105.498 163.764 105.354 166.026 105.398 168.107 105.628 169.802 105.724 171.384 19 34 147.078 132.936 146.321 138.874 145.624 144.607 145.769 150.837 146.634 157.055 147.682 163.264 148.531 171.099 148.932 180.697 148.41 186.65 147.823 194.626 149.526 197.886 150.787 200.348 151.815 203.49 152.842 206.245 153.338 208.45 153.691 210.256 19 34 48 88 47.5993 92.2117 47.2093 96.3235 47.3165 100.757 47.0858 104.787 47.6673 108.716 48.6409 112.901 50.7929 116.669 52.1796 120.269 53.2256 122.995 53.5724 123.599 53.6899 123.996 53.7718 124.015 53.7359 124.046 53.6491 124.033 53.7038 123.893 20 35 32 127 31.9834 127.206 32.3223 130.042 32.7855 134.794 34.0219 138.974 35.104 145.035 37.1035 151.321 38.9819 158.041 40.5171 165.18 41.0598 172.852 40.749 180.983 40.0809 188.749 40.5973 196.192 41.175 202.602 43.2495 207.348 44.2577 212.304 20 35 32 157 31.3402 159.445 30.533 163.895 30.3485 169.179 30.772 174.671 32.2347 181.089 34.3643 187.527 35.6355 193.817 37.5318 201.327 38.737 208.328 38.8537 215.939 37.6026 224.561 39.2672 233.735 41.1364 241.861 43.1844 247.88 45.2791 251.915 20 35 42 177 42.0314 176.948 42.0328 176.996 42.0559 177.002 42.0719 177.12 42.8003 180.763 44.9229 186.971 47.3552 193.248 48.0014 199.85 48.4356 207.28 48.9669 214.978 48.9722 223.488 45.9424 233.641 47.4906 241.794 49.7533 247.986 51.9542 252.214 20 35 57 167 56.9999 166.999 57.1067 166.903 56.6129 167.799 56.6286 172.55 57.1288 178.53 58.5407 184.601 57.7514 192.051 57.6895 199.74 57.4575 206.986 58.0177 214.293 58.4121 221.327 60.9309 227.119 62.9372 231.702 64.8835 235.608 66.7458 238.735 20 35 62 127 61.3923 130.898 61.1825 134.044 61.2052 136.21 61.6335 138.522 62.3696 142.848 64.5107 148.902 65.8542 154.249 67.0487 160.836 67.0132 167.668 66.3762 174.637 65.9044 180.858 64.9037 186.86 64.1326 191.809 64.0622 195.863 64.0659 200.163 20 35 67 162 66.9207 162.386 67.288 164.083 65.685 167.597 61.6563 171.345 61.2141 177.402 60.615 184.045 59.1345 191.636 58.7667 199.237 58.5434 206.522 59.0957 213.667 59.5475 220.412 62.4056 226.381 64.4103 230.899 66.4017 234.682 68.3287 237.792 20 35 67 167 66.9948 167.072 67.0837 167.041 66.2717 168.565 62.3314 171.966 61.7811 177.653 60.8844 184.04 59.4038 191.631 59.036 199.233 58.8127 206.518 59.365 213.663 59.8169 220.408 62.6749 226.377 64.6796 230.849 66.6772 234.52 68.6042 237.598 20 35 77 112 74.9324 115.508 73.1287 118.526 74.6025 121.009 76.611 123.636 79.5714 126.281 87.96 129.573 88.5071 134.018 88.6705 138.936 88.6862 142.846 88.4235 146.186 88.0253 148.78 88.2102 150.502 88.2914 152.44 88.4463 154.22 88.2624 156.167 20 35 92 107 92.006 110.634 91.9754 114.476 91.8934 118.099 91.9614 120.93 91.6743 124.344 92.0579 126.703 91.8125 130.452 91.9977 135.17 91.7698 139.108 91.3891 142.379 91.1821 144.624 91.2279 145.988 91.2015 147.4 91.1231 148.925 90.933 151.027 20 35 112 157 109.986 161.991 103.431 166.138 91.8645 174.965 77.322 176.758 66.7573 172.592 66.1833 179.097 65.1239 187.345 64.2588 194.537 64.2694 201.814 64.8743 208.588 65.5104 214.451 67.7855 219.638 69.7618 224.194 71.2493 227.411 72.1946 229.839 20 35 117 162 109.075 164.716 102.277 168.719 91.9474 176.849 76.3289 179.188 65.215 174.516 64.5792 181.182 63.1895 189.595 62.2832 196.815 62.0798 204.153 62.6809 211.183 63.3306 217.317 67.0857 224.3 69.2435 228.963 71.3569 232.509 72.6276 235.166 20 35 147 132 146.546 138.517 146.555 145.115 146.906 151.781 147.686 158.282 147.989 166.219 146.665 176.078 144.588 181.939 143.207 191.645 144.691 195.155 146.301 197.544 147.34 200.734 148.237 203.526 148.501 205.84 148.598 207.896 148.846 209.44 20 35 59.397 108.894 59.0844 113.848 59.1509 119.156 58.525 123.759 59.9798 127.355 63.2046 130.669 65.8185 135.552 68.7848 139.996 71.7008 144.55 73.5795 148.515 75.3054 151.464 75.2873 154.604 75.9445 156.303 77.3105 156.896 78.1386 158.262 78.9963 159.703 20 35 59.397 123.037 58.607 127.792 58.3743 132.805 58.0983 137.875 58.8095 143.309 60.0224 149.33 61.9106 155.645 63.5707 162.022 64.5834 169.199 65.1857 176.189 65.3155 183.549 64.769 190.238 64.2863 196.966 63.799 202.483 63.7164 207.385 64.0327 211.149 20 35 66.468 115.966 65.7717 120.375 65.5816 124.469 66.346 128.2 67.5302 131.147 69.2386 134.21 70.5608 139.881 73.261 144.171 75.5006 148.333 77.3792 151.891 80.2181 153.804 80.857 156.205 81.3299 158.021 81.6406 159.656 81.8733 161.452 82.5401 163.067 20 35 66.468 144.25 64.5841 148.578 63.1531 152.86 61.756 157.691 60.685 162.95 60.1053 169.091 60.2225 175.935 60.6746 183.781 60.9717 191.386 61.4382 198.799 62.3527 205.936 63.341 212.54 65.3676 218.532 67.3031 223.351 69.323 227.392 71.0987 230.588 20 35 108.894 151.321 107.361 156.905 100.452 161.563 84.4315 166.127 70.2455 167.076 66.0731 170.309 65.606 177.055 64.64 184.999 64.1322 192.419 64.4492 199.773 65.2475 206.768 66.1859 213.371 69.0065 218.967 70.8017 223.445 72.6946 226.985 74.0924 229.642 20 35 108.894 158.392 106.953 163.532 99.9462 167.601 88.8453 175.48 73.0587 178.7 63.1293 175.477 62.286 182.064 60.7761 189.773 60.0844 197.231 59.9589 204.568 60.6035 211.643 61.4909 218.41 64.3496 224.638 66.5117 229.185 68.8113 233.027 70.8692 236.122 20 35 115.966 158.392 108.693 161.294 101.753 165.561 89.7387 173.44 74.8625 176.416 64.6654 173.193 64.0574 179.935 62.6003 187.841 61.9165 195.237 61.8205 202.606 62.5817 209.659 63.5017 216.111 66.9989 222.725 69.1281 227.501 71.7008 231.485 73.5661 234.365 20 35 115.966 165.463 107.758 168.305 100.775 172.911 90.9804 183.787 76.6396 187.004 64.2452 182.024 61.7119 184.787 60.1715 192.481 59.2854 200.024 59.1608 207.475 59.7298 214.211 60.5921 221.287 64.3598 227.487 66.6857 231.962 69.2583 235.695 71.0938 238.63 20 35 54 104 53.682 108.662 53.6238 113.794 52.5381 118.612 52.369 123.657 53.4947 129.066 55.6487 135.036 57.4104 140.834 58.3466 146.142 58.5903 150.865 58.6127 154.188 58.7894 155.589 59.2162 156.698 59.4557 157.335 59.7565 157.811 59.557 158.118 20 35 54 114 53.4526 118.535 53.251 123.702 52.3217 128.813 52.3285 134.454 53.5128 140.893 55.6801 147.5 57.5893 153.794 58.7708 160.461 58.764 167.38 57.6445 174.801 56.4898 181.878 54.8224 189.148 53.2748 195.595 52.7385 201.159 52.9738 205.719 20 35 84 124 83.7595 126.575 83.4774 130.309 84.6768 133.607 85.5404 136.169 84.4398 138.329 86.2302 141.422 87.8777 145.587 87.953 149.907 87.2646 154.624 86.5205 158.305 86.0462 161.224 85.5294 163.346 84.9503 165.699 85.4247 167.411 85.4129 169.426 20 35 104 154 103.001 159.295 95.4919 163.623 77.9742 167.589 62.8004 167.174 61.2309 173.098 60.392 179.719 59.2847 187.352 58.5932 195.038 58.8034 202.553 59.8788 209.976 61.185 217.566 64.69 223.85 66.8403 228.666 69.3443 232.717 71.531 235.907 20 35 144 124 143.69 129.1 143.785 134.106 144.217 139.465 145.196 145.399 146.008 151.976 145.549 159.284 144.13 163.13 143.823 167.257 144.81 173.67 145.617 177.678 146.346 180.523 146.902 183.158 147.145 185.428 147.277 187.608 147.333 189.628 20 35 19.799 132.936 18.7597 137.566 18.288 142.541 18.3082 148.081 18.8202 153.854 20.0389 160.068 21.9225 166.371 23.4931 172.821 24.8414 179.786 25.8025 187.102 26.0284 195.193 26.0054 203.673 26.4706 211.197 27.0581 217.54 28.459 222.852 29.7984 227.198 20 35 19.799 147.078 18.812 151.91 18.2771 156.58 18.088 161.989 18.5662 167.438 19.6103 173.331 21.4848 179.478 23.0594 185.998 24.0579 192.542 24.9388 199.684 25.1361 207.738 25.2447 217.006 25.7182 225.294 26.6278 232.693 27.9243 238.037 29.1516 241.534 20 35 76.3675 132.936 75.3709 138.29 74.3126 142.372 72.6588 146.48 71.7451 151.418 70.9913 157.645 71.5037 165.029 72.4075 172.673 72.9693 180.374 73.6428 187.883 73.8387 195.76 73.6967 203.358 73.7637 208.919 74.264 213.519 74.6204 217.871 75.0247 221.039 20 35 76.3675 175.362 76.2577 175.525 76.0071 179.507 74.2077 183.994 68.35 189.39 58.6059 189.219 54.3909 193.578 51.4491 201.421 50.15 208.617 50.18 215.234 49.4349 222.144 57.3628 230.315 62.0545 235.666 64.9027 240.421 68.3017 243.833 70.8086 246.76 20 35 90.5097 118.794 89.5253 123.547 88.5283 128.206 87.8561 131.904 88.5301 135.211 88.4534 138.353 91.2944 141.947 94.8101 146.386 97.7158 150.704 99.3935 154.453 99.9706 157.105 99.7639 159.479 99.5074 161.376 99.5444 163.018 99.6584 164.611 100.185 166.179 20 35 118.794 161.22 113.179 166.386 106.336 171.891 96.2974 184.322 84.1279 191.491 73.6494 192.966 66.8246 191.327 63.8409 195.96 62.5024 203.757 62.6051 211.157 62.8422 216.297 64.2004 225.682 68.5589 229.855 71.0992 234.347 74.3258 237.694 76.6791 240.481 20 35 39.598 96.1665 38.9068 100.414 38.7867 105.239 39.3032 110.211 40.6034 115.756 42.0245 122.121 44.0634 128.582 45.8786 135.154 46.9347 142.353 47.0598 149.724 45.5746 157.735 43.9448 165.218 42.3304 172.518 41.0409 179.081 41.208 184.627 41.4146 189.374 21 36 42 142 41.7383 145.248 41.8422 146.615 41.846 146.727 41.8851 146.879 41.3049 151.065 43.548 157.39 44.7358 164.244 45.7029 171.783 46.4901 179.622 45.9222 187.205 46.5818 194.636 46.3125 201.131 47.376 206.371 48.5153 211.187 50.0337 214.826 21 36 62 137 60.5914 142.655 60.3294 147.909 60.4736 153.303 60.8737 159.092 61.8873 165.23 63.2395 171.598 63.9093 177.907 64.5102 182.782 64.6728 188.061 64.3499 193.019 63.8928 197.799 63.591 202.128 63.3691 205.457 63.6864 209.377 63.6897 212.177 21 36 77 117 75.1619 120.204 75.7368 122.724 77.6687 125.353 80.7133 127.965 88.1518 130.954 88.8409 135.399 89.0009 140.166 89.0305 144.106 88.7678 147.518 88.5037 150.03 88.6886 151.611 88.8449 153.353 89.0066 155.031 88.7816 157.043 88.6066 159.192 21 36 77 157 73.7887 160.457 68.7021 164.048 63.7401 167.476 63.3944 173.761 63.0423 180.638 61.6019 188.544 60.789 195.727 60.5851 203.039 61.19 210.042 61.8458 215.964 64.4073 222.147 66.4141 226.748 68.5764 230.753 70.535 233.82 73.1991 236.287 21 36 77 177 77.0186 177.013 76.9748 177.031 72.9653 177.734 63.8579 174.621 63.327 181.437 61.8867 189.343 61.0737 196.525 60.8603 203.848 61.4568 210.891 62.0687 217.03 64.9809 223.28 67.0192 227.812 69.0861 231.505 70.9725 234.54 73.6204 236.95 21 36 97 112 96.7197 115.678 96.1544 119.384 97.1535 122.753 99.4044 127.729 105.545 133.773 108.513 139.129 111.032 143.214 112.847 146.23 113.412 148.486 113.281 150.764 113.644 153.059 114.059 155.003 114.462 156.51 114.889 157.608 115.397 158.778 21 36 127 167 122.054 170.017 112.766 179.655 101.158 183.023 92.0342 201.915 85.8842 209.736 83.2403 213.254 79.3347 221.175 78.1682 227.198 81.6941 227.946 83.4131 231.266 84.3438 235.009 85.2894 238.696 86.0051 240.98 86.9193 242.675 87.3437 244.188 21 36 73.5391 115.966 71.7087 119.148 73.8345 121.69 76.1905 124.33 79.7086 126.911 87.8535 129.892 88.8382 134.265 89.0231 138.987 89.1693 142.931 88.8896 146.274 88.6621 148.681 88.7933 150.248 88.9198 151.924 89.0652 153.609 88.9182 155.669 88.827 157.745 21 36 123.037 172.534 115.295 190.159 103.676 209.633 89.0821 228.404 74.1356 243.551 62.5295 251.648 55.7858 253.38 56.1309 248.667 63.9309 238.234 80.7356 225.533 84.0165 227.652 86.1576 231.049 88.7429 234.216 92.2285 235.996 93.9763 238.118 97.0245 239.526 21 36 74 114 73.5536 117.332 76.0397 120.122 77.7122 122.758 79.5995 125.33 83.3229 128.705 84.376 133.515 84.5196 138.297 84.8587 142.9 84.8501 146.473 84.991 149.023 85.6006 150.647 85.9098 152.38 86.366 154.105 86.3423 156.2 86.4866 158.33 21 36 74 124 74.1496 127.746 75.1645 131.526 76.2059 134.476 78.2168 137.08 80.2847 140.535 82.067 145.027 82.0343 149.49 82.0462 154.062 82.4871 157.65 82.6613 160.667 83.3837 162.41 83.3596 164.536 83.8482 166.283 84.0966 168.298 84.5086 170.201 21 36 74 134 73.7494 136.232 73.8452 140.24 74.7461 145.095 75.4027 150.359 76.7672 153.605 77.3632 158.558 77.5301 162.25 77.1401 166.305 76.6494 170.165 76.2569 174.136 75.8885 177.179 75.4724 180.066 75.436 182.154 75.4578 184.002 75.6005 185.714 21 36 33.9411 118.794 33.5815 123.773 33.7824 129.084 34.4251 135.081 35.353 141.361 37.2842 147.66 39.4282 154.395 41.0361 161.619 41.8636 169.279 42.0845 177.344 42.2996 184.846 42.9017 192.314 43.7054 198.686 45.5296 203.858 47.2944 208.272 48.5632 212.016 21 36 62.2254 118.794 62.881 123.443 62.4021 127.879 63.1172 132.072 64.2859 137.365 66.161 143.438 68.1666 149.498 69.2835 156.093 69.3437 162.574 68.5525 168.872 67.5137 175.643 66.6358 181.995 65.8701 187.367 65.5514 192.289 65.5568 196.926 65.9547 200.321 21 36 62.2254 132.936 61.3827 138.64 60.6263 143.963 60.7974 149.92 61.2613 156.47 62.4051 163.733 63.5337 171.282 64.331 179.436 65.195 187.416 65.5532 195.304 65.6513 203.33 66.2971 209.625 66.9034 215.079 67.8319 219.603 68.8012 223.214 69.9882 226.215 22 37 52 167 52.1749 169.038 52.4537 174.305 53.9135 179.786 55.8466 184.775 56.5347 191.795 56.779 199.492 56.5401 206.583 57.1003 213.89 57.3255 221.405 59.3705 227.374 60.605 232.82 62.4857 237.094 64.2272 240.38 65.5572 243.632 67.2706 246.055 22 37 57 137 56.965 141.402 57.4144 145.108 58.3857 149.528 60.0798 154.901 61.8626 161.011 62.8853 167.788 63.3948 174.286 63.4096 181.051 62.607 187.29 61.365 193.686 60.8846 198.945 60.7924 204.261 61.2187 208.411 61.3557 211.589 62.057 214.386 22 37 62 122 61.3561 126.231 63.4579 128.88 67.5746 131.463 68.6365 137.248 71.6099 141.565 75.1898 145.547 77.4814 148.661 79.0122 151.43 78.2086 154.807 78.0353 157.068 79.0701 157.902 79.7572 159.303 80.5554 160.84 81.1987 162.66 81.5804 164.459 22 37 77 117 77.4675 119.704 79.4425 122.402 81.7202 125.05 84.3974 128.616 84.3231 133.917 84.3016 138.891 84.7648 143.253 84.6098 146.616 84.1563 149.587 84.3541 151.635 84.2693 153.69 84.4619 155.589 84.5799 157.367 84.6348 159.502 84.4987 161.534 22 37 117 197 104.616 216.902 90.2014 233.611 75.096 249.567 62.8463 258.859 56.1469 260.519 56.9624 254.151 68.781 239.759 80.4884 232.066 85.9678 231.162 86.7962 235.29 87.6224 239.511 87.9877 242.157 88.9019 244.203 89.7635 245.792 91.0818 247.331 22 37 122 177 112.647 190.864 103.173 192.282 93.5283 208.216 87.9534 213.101 85.8473 215.599 84.7234 219.773 83.8256 225.044 84.079 229.587 84.1697 233.381 84.7228 237.158 85.1425 240.938 85.4017 243.154 85.7305 244.8 85.7985 246.181 85.9738 247.049 22 37 122 182 111.221 200.347 99.7248 210.791 90.843 214.456 86.9932 212.913 84.8855 215.449 82.1487 221.979 81.0569 227.748 80.4552 232.422 85.9346 231.518 86.571 235.648 86.9907 239.885 87.4648 242.465 88.2212 244.413 88.6456 245.986 89.3805 247.411 22 37 122 187 110.578 206.501 96.5814 223.937 86.2598 231.55 76.1244 237.829 74.7839 233.454 75.4192 231.627 78.785 231.663 78.8978 234.844 84.8788 233.388 85.4319 237.361 85.8516 241.14 86.1108 243.57 86.296 245.435 86.3641 246.816 86.4449 247.684 22 37 122 192 110.698 210.89 99.9601 222.76 91.6711 227.77 86.1485 224.914 85.7256 222.911 85.6843 224.036 85.793 226.973 85.4909 231.631 85.1526 235.573 85.3616 239.45 85.4525 242.874 85.4709 245.016 85.5893 246.638 85.6421 248.013 85.6713 248.851 22 37 127 177 118.557 189.349 110.538 188.883 108.185 192.649 98.9139 203.302 96.4452 206.874 99.2615 208.714 101.469 213.104 105.395 215.521 108.273 217.352 110.883 219.73 112.657 221.681 114.61 223.562 118.55 223.963 122.089 224.417 126.764 224.918 22 37 147 137 147.49 143.153 148.45 149.127 149.258 156.211 148.762 168.725 146.781 176.292 146.434 183.832 147.646 187.796 148.602 190.553 149.316 193.868 149.862 197.18 150.025 199.87 150.087 201.848 150.084 203.802 150.465 205.082 150.683 205.549 22 37 24.0416 130.108 23.8198 135.614 24.4577 141.318 25.7049 146.998 27.5972 153.562 29.1265 160.755 30.4438 168.355 31.568 175.35 31.8259 182.972 31.9879 190.271 32.2633 197.898 32.3133 204.883 32.9677 211.109 34.3601 216.366 36.6494 219.028 38.2448 221.635 22 37 45.2548 115.966 44.8319 121.186 45.5882 127.036 47.3723 133.268 48.9092 139.383 49.901 145.564 50.7667 151.762 51.2516 158.613 51.7927 163.83 51.6371 165.31 51.806 165.144 51.7425 164.94 51.8589 165.019 51.7521 164.875 51.7753 165.078 51.7858 164.95 22 37 59.397 144.25 58.5065 149.795 58.3986 155.289 58.5615 161.408 59.6442 168.057 60.5855 176.065 61.1691 184.2 61.7884 192.61 62.3863 200.993 62.8174 209.626 64.0916 215.846 65.5674 220.82 67.1571 225.18 68.5853 228.63 70.294 231.545 72.032 233.551 22 37 66.468 130.108 66.4432 134.017 67.3324 138.049 68.3446 143.128 69.91 147.925 70.2947 153.485 70.8773 158.947 70.9077 164.817 70.4291 170.916 69.8494 177.053 69.2954 182.534 69.0209 186.241 69.0386 188.961 69.0162 192.388 69.6271 195.106 70.1921 197.569 22 37 66.468 151.321 65.414 155.968 64.973 161.324 64.4864 166.942 64.4508 173.474 64.6289 181.235 64.954 188.421 65.4806 195.511 66.1318 202.716 66.2033 209.9 66.8762 215.496 67.8253 220.332 69.0228 224.367 70.0625 227.446 71.2055 229.97 72.505 231.977 22 37 80.6102 115.966 80.6807 118.891 81.2284 121.763 81.7076 124.304 82.0908 127.904 82.1721 133.207 82.1179 138.202 82.7236 142.67 82.7783 146.085 82.5572 148.859 82.6022 151.05 82.4938 153.07 82.678 155.003 82.8367 156.77 83.0179 158.851 83.069 160.78 22 37 80.6102 165.463 74.4281 169.632 62.0097 169.344 60.9574 175.404 60.4017 182.109 58.9426 189.764 58.4159 197.362 58.2913 204.726 58.9358 211.957 59.7772 218.882 62.591 225.11 64.629 229.725 66.8382 233.567 68.8961 236.768 71.1801 239.294 73.6104 241.26 22 37 101.823 179.605 92.2002 193.426 81.5224 204.073 70.2685 210.87 62.9787 208.132 59.4918 208.057 59.0617 208.091 59.3616 214.085 60.3522 217.514 61.4001 224.821 66.5056 230.912 68.6734 235.036 72.7014 238.782 74.0921 241.092 75.1184 242.969 76.8094 244.441 22 37 108.894 172.534 98.9704 185.455 87.8618 197.659 77.403 198.757 71.2605 199.132 68.6939 196.575 67.8771 201.54 67.8484 208.902 68.4278 214.158 69.694 220.45 74.0612 225.33 76.5714 229.244 79.6708 232.239 81.4317 234.53 82.5807 236.256 83.9224 237.232 22 37 115.966 193.747 103.745 213.678 87.7708 233.177 71.5837 250.425 58.5295 260.618 51.112 262.836 51.9682 255.917 67.5292 237.197 79.8521 228.266 83.8789 229.807 85.6213 233.377 87.0402 237.085 88.5689 239.502 90.5611 241.457 92.9817 243.017 95.3004 244.637 22 37 123.037 179.605 112.717 196.538 101.415 205.05 92.0937 208.771 86.4071 214.091 84.2631 215.875 82.4099 220.705 82.077 225.161 83.3268 228.922 84.4523 232.653 85.1332 236.5 85.9967 240.27 86.6668 242.788 87.2988 244.801 87.7562 246.42 88.2763 247.757 22 37 123.037 193.747 112.748 210.763 100.835 224.281 90.5575 231.659 83.3807 229.655 83.0172 226.713 82.9308 227.742 82.3761 231.887 82.2445 235.329 82.3681 238.117 82.3861 241.16 82.5288 244.239 82.6341 245.883 82.8348 246.931 82.9512 248.198 82.9895 248.841 22 37 34 114 34.2401 119.157 34.8161 125.076 35.7528 131.233 37.6213 137.351 40.1914 143.834 42.1445 150.672 42.394 158.333 41.3638 166.129 42.0446 172.925 42.9974 179.717 44.824 184.814 45.3876 188.167 45.4538 189.573 45.38 189.688 45.2558 189.88 22 37 74 154 70.9742 158.05 68.8876 162.776 66.1269 167.572 65.6288 174.305 65.3186 182.351 65.3382 190.234 65.9036 198.012 66.7809 205.243 67.9875 212.545 70.2791 218.132 71.8094 222.792 73.3202 226.376 74.2948 229.067 75.7703 231.279 77.4242 232.932 22 37 84 154 78.1851 156.139 75.4657 158.35 73.5635 160.731 73.7068 164.494 74.1205 168.647 74.8874 174.403 75.4078 180.507 75.3399 187.215 74.7069 193.33 74.3307 199.751 74.2775 204.418 74.328 208.669 74.2974 211.729 74.8071 214.836 74.9277 216.709 22 37 104 164 92.5618 173.156 76.3729 175.013 66.1788 172.707 65.2062 179.205 63.7876 187.094 62.8989 194.755 63.1091 202.228 63.9625 209.384 65.3088 216.722 68.9566 222.818 71.1068 227.616 74.5649 231.606 76.9869 234.737 78.5609 236.929 80.3338 238.505 22 37 104 174 94.454 186.58 82.3348 195.523 70.7092 195.001 63.783 190.656 61.8216 196.956 60.5247 204.555 60.5993 212.132 61.2774 216.951 62.4569 225.342 67.2268 230.44 69.6252 234.806 73.3875 238.504 75.8547 241.543 77.325 243.637 78.4504 244.676 22 37 104 184 93.536 202.086 81.3802 214.618 68.3904 226.181 61.8491 221.264 58.9114 216.39 58.2634 215.782 59.8713 216.339 60.7778 218.796 61.9574 227.528 66.7272 232.027 69.1779 236.393 72.7517 239.862 75.08 242.82 76.5504 244.914 78.1648 246.413 22 37 124 184 113.558 200.968 102.126 210.624 92.7017 213.556 88.1598 214.05 87.8547 212.118 85.3285 218.548 85.6568 222.519 87.4106 226.421 89.617 230.067 91.8305 233.339 94.8764 236.515 98.6615 238.475 101.935 239.629 105.094 241.005 108.565 242.3 22 37 33.9411 132.936 33.9912 138.28 34.5874 144.273 35.7006 150.49 37.6648 156.822 39.4784 163.603 40.979 170.911 41.9277 178.534 42.1837 186.633 42.2042 194.294 42.6517 201.928 43.2833 208.539 44.7834 213.898 46.2282 218.535 47.6355 222.227 48.8124 225.128 22 37 76.3675 132.936 75.7376 137.089 75.8913 140.843 76.575 145.757 78.1451 150.701 79.1703 155.641 79.6641 161.098 79.1157 167.501 78.2899 174.002 77.6296 179.929 77.0438 185.747 76.7781 190.877 76.5199 195.242 76.5343 197.925 76.9978 200.808 77.3522 203.297 22 37 76.3675 161.22 70.6803 164.775 61.4253 166.969 59.9783 172.971 59.3204 179.821 58.0053 187.609 57.6322 195.464 58.0416 203.051 59.0469 210.17 60.5071 218.654 64.039 224.56 66.4231 229.422 69.1956 233.272 71.3679 236.47 73.4909 238.925 75.5227 240.83 22 37 104.652 161.22 92.3177 169.713 78.0767 171.827 69.0009 172.608 67.2063 178.936 65.7188 187.023 64.9665 194.938 65.1737 202.663 65.9603 209.816 67.5377 217.758 71.4517 222.763 73.7029 227.271 76.2812 230.797 78.1115 233.584 79.6089 235.648 81.3225 237.298 22 37 118.794 189.505 107.522 208.031 93.1054 225.311 80.2424 238.12 69.8486 243.174 67.1375 239.587 67.0653 237.324 73.6765 231.572 79.3086 230.365 84.2612 230.82 86.8595 233.736 89.5836 236.913 92.9173 238.765 95.5303 240.564 98.1616 242.294 101.282 243.636 22 37 108 188 96.93 206.255 81.9338 226.063 66.0902 243.346 52.5597 254.22 44.4628 257.022 43.7753 251.422 50.6502 239.837 63.9643 226.613 68.1654 228.978 72.0399 232.377 74.6053 236.182 77.3186 238.655 79.371 240.6 81.2367 242.883 83.2064 244.898 22 37 124.451 181.019 117.466 184.542 107.265 188.828 101.187 197.36 93.6687 207.126 87.9941 213.78 86.7417 217.946 87.1169 222.503 89.642 225.451 94.3993 226.558 97.4906 228.654 99.7779 230.531 102.051 232.988 104.833 235.575 107.11 237.906 110.273 239.507 23 38 22 147 22.3822 152.232 23.6107 159.616 25.6729 165.52 27.1123 172.404 28.5572 179.334 29.4865 187.339 29.5252 195.517 29.5384 203.165 29.8972 210.334 30.4474 217.402 31.9971 223.287 33.3349 227.725 34.6077 230.404 36.0358 233.596 37.9063 235.184 23 38 27 157 27.5506 162.469 28.5736 168.512 30.5065 174.905 32.2742 181.385 33.4536 188.093 34.008 195.241 33.9594 202.742 34.1 211.815 34.7162 219.586 35.6493 226.346 36.8581 231.341 38.1706 234.967 39.4779 237.209 40.6633 238.645 42.1462 240.104 23 38 37 137 37.3615 139.536 38.5171 144.58 40.0855 150.42 42.3876 156.933 44.7506 163.927 45.7177 171.465 46.6453 179.379 46.6324 186.906 47.1688 194.338 46.8922 200.951 47.9312 206.198 49.0705 210.866 50.5889 214.505 49.9328 217.76 52.3241 219.519 23 38 42 152 42.3488 155.698 42.3886 156.013 42.3828 156.031 42.2159 159.022 43.4527 165.448 44.1884 173.024 44.7501 180.917 44.055 188.55 44.7372 195.633 44.6 202.092 45.7516 207.258 46.6537 212.18 47.6258 216.139 47.2455 219.24 50.0203 221.122 23 38 52 172 53.1153 174.543 54.598 179.868 56.4816 185.168 57.1697 192.188 57.414 199.885 57.1692 206.968 57.7294 214.274 58.1238 221.308 60.6426 227.1 62.6489 231.684 64.5952 235.589 66.4575 238.717 68.1427 241.58 69.9954 243.784 71.7126 246.046 23 38 62 122 63.7817 124.976 69.3353 126.985 70.0792 132.998 73.0097 137.406 76.716 141.291 78.9613 144.756 79.9335 147.827 79.6137 150.725 79.7078 152.725 79.8707 154.482 80.2212 156.154 81.09 157.481 81.8397 159.384 81.8069 161.359 82.001 163.076 23 38 67 172 63.5372 174.47 61.0976 178.818 60.1932 184.916 58.7396 192.393 58.3718 199.995 58.1399 207.241 58.7001 214.477 59.1519 221.222 62.0074 227.014 64.0137 231.532 65.9778 235.315 67.9048 238.424 70.1249 240.918 72.2475 242.892 73.8384 244.711 23 38 72 162 67.0815 164.542 65.5128 168.883 65.5091 174.388 65.315 176.344 65.8158 181.253 66.777 186.27 66.8491 193.062 66.7571 198.842 66.5537 204.168 66.5765 209.044 66.5764 213.342 66.7806 216.757 67.3424 219.692 67.5235 221.598 68.336 223.462 23 38 87 187 78.1314 193.989 71.9049 193.585 64.929 186.133 63.4503 193.647 62.5095 200.701 62.3432 208.635 62.9679 215.223 63.6302 221.612 69.5569 229.847 72.2292 233.319 76.2258 237.172 77.8574 239.445 78.7381 241.069 79.9556 242.176 80.6059 242.766 23 38 112 182 100.546 183.269 91.4225 202.162 85.2724 209.983 82.6738 213.498 78.7682 221.419 77.6017 227.442 81.1275 228.19 83.739 231.077 84.6505 234.967 85.5762 238.978 86.3141 241.404 87.2283 243.099 87.6527 244.612 88.3768 245.845 88.843 246.202 23 38 112 187 101.1 187.379 91.3509 207.085 85.5413 213.602 84.2618 214.066 80.4068 221.903 79.2868 227.92 79.8526 231.452 85.332 230.85 86.1604 234.902 87.0024 239.018 87.774 241.52 88.6883 243.566 89.5499 245.155 91.1719 246.694 92.6207 247.515 23 38 117 182 105.658 184.241 98.6753 197.167 90.3197 207.179 87.3125 211.419 82.1487 221.436 80.9084 227.536 80.3067 232.21 85.7861 231.306 86.6146 235.433 87.4407 239.655 87.9149 242.235 88.8291 244.281 89.6908 245.87 91.009 247.409 92.4578 248.23 23 38 66.468 144.25 66.5704 146.34 66.9066 149.245 68.2583 153.308 69.6436 160.01 70.6082 166.836 70.8132 172.878 70.7991 179.086 69.9877 184.519 68.9591 190.795 68.6891 195.725 68.604 198.96 68.6782 202.381 68.7215 204.989 69.1295 207.463 69.7385 209.37 23 38 87.6812 193.747 77.3361 202.038 66.9504 205.819 60.9838 205.078 58.2777 202.702 57.4376 206.321 57.5052 213.462 58.3306 216.891 59.5091 224.95 64.1349 230.585 66.323 234.997 68.9317 238.55 70.3669 241.034 71.8041 243.37 74.0586 245.308 75.586 246.623 23 38 94.7523 179.605 79.4151 183.349 66.4503 177.928 64.8879 182.989 63.345 190.931 62.4128 198.301 62.2712 205.809 62.8769 212.769 63.7795 219.502 68.6162 226.696 71.5299 231.503 75.7424 235.562 77.718 238.155 79.1921 240.205 80.4152 241.266 81.1413 242.053 23 38 94.7523 200.818 84.0983 211.911 71.9836 222.244 66.0981 218.848 62.2553 218.759 60.9369 218.905 63.0079 218.388 64.4516 219.817 63.9094 224.667 69.363 230.927 71.4695 233.786 75.6963 237.645 77.2654 239.727 78.3711 241.442 79.5942 242.54 80.2058 243.315 23 38 115.966 186.676 104.952 186.161 95.5572 202.919 90.7003 208.751 88.0913 212.451 84.8149 220.037 84.849 224.051 86.1721 228.212 87.1514 232.494 88.0679 236.61 88.9254 240.538 89.5626 243.227 90.2522 245.352 91.5797 247.046 93.0516 248.663 94.4433 249.443 23 38 104 204 90.1691 221.521 77.5662 233.487 68.7424 235.819 64.294 235.13 63.4675 234.503 69.3024 229.308 77.4266 225.006 81.4168 226.462 84.3892 229.269 86.9412 232.379 89.4634 234.508 91.5636 236.47 93.961 238.081 96.7335 239.441 99.4078 240.231 23 38 114 174 103.705 177.448 100.996 188.124 92.3296 198.987 89.5237 203.424 90.7291 207.224 88.6492 214.09 89.9798 218.403 93.4054 220.999 96.3677 223.551 101.623 223.778 108.527 221.88 113.328 221.951 116.909 222.96 121.536 223.248 125.563 222.842 23 38 114 194 102.945 201.235 93.3296 207.345 87.889 212.536 85.6256 214.333 82.5176 221.079 82.9534 224.802 85.1864 228.179 87.0549 231.918 88.6189 235.461 90.4622 238.946 92.9648 241.26 94.8183 243.267 97.6844 244.889 100.765 246.239 103.601 246.936 23 38 90.5097 203.647 75.2034 223.064 58.9876 240.369 45.3579 251.645 37.6253 254.051 36.3286 248.971 40.9298 239.501 52.3365 226.31 59.628 230.785 64.3197 235.179 67.0783 239.631 70.4773 242.89 72.8427 245.579 74.6217 247.644 76.274 249.645 77.7051 250.848 23 38 104.652 189.505 93.2531 198.551 83.6159 201.714 75.8516 211.067 70.7977 216.025 69.1868 219.648 70.5995 221.072 70.8909 224.521 72.9598 227.069 76.2162 229.704 78.4484 233.007 80.8613 235.282 82.6916 237.334 84.1378 239.157 85.8739 240.613 87.1758 241.673 23 38 28 128 28.8189 133.949 29.9458 140.14 31.8977 146.417 33.6638 153.084 35.0043 160.378 35.8201 168.139 36.0926 176.424 36.185 183.974 36.6621 191.366 37.3657 197.768 38.9307 202.975 40.5082 207.409 42.0171 210.908 42.9893 213.781 44.2471 216.285 23 38 68 128 68.6889 131.775 71.1315 135.781 74.8577 139.783 78.1949 144.031 80.8954 147.906 82.8065 151.858 84.1586 155.052 84.6973 157.52 85.4658 159.161 86.0559 160.753 86.7406 162.303 87.2535 164.077 88.0007 165.879 88.63 167.455 89.4102 168.683 23 38 88 168 74.5662 170.376 67.5992 173.975 65.3109 181.146 63.6247 189.134 62.8888 197.052 63.1379 204.64 63.3899 211.827 65.2799 220.447 69.2031 225.626 71.6362 230.498 74.437 233.731 76.5203 236.353 78.4704 238.649 80.4638 240.616 82.2819 242.041 23 38 108 208 94.046 225.673 81.2473 238.455 69.8222 245.711 67.2335 240.928 67.0592 239.316 71.6863 236.101 77.7137 234.252 84.9296 233.119 87.9802 235.603 90.6387 238.141 93.3072 239.801 95.2519 241.805 97.2049 243.978 99.7584 245.647 102.982 246.453 23 38 96.1665 181.019 84.6963 186.411 76.7112 191.59 70.6311 198.398 66.2317 204.713 63.7712 211.258 62.2273 218.105 60.0584 225.467 65.8288 230.18 69.4848 233.787 72.1988 238.476 74.7989 240.549 76.5637 242.663 77.3238 245.714 78.9491 248.624 80.7733 250.605 23 38 96.1665 209.304 82.1836 227.884 68.3221 243.34 55.1998 253.367 47.4341 255.561 46.9376 250.305 54.9206 238.319 67.2592 227.981 73.0295 230.328 76.0024 232.927 78.5457 236.461 81.0332 238.204 83.1349 240.587 83.5061 243.756 85.2663 246.507 87.2851 248.208 24 39 42 162 42.7179 168.181 44.2435 174.906 45.5086 181.669 46.5506 188.668 47.1606 196.268 47.633 202.404 47.7246 203.428 47.7218 203.432 47.579 204.442 48.3608 208.253 49.155 212.673 50.6734 215.868 49.9434 218.982 51.7404 220.706 52.7409 222.119 24 39 47 132 48.7123 138.617 49.8584 144.932 51.6719 150.848 54.2084 157.792 55.306 165.152 55.8042 172.841 54.3595 179.942 52.0511 187.232 50.1026 193.815 49.3337 199.188 46.7499 204.254 44.6721 209.074 46.6336 211.37 48.3459 213.494 49.5842 214.867 24 39 62 162 61.6841 168.076 62.0424 174.85 62.3461 182.118 62.7572 188.928 63.1029 195.251 63.7209 202.844 63.7985 210.408 64.5085 216.199 65.3283 221.015 66.4978 225.287 67.6844 228.619 68.953 231.502 70.4133 233.681 71.2869 235.358 71.7525 236.453 24 39 107 177 110.882 185.315 104.071 194.02 92.9306 203.122 96.8429 205.17 98.5946 209.837 102.776 212.317 106.48 213.799 109.347 216.018 111.07 217.978 112.715 220.122 116.62 220.552 120.651 220.768 124.881 221.401 129.033 221.468 131.843 221.756 24 39 112 182 110.152 188.572 100.88 199.234 91.8174 206.626 93.4644 210.063 90.8141 217.171 92.2317 221.089 93.9882 225.068 97.3658 227.292 102.9 227.276 109.034 226.584 117.14 223.098 121.322 223.227 125.552 223.753 129.838 223.948 132.625 224.066 24 39 52.3259 151.321 53.4104 157.675 54.8849 164.648 55.9928 172.681 56.8079 180.997 57.4709 189.441 58.0253 198.009 58.3701 206.703 59.4641 213.237 60.3063 218.596 61.7187 223.186 62.8948 226.797 63.9482 229.933 65.368 232.499 66.4177 234.475 67.1411 235.846 24 39 66.468 123.037 72.173 125.105 77.2433 129.005 77.7397 134.17 80.1932 138.435 81.14 142.792 81.4731 146.145 81.2664 148.966 81.2602 151.158 81.1518 153.108 81.3413 154.98 81.5 156.746 81.845 158.779 81.9229 160.734 81.9462 162.521 82.0801 163.738 24 39 34 154 35.241 159.89 36.9859 166.177 38.6255 172.708 40.0059 179.531 40.9833 186.771 41.1807 194.399 41.0401 201.883 41.222 208.818 41.7662 214.222 43.1374 219.154 44.4135 223.129 45.9448 226.41 47.2811 229.166 48.4506 231.299 49.6089 232.946 24 39 64 134 65.443 138.657 67.6262 144.192 69.7598 149.467 71.0644 155.606 71.3369 161.114 70.8011 166.416 70.1593 171.211 69.785 175.337 69.4366 178.932 69.3944 181.824 69.4521 184.278 69.968 185.99 70.4085 187.576 70.8675 189.619 71.4104 190.504 24 39 64 144 64.7408 150.231 66.5122 156.684 68.178 163.461 69.0979 170.975 69.7525 177.992 69.87 185.504 69.5638 192.497 69.1627 199.394 69.008 205.07 69.1208 209.867 69.3434 213.547 69.8745 216.941 70.251 219.311 70.8665 221.421 70.9475 222.905 24 39 74 224 58.598 239.22 45.0122 250.268 37.7882 251.917 35.0247 249.322 38.8149 241.54 48.9764 229.52 60.4495 226.438 65.2194 231.626 67.6701 235.993 70.949 239.493 73.2146 242.599 74.9271 244.864 76.931 246.716 78.2887 247.882 79.2136 248.855 24 39 94 184 85.866 198.94 79.3134 207.333 74.9966 211.711 73.4671 215.954 72.9241 221.445 73.9644 224.38 76.5122 226.524 79.7903 229.158 81.9196 232.284 84.2317 234.476 85.932 236.496 87.185 238.188 88.8374 239.566 90.0464 240.49 90.6781 241.219 24 39 104 184 97.3797 196.669 89.4502 206.929 86.427 211.142 83.3191 218.275 83.9121 222.104 85.8978 225.828 88.3913 229.305 90.6048 232.666 93.7945 235.549 97.5796 237.51 100.854 238.664 103.941 240.096 107.484 241.357 111.363 241.597 113.48 242.088 24 39 33.9411 161.22 35.135 167.391 37.1774 173.842 38.8996 180.683 40.4171 187.63 41.5033 194.977 41.7435 202.886 41.5461 211.815 42.0354 220.195 42.8247 227.793 44.418 233.524 46.0264 237.608 47.8073 240.387 49.5003 242.423 51.3295 243.624 52.5078 244.673 24 39 48.0833 118.794 48.8359 124.716 50.7774 130.616 52.3774 136.908 52.9524 142.646 52.6709 148 51.5124 153.457 51.1829 155.997 51.3611 157.355 51.536 157.8 51.7645 157.823 51.5245 157.803 51.6295 157.812 51.6414 157.698 51.5829 157.889 51.627 157.872 24 39 90.5097 189.505 81.2732 196.063 74.1875 203.399 69.7775 207.397 68.1093 213.184 68.1178 218.317 68.0619 222.311 68.3384 226.12 72.2255 229.298 74.4577 232.768 77.4306 235.916 79.57 238.477 81.0162 240.361 82.7523 241.817 84.0541 242.899 85.2277 243.839 24 39 90.5097 231.931 75.1787 248.082 61.4696 259.553 53.5432 262.397 53.1574 256.806 63.2319 243.272 79.4162 230.075 84.3689 230.53 86.9672 233.446 89.6912 236.624 93.0249 238.475 95.6379 240.275 98.2693 242.005 101.389 243.346 104.675 243.957 106.416 244.656 24 39 104.652 175.362 103.046 184.235 95.0271 194.694 89.5844 201.02 90.648 204.705 90.3985 210.41 92.9508 213.945 97.4123 215.617 101.255 217.47 103.777 219.106 106.865 220.488 111.553 220.652 114.613 222.053 120.045 221.598 122.659 222.497 123.783 223.891 25 40 27 147 28.8443 153.618 30.3679 160.894 31.7162 168.417 32.8368 175.365 33.0613 182.984 33.3489 190.122 33.6006 197.691 33.4281 204.988 33.9098 211.381 35.3965 216.564 38.0376 218.906 39.487 221.718 40.2815 225.147 41.4405 226.873 42.2494 228.411 25 40 32 152 33.9525 154.663 36.0485 160.485 37.2518 167.75 37.8945 175.29 37.9456 183.295 38.0797 190.623 38.5406 198.095 38.9603 204.797 40.6374 209.829 42.613 213.724 42.3718 218.138 43.644 221.135 44.5749 224.261 45.9107 225.645 47.344 226.513 25 40 32 162 32.0223 162.062 33.7005 164.648 36.371 168.841 37.2733 176.032 37.4638 183.892 37.7772 191.036 38.2382 198.508 38.5744 205.29 40.2515 210.323 42.7324 214.131 42.4912 218.545 43.7362 221.542 44.4071 224.926 45.7885 226.308 47.2218 227.176 25 40 32 172 33.9226 178.392 35.883 185.421 37.3206 192.434 37.9996 199.498 37.9426 207.193 37.6142 216 38.6748 224.322 40.0761 231.904 41.6793 237.317 42.9486 240.761 44.2741 243.161 45.7937 245.24 46.5531 245.79 47.3331 246.711 48.2543 247.698 25 40 42 137 43.7344 143.134 45.3303 149.36 47.9965 156.147 48.602 163.063 48.8682 169.861 50.7389 176.881 51.7567 183.687 50.8443 188.843 50.0997 190.525 50.0693 190.408 50.1002 190.385 50.1054 190.291 50.1109 190.393 50.0463 190.195 50.1051 190.2 25 40 57 187 56.9995 187.793 57.7446 193.706 57.4253 201.241 57.1253 208.398 57.6598 215.593 58.0608 223.137 60.9188 228.449 62.8793 233.013 64.7601 236.823 66.5452 239.942 68.2092 242.511 69.9685 244.752 71.1801 247.157 72.233 248.254 73.2231 249.395 25 40 62 147 63.7096 152.284 65.3997 158.351 66.3055 165.189 66.7959 171.732 66.5622 178.072 66.1098 183.487 65.053 189.49 64.3111 194.49 64.1842 198.955 64.5614 203.159 64.8307 206.211 65.604 209.045 66.281 211.124 66.5487 212.612 67.012 213.384 25 40 62 187 59.8117 187.275 58.3879 194.549 57.9276 201.689 57.6576 209.033 58.1937 215.746 58.5946 223.291 61.9988 228.661 63.9369 233.067 65.8683 236.742 67.7306 239.805 69.6637 242.179 71.6793 244.141 72.9355 246.032 74.0757 246.826 75.1416 247.931 25 40 62 247 49.9868 255.911 42.0756 258.812 41.3653 253.577 44.7588 246.117 53.3203 234.016 65.2928 222.336 71.2195 230.571 73.5131 232.991 77.2878 236.589 78.7619 238.589 79.9193 240.395 81.2207 241.502 81.8709 242.031 81.8694 242.675 81.5826 243.205 25 40 67 177 65.8229 182.294 64.4679 190.707 63.521 197.926 63.3702 205.541 64.0068 212.612 64.6995 219.098 70.4328 227.417 73.78 232.361 77.5489 235.861 79.2747 237.919 80.7577 239.919 81.7711 240.792 82.506 241.351 82.6832 242.01 82.3692 242.473 25 40 67 182 64.3627 183.286 62.942 191.215 62.0357 198.45 61.8323 205.853 62.4121 212.925 62.9542 219.207 67.7286 226.841 70.8922 231.702 74.8885 235.555 76.5848 238.181 77.4655 239.805 79.7266 241.635 80.3768 242.225 80.3753 242.847 80.36 243.522 25 40 102 192 92.8406 203.179 90.3535 207.211 90.6883 211.774 87.9716 219.006 89.7874 222.746 91.9351 226.515 95.7383 228.555 101.446 228.721 107.649 228.058 116.007 224.314 120.434 224.306 124.587 224.705 128.767 225.255 131.448 225.52 135.074 224.925 25 40 66.468 137.179 67.9838 142.95 70.1584 147.933 71.5845 153.301 72.3394 157.91 72.2326 162.071 71.4734 166.274 71.3997 169.265 71.7286 171.636 71.7232 174.034 72.228 175.846 72.7519 177.571 73.2449 179.057 73.7525 180.806 74.2772 181.647 74.5516 182.217 25 40 66.468 250.316 52.5171 261.926 44.0292 265.007 42.9749 259.573 47.0118 249.037 56.6127 236.167 71.4303 228.236 74.7259 230.638 77.2165 233.809 80.4443 236.462 82.0215 238.12 83.1691 239.774 84.2577 240.596 84.7825 241.071 85.127 241.687 85.0816 242.137 25 40 54 134 56.159 140.599 57.9269 146.736 58.9923 153.141 58.9182 159.939 57.8307 167.38 56.8226 174.404 55.9905 181.283 55.2739 187.267 55.062 192.755 55.172 197.377 55.509 200.732 55.9819 203.745 56.6619 206.164 57.0416 207.799 57.4052 209.004 25 40 64 124 66.5499 128.921 69.5833 133.023 73.0812 136.953 75.4696 140.251 76.4463 143.311 77.1848 145.669 77.3193 147.574 77.3567 149.494 77.6873 151.323 77.7044 153.018 77.9758 155.038 78.2833 156.793 78.3935 158.45 78.4729 159.576 78.5419 160.314 25 40 64 244 50.3744 255.066 41.921 258.251 40.7151 252.97 45.0588 243.621 55.2414 230.788 63.5253 224.6 68.2556 230.163 70.654 234.529 74.4163 238.227 76.8834 241.265 78.3538 243.359 79.4792 244.399 80.2711 245.157 80.9831 245.976 81.7318 246.835 25 40 74 184 69.057 182.513 67.3931 190.289 66.3606 197.865 66.3455 205.488 66.9586 212.372 68.3361 219.796 72.8329 225.598 75.1924 229.894 78.6039 233.016 80.7175 235.608 81.9603 237.563 83.4141 238.807 84.4737 239.863 85.0926 240.587 85.4975 241.171 25 40 74 244 62.0472 252.732 54.9492 254.726 55.6599 248.991 65.7362 236.312 78.6047 226.799 82.5948 228.255 85.4112 231.165 87.9633 234.341 90.8134 236.469 92.8995 238.435 95.4957 240.026 98.5761 241.345 101.449 242.041 102.99 242.797 104.738 243.033 25 40 62.2254 147.078 63.9157 153.672 65.8536 160.112 66.8909 167.385 67.3578 174.45 67.1539 181.952 66.4356 188.868 65.6197 195.833 65.0696 201.757 64.9706 206.939 65.1812 211.285 65.6984 214.899 66.2235 218.018 66.8778 220.42 67.0298 222.229 67.5061 223.542 26 41 42 167 41.9923 166.99 42.4729 167.034 43.1246 173.949 43.1671 181.819 42.472 189.32 43.1426 196.629 43.1285 202.902 44.4966 207.772 45.3987 212.704 45.1587 217.169 46.5113 219.993 48.9659 221.91 49.8965 223.526 50.8887 224.561 51.3153 225.2 26 41 47 162 47.1378 162.13 45.7684 166.413 46.4973 173.421 47.2222 181.158 46.155 188.64 46.8256 195.406 46.5562 201.901 47.5156 207.056 48.4928 211.685 50.0627 215.309 49.4066 218.615 51.8888 220.378 53.0098 221.831 53.8564 222.965 54.1849 223.608 26 41 47 167 47.9784 168.738 47.9874 168.756 46.1021 173.835 46.6637 181.352 45.5965 188.834 46.2671 195.6 46.0393 201.969 47.0926 207.188 48.0698 211.912 49.6397 215.536 48.9098 218.798 51.392 220.56 52.513 221.995 53.3596 223.13 53.8227 223.718 26 41 52 147 53.6372 153.059 55.6032 159.819 55.9667 166.956 55.3944 174.562 53.7884 181.689 50.9686 188.96 49.9767 195.555 50.1512 200.91 48.1412 205.866 47.1384 210.312 48.4796 212.919 50.4995 215.094 51.5835 216.675 52.9609 217.506 53.2027 218.222 26 41 31.1127 144.25 33.1386 150.902 34.2674 158.704 34.9258 166.246 34.7343 174.397 34.3881 181.693 35.1781 188.975 35.7247 195.302 38.2509 199.813 39.4132 204.16 39.4655 208.395 40.844 210.989 43.0677 213.15 43.8102 214.654 44.7411 215.631 44.743 216.36 26 41 31.1127 158.392 32.6822 165.493 34.0424 172.734 35.0753 179.899 35.2526 187.855 35.3844 195.263 35.6988 202.777 36.2435 209.069 37.3427 214.715 38.4704 220.358 40.2108 223.716 41.7031 226.277 42.7694 228.938 43.824 230.853 44.662 232.331 45.1264 233.034 26 41 31.1127 165.463 32.6651 171.551 34.0479 178.411 34.9218 186.172 35.0719 193.811 35.0497 201.608 35.3793 208.886 36.0065 214.649 37.4253 219.154 38.5003 224.802 39.7757 228.548 41.0958 231.529 43.0291 232.326 44.0637 234.16 44.9032 235.459 45.3541 236.179 26 41 45.2548 144.25 47.3689 150.582 49.5472 157.719 50.8232 165.135 51.2329 172.924 52.2647 180.486 50.2811 188.008 48.153 194.673 49.4071 199.903 48.9594 204.754 48.4232 208.847 48.7413 211.888 51.154 213.984 51.7311 215.684 52.7951 216.726 52.9065 217.493 26 41 45.2548 151.321 47.6895 157.812 49.2347 164.905 50.2615 172.312 50.3366 180.085 50.8157 187.588 51.1128 194.852 49.2647 201.334 50.1639 206.555 51.2752 211.268 51.6085 215.111 51.4905 218.334 53.4751 220.473 54.0184 222.007 54.803 223.217 54.9634 223.975 26 41 66.468 165.463 67.0055 171.373 67.674 177.69 68.205 185.417 68.2654 193.663 68.2807 200.978 68.2004 207.444 68.4232 212.425 68.7053 216.998 68.9857 220.299 69.8054 223.184 70.3653 225.521 70.9737 227.24 71.1173 228.325 71.56 229.285 71.4565 229.662 26 41 24 174 25.5488 180.239 26.9038 187.321 27.6216 194.519 27.7441 202.457 27.844 211.749 28.3904 219.878 29.1491 226.72 30.4515 232.085 31.6284 235.487 32.9685 237.677 34.1643 239.196 35.9644 241.233 37.0509 242.433 37.9831 243.499 38.4351 244.022 26 41 68 148 69.8 154.229 70.7272 161.216 70.8542 167.94 70.0786 175.236 68.925 182.199 67.8974 189.034 66.9356 194.884 66.5124 199.991 66.4038 204.375 66.7766 207.881 67.1245 210.996 67.5898 213.492 67.7187 215.336 68.1709 216.637 68.1939 217.431 27 42 32 187 32.8855 193.597 33.4247 200.792 33.5941 208.737 33.6291 217.794 34.4058 226.119 35.6394 233.812 37.2907 239.486 38.8545 243.648 40.3187 246.284 42.161 248.603 43.2609 249.561 44.167 250.486 45.1876 251.753 45.3968 252.199 45.5629 252.62 27 42 62 177 62.5127 184.768 63.0281 192.679 63.5199 200.668 63.5214 208.905 64.0447 214.963 64.6297 220.022 65.532 224.371 66.5951 227.739 67.7213 230.82 68.967 233.241 69.7537 235.127 70.2867 236.287 70.9046 237.308 71.0182 237.89 70.9862 238.106 27 42 24.0416 179.605 25.6108 187.252 26.1743 194.326 26.2621 202.148 26.3359 211.397 26.7685 218.735 27.4052 223.92 28.4228 228.08 29.4806 230.786 30.8338 232.724 32.2085 235.257 34.097 237.46 35.103 238.604 35.9055 239.384 36.2885 239.803 36.3186 240.068 27 42 38.1838 165.463 39.5456 169.261 40.7492 175.872 41.0691 183.835 40.8669 191.439 41.3282 198.942 41.8148 205.522 43.4838 210.626 44.6969 215.239 44.9435 219.37 46.1856 222.308 47.7226 224.765 48.8189 226.345 49.9978 227.375 50.3689 228.092 50.5204 228.259 27 42 66.468 165.463 67.3421 172.094 67.931 179.13 68.0212 186.854 67.8402 194.023 67.5661 200.978 67.5115 206.553 67.4613 210.996 67.709 214.534 68.3091 217.625 68.4498 219.571 69.236 221.592 69.2828 222.482 69.5954 223.32 69.4593 223.66 69.2588 223.613 27 42 64 154 65.2367 160.787 65.3602 167.593 64.6172 174.737 63.782 181.534 62.7039 188.074 61.7524 193.801 61.3534 199.196 61.5774 203.715 61.7214 207.116 62.2675 210.238 62.8894 212.705 63.079 214.374 63.4375 215.689 63.4794 216.442 63.3774 216.723 28 43 32 162 32.5252 169.734 32.6535 177.607 32.6984 184.594 33.0123 192.229 33.2641 198.851 34.2807 204.455 37.3745 207.201 38.5767 209.334 39.3378 210.741 39.9402 211.215 40.036 211.652 40.0439 211.661 40.0296 211.637 40.055 211.696 39.9972 211.663 28 43 37 172 39.9292 174.827 39.7686 182.697 39.5474 190.128 40.1026 197.437 40.6489 203.874 42.5462 208.514 43.6846 213.077 43.4446 217.548 44.761 220.458 46.6829 222.932 47.5815 224.611 48.9197 225.484 49.2567 226.196 49.331 226.414 48.3663 226.614 28 43 42 157 42.3117 164.51 42.0994 172.386 43.6895 179.286 44.4394 186.601 44.5078 192.986 46.0065 198.009 44.9183 203.138 43.6688 207.687 45.6303 209.645 47.3188 211.513 48.217 212.583 48.5581 212.882 48.5155 213.043 48.5903 213.033 48.4973 213.077 28 43 45.2548 172.534 45.2457 174.581 45.7549 182.465 45.3753 190.127 46.0745 197.53 45.8306 203.939 47.2599 209.05 48.3922 213.763 48.9966 217.684 48.7646 220.885 50.7456 223.105 51.6848 224.612 52.6932 225.709 53.005 226.442 53.1541 226.634 52.4205 226.698 28 43 64 154 64.0583 160.429 63.2573 167.321 62.4049 174.129 61.8149 180.367 61.3023 185.681 61.1354 190.526 61.1901 195.263 61.7508 198.472 62.1635 201.635 62.6848 204.253 63.1695 205.84 63.6946 206.793 63.8154 207.437 63.6858 207.775 63.4054 207.675 29 44 48 168 47.934 175.934 47.8508 183.307 48.0299 190.604 48.2881 196.911 48.8364 202.271 50.0682 206.714 50.9536 210.349 51.5801 213.438 52.3474 216.022 52.9997 217.757 53.6896 219.063 54.0282 219.807 53.8552 220.124 53.2765 220.069 52.5702 219.389 "

In [11]:
def video_covs(path,trayectorias,area):
    #tr_list = mi lista de trayectorias 
    #En el tensor P obtengo listas de d*w*h
    #En el tensor Q obtengo listas de n*w*h, donde n es el numero de elementos de la diagonal superior
    #tr_matrix es una matriz de trayectorias con dimension [nro de trayectorias][nro de frames*2] (x, y)
    #list_cov es una lista con todas mis covarianzas, donde tiene [nro del frame-15][nro de covarianzas por frame]
    #Obtengo como salidas, una lista de listas con todas mis covarianzas, y una matriz de trayectorias, de donde las puedo extraer
    #Para ejecutar esta funcion debo tener antes, mis caracteristicas integrales int_great_array
    
    flujo,f,great_p,great_q,Q=[],[],[],[],[]
    images = [path+image for image in os.listdir(path)]
    n_images = len(images)

    if isinstance(sqrt(n_images)%1,float):limits = int(sqrt(n_images))+1 
    else:limits = int(sqrt(n_images))  
    for i,image in enumerate(images):
        im = Image.open(image)
        a='Trayectorias_densas/FramesP/frame'+str(i+1)+'.jpg'
        if(i<10):
            zeros='0000000'
            b='Trayectorias_densas/FramesF/'+zeros+str(i+1)+'.ppm'
            flujo.append(b)
        else:
            zeros='000000'
            b='Trayectorias_densas/FramesF/'+zeros+str(i+1)+'.ppm'
            flujo.append(b)
            
        f.append(a)    
        try: 
            #print(cv2.imread(flujo[i]).shape)
            x,p,q=int_great_array(cv2.imread(f[i]),cv2.imread(flujo[i]))
        except:
           #xf=cv2.imread(flujo[i-1])
            x,p,q=int_great_array(cv2.imread(f[i]),cv2.imread(flujo[i-1]))
                                  
        great_p.append(p)
        great_q.append(q)
    #Obtengo el tensor P y Q
    
    
    d=great_p[0].shape[0]
    tr_list=cortar(trayectorias)
    #nro numero de trayectorias del video
    nro=len(tr_list)
    #nro2 numero de frames del video*2
    nro2=n_images*2
    tr_matrix=np.zeros((nro,nro2))
    x,y=tr_matrix.shape
    for i in range(0,x):
        k=2
        for j in range(int((tr_list[i][0]*2)),int(tr_list[i][0]*2)+32):
            tr_matrix[i][j]=tr_list[i][k]
            k=k+1 
    #Obtengo mi matriz de trayectorias segun el caso
    
    #El 32 debido a que empieza en el frame 16 y termina 16 frames antes del final.
    list_cov=[]
    for i in range(32,nro2-32,2):
        #aca iteran frames
        cov_fr=[]
        for j in range(nro):
            #aca iteran trayectorias
            if(np.float32(tr_matrix[j][i])!=0):
                temp=int(i/2)
                y1=int(tr_matrix[j][i]+area)
                y2=int(tr_matrix[j][i]-area)
                x1=int(tr_matrix[j][i+1]+area)
                x2=int(tr_matrix[j][i+1]-area)
                Q=great_q[temp]
                P=great_p[temp]
                cov=matriz_covarianza(P,Q,y1,x1,y2,x2)
                #cov_fr.append(cov)
            else:
                cov=np.zeros((d,d))    
            cov_fr.append(cov)
        list_cov.append(cov_fr)
    #Obtengo list_cov
    
    return list_cov,tr_matrix,great_p,great_q

In [15]:
def video_covs_x16(path,trayectorias,area):
    #tr_list = mi lista de trayectorias 
    #En el tensor P obtengo listas de d*w*h
    #En el tensor Q obtengo listas de n*w*h, donde n es el numero de elementos de la diagonal superior
    #tr_matrix es una matriz de trayectorias con dimension [nro de trayectorias][nro de frames*2] (x, y)
    #list_cov es una lista con todas mis covarianzas, donde tiene [nro del frame-15][nro de covarianzas por frame]
    #Obtengo como salidas, una lista de listas con todas mis covarianzas, y una matriz de trayectorias, de donde las puedo extraer
    #Para ejecutar esta funcion debo tener antes, mis caracteristicas integrales int_great_array
    
    flujo,f,great_p,great_q,Q=[],[],[],[],[]
    images = [path+image for image in os.listdir(path)]
    n_images = len(images)

    if isinstance(sqrt(n_images)%1,float):limits = int(sqrt(n_images))+1 
    else:limits = int(sqrt(n_images))  
    for i,image in enumerate(images):
        im = Image.open(image)
        a='Trayectorias_densas/FramesP/frame'+str(i+1)+'.jpg'
        if(i<10):
            zeros='0000000'
            b='Trayectorias_densas/FramesF/'+zeros+str(i+1)+'.ppm'
            flujo.append(b)
        else:
            zeros='000000'
            b='Trayectorias_densas/FramesF/'+zeros+str(i+1)+'.ppm'
            flujo.append(b)
            
        f.append(a)    
        try: 
            #print(cv2.imread(flujo[i]).shape)
            x,p,q=int_great_array(cv2.imread(f[i]),cv2.imread(flujo[i]))
        except:
           #xf=cv2.imread(flujo[i-1])
            x,p,q=int_great_array(cv2.imread(f[i]),cv2.imread(flujo[i-1]))
                                  
        great_p.append(p)
        great_q.append(q)
    #Obtengo el tensor P y Q
    
    
    d=great_p[0].shape[0]
    tr_list=cortar(trayectorias)
    #nro numero de trayectorias del video
    nro=len(tr_list)
    #nro2 numero de frames del video*2
    nro2=n_images*2
    tr_matrix=np.zeros((nro,nro2))
    x,y=tr_matrix.shape
    for i in range(0,x):
        k=2
        for j in range(int((tr_list[i][0]*2)),int(tr_list[i][0]*2)+32):
            tr_matrix[i][j]=tr_list[i][k]
            k=k+1 
    #Obtengo mi matriz de trayectorias segun el caso
    
    #El 32 debido a que empieza en el frame 16 y termina 16 frames antes del final.
    list_cov=[]
    print(tr_matrix.shape)
    for i in range(nro):
        #iteran trayectorias
        cov_fr=[]
        Qx16=[]
        Px16=[]
        T_area=[]
        print(i)
        for j in range(32,nro2-32,2):
            if(np.float32(tr_matrix[i][j])==0):
                        
            else:
                cov=np.zeros((d,d)) 
                
            if i==626:
                print(Px16)
            cov=matriz_covarianza_x16(Px16,Qx16,T_area)    
            cov_fr.append(cov)    
        
            
        list_cov.append(cov_fr)
    #Obtengo list_cov
    
    return list_cov,tr_matrix,great_p,great_q

IndentationError: expected an indented block (<ipython-input-15-8968db9b75d7>, line 69)

In [16]:
path="Trayectorias_densas/FramesP/"
area=20

### Compilado de Covarianza volumetrica

##### Estos son resultados de covarianza volumetrica para una covarianza por trayectoria, del metodo modelado de covarianza por profundidad modelado de la siguiente forma

$$Cov_{(ij)}^{\Sigma^{t}}=\frac{1}{(n*16)-1}(Q_{tr1}+Q_{tr2}+...Q_{tr16})-\frac{1}{n*16}(P_{tr1}+P_{tr2}+...P_{tr16})(P_{tr1}+P_{tr2}+...P_{tr16})^{T}$$

#### Donde:

$$ X_{tr1}=X_{x^{x'y'}}+X_{x^{x''y''}}-X_{x^{x''y'}}-X_{x^{x'y''}} $$

In [17]:
def video_covs_x16(path,trayectorias,area):
    #tr_list = mi lista de trayectorias 
    #En el tensor P obtengo listas de d*w*h
    #En el tensor Q obtengo listas de n*w*h, donde n es el numero de elementos de la diagonal superior
    #tr_matrix es una matriz de trayectorias con dimension [nro de trayectorias][nro de frames*2] (x, y)
    #list_cov es una lista con todas mis covarianzas, donde tiene [nro del frame-15][nro de covarianzas por frame]
    #Obtengo como salidas, una lista de listas con todas mis covarianzas, y una matriz de trayectorias, de donde las puedo extraer
    #Para ejecutar esta funcion debo tener antes, mis caracteristicas integrales int_great_array
    
    flujo,f,great_p,great_q,Q=[],[],[],[],[]
    images = [path+image for image in os.listdir(path)]
    n_images = len(images)

    if isinstance(sqrt(n_images)%1,float):limits = int(sqrt(n_images))+1 
    else:limits = int(sqrt(n_images))  
    for i,image in enumerate(images):
        im = Image.open(image)
        a='Trayectorias_densas/FramesP/frame'+str(i+1)+'.jpg'
        if(i<10):
            zeros='0000000'
            b='Trayectorias_densas/FramesF/'+zeros+str(i+1)+'.ppm'
            flujo.append(b)
        else:
            zeros='000000'
            b='Trayectorias_densas/FramesF/'+zeros+str(i+1)+'.ppm'
            flujo.append(b)
            
        f.append(a)    
        try: 
            #print(cv2.imread(flujo[i]).shape)
            x,p,q=int_great_array(cv2.imread(f[i]),cv2.imread(flujo[i]))
        except:
           #xf=cv2.imread(flujo[i-1])
            x,p,q=int_great_array(cv2.imread(f[i]),cv2.imread(flujo[i-1]))
                                  
        great_p.append(p)
        great_q.append(q)
    #Obtengo el tensor P y Q
    
    
    d=great_p[0].shape[0]
    tr_list=cortar(trayectorias)
    #nro numero de trayectorias del video
    nro=len(tr_list)
    #nro2 numero de frames del video*2
    nro2=n_images*2
    tr_matrix=np.zeros((nro,nro2))
    x,y=tr_matrix.shape
    for i in range(0,x):
        k=2
        for j in range(int((tr_list[i][0]*2)),int(tr_list[i][0]*2)+32):
            tr_matrix[i][j]=tr_list[i][k]
            k=k+1 
    #Obtengo mi matriz de trayectorias segun el caso
    
    #El 32 debido a que empieza en el frame 16 y termina 16 frames antes del final.
    list_cov=[]
    print(tr_matrix.shape)
    for i in range(nro):
        #iteran trayectorias
        cov_fr=[]
        Qx16=[]
        Px16=[]
        T_area=[]
        print(i)
        for j in range(32,nro2-32,2):
            if(np.float32(tr_matrix[i][j])==0):
                cov=np.zeros((d,d))        
            else:
                temp=int(j/2)
                temp_area=[]
                y1=int(tr_matrix[i][j]+area)
                y2=int(tr_matrix[i][j]-area)
                x1=int(tr_matrix[i][j+1]+area)
                x2=int(tr_matrix[i][j+1]-area)
                Q=great_q[temp]
                Qx16.append(Q)
                P=great_p[temp]
                Px16.append(P)
                temp_area.append(y1)
                temp_area.append(x1)
                temp_area.append(y2)
                temp_area.append(x2)
                T_area.append(temp_area)   
                cov=matriz_covarianza_x16(Px16,Qx16,T_area) 
                
            #if i==626:
            #    print(Px16)
                
            cov_fr.append(cov)    
        
            
        list_cov.append(cov_fr)
    #Obtengo list_cov
    
    return list_cov,tr_matrix,great_p,great_q

In [18]:
covs,matrix,great_p,great_q=video_covs_x16(path,trayectorias,area)

(944, 120)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274

In [ ]:
covs,matrix,great_p,great_q=video_covs(path,trayectorias,area)

In [19]:
print(len(covs))
print(matrix.shape)
print(len(great_p))
print(len(great_q))
print(great_p[0].shape)
print(covs[0][0].shape[0])
print(great_q[0].shape)

944
(944, 120)
60
60
(7, 273, 437)
7
(28, 273, 437)


In [21]:
Q=great_q[30]
P=great_p[30]
y1=272
x1=436
y2=0
x2=0

In [22]:
cov=matriz_covarianza(P,Q,y1,x1,y2,x2)

In [20]:
max_matrix,min_matrix=normalized_matrices(covs)

## Matrices de covarianza con flujo, de una trayectoria

In [21]:
import random
rand1= random.randint(0,matrix.shape[0])
rand2= random.randint(0,len(covs)+1)
while(matrix[rand1,rand2+32]==0):
    rand1= random.randint(0,matrix.shape[0])
    rand2= random.randint(0,len(covs)+1)   
    
print(rand2)

while(matrix[rand1,rand2+32]!=0):
    rand2=rand2-1  
    
ran1=rand1
ran2=rand2

    
fig=plt.figure(figsize=(12,21))
#img=cv2.imread('Trayectorias_densas/25_5_4/frame30.jpg')
for i in range(8):
    fig.add_subplot(int(str(52)+str(i+1)))    
    n1=rand2
    #n1=numero del frame
    img=cv2.imread('Trayectorias_densas/25_5_4/frame'+str(n1)+'.jpg')
    if(len(covs[n1])-1==0):
        #n2=numero de la trayectoria
        n2=0
    else:
        n2=rand1
    y=int(matrix[n2][n1+33])
    x=int(matrix[n2][(n1)+34])
    plt.title("frame "+str(n1+17)+" trayectoria "+str(n2)+" ("+str(x)+","+str(y)+")")
    cv2.circle(img,(np.float32(x),np.float32(y)),1,(0,255,0),-1)
    cv2.rectangle(img,(np.float32(x-20),np.float32(y+20)),(np.float32(x+20),np.float32(y-20)),(0,0,255),2)

    plt.imshow(normalized_draw((covs[n1+1][n2]),max_matrix,min_matrix),cmap="gist_rainbow")
    plt.axis("Off")
    plt.box('Off')
    plt.colorbar(ticks=[1,0.8,0.6,0.4,0.2,0,-0.2,-0.4,-0.6,-0.8,-1])
    rand2=rand2+2
    #img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB);

IndexError: index 917 is out of bounds for axis 1 with size 120

# Covarianza Volumetrica(Covarianza por trayectoria)

### Pasos
- Obtener los tensores del video
- A partir de los tensores y las trayectorias manipular la covarianza
- Obteniendo una lista de covarianzas por trayectorias(para este ejemplo 944 covarianzas)

In [22]:
def video_trayectory_covs(path,trayectorias,area):
    """Esta funcion es la que es, es la main de las main, las demas valen verga"""
    #tr_list = mi lista de trayectorias 
    #En el tensor P obtengo listas de d*w*h
    #En el tensor Q obtengo listas de n*w*h, donde n es el numero de elementos de la diagonal superior
    #tr_matrix es una matriz de trayectorias con dimension [nro de trayectorias][nro de frames*2] (x, y)
    #list_cov es una lista con todas mis covarianzas, donde tiene [nro del frame-15][nro de covarianzas por frame]
    #Obtengo como salidas, una lista de listas con todas mis covarianzas, y una matriz de trayectorias, de donde las puedo extraer
    #Para ejecutar esta funcion debo tener antes, mis caracteristicas integrales int_great_array
    
    flujo,f,great_p,great_q,Q=[],[],[],[],[]
    images = [path+image for image in os.listdir(path)]
    n_images = len(images)

    if isinstance(sqrt(n_images)%1,float):limits = int(sqrt(n_images))+1 
    else:limits = int(sqrt(n_images))  
    for i,image in enumerate(images):
        im = Image.open(image)
        a='Trayectorias_densas/FramesP/frame'+str(i+1)+'.jpg'
        if(i<10):
            zeros='0000000'
            b='Trayectorias_densas/FramesF/'+zeros+str(i+1)+'.ppm'
            flujo.append(b)
        else:
            zeros='000000'
            b='Trayectorias_densas/FramesF/'+zeros+str(i+1)+'.ppm'
            flujo.append(b)
            
        f.append(a)    
        try: 
            #print(cv2.imread(flujo[i]).shape)
            x,p,q=int_great_array(cv2.imread(f[i]),cv2.imread(flujo[i]))
        except:
           #xf=cv2.imread(flujo[i-1])
            x,p,q=int_great_array(cv2.imread(f[i]),cv2.imread(flujo[i-1]))
                                  
        great_p.append(p)
        great_q.append(q)
    #Obtengo el tensor P y Q
    
    
    d=great_p[0].shape[0]
    tr_list=cortar(trayectorias)
    #nro numero de trayectorias del video
    nro=len(tr_list)
    #nro2 numero de frames del video*2
    nro2=n_images*2
    tr_matrix=np.zeros((nro,nro2))
    x,y=tr_matrix.shape # x = numero de trayectorias ; y = numero de frames * 2
    for i in range(0,x):
        k=2
        for j in range(int((tr_list[i][0]*2)),int(tr_list[i][0]*2)+32):
            tr_matrix[i][j]=tr_list[i][k]
            k=k+1 
    #Obtengo mi matriz de trayectorias segun el caso
    #El 32 debido a que empieza en el frame 16 y termina 16 frames antes del final.
    list_cov=[]
    for i in range(0,x):
        #Iterar sobre las trayectorias
        for j in range(0,y,2):
            #Se itera sobre los frames (es de dos en dos por tr_matrix
            lista_covarianzas_x_trayectoria = []
            if (tr_matrix[i][j] != 0):
                for k in range(0,32,2):
                    temp=int((j+k)/2)
                    y1=int(tr_matrix[i][j+k]+area)
                    y2=int(tr_matrix[i][j+k]-area)
                    x1=int(tr_matrix[i][j+k+1]+area)
                    x2=int(tr_matrix[i][j+k+1]-area)
                    Q=great_q[temp]
                    P=great_p[temp]
                    cov=matriz_covarianza(P,Q,y1,x1,y2,x2)
                    lista_covarianzas_x_trayectoria.append(cov)
                list_cov.append(lista_covarianzas_x_trayectoria)
                break 
    #Obtengo list_cov
    
    return list_cov,tr_matrix,great_p,great_q

In [25]:
covs,matrix,great_p,great_q=video_trayectory_covs(path,trayectorias,area)

AttributeError: 'module' object has no attribute 'COLOR_BGR2YCrCb'

In [23]:
print(len(covs))
print(matrix.shape)
print(len(great_p))
print(len(great_q))
print(great_p[0].shape)
print(len(covs))

944
(944, 120)
60
60
(7, 273, 437)
944


In [29]:
medias= []
for cov in covs:
    media=mean(cov,0.1)
    medias.append(media)

NameError: name 'mean' is not defined

In [25]:
len(covs)

944

In [26]:
media = mean(covs[10],0.1)

NameError: name 'mean' is not defined

In [27]:
medias=[]
for i in range(len(covs)):
    media=mean(covs[i],0.1)
    medias.append(media)
    if(i%50==0):
        print(i)
    

NameError: name 'mean' is not defined

In [31]:
medias[1]

IndexError: list index out of range

In [32]:
def exp_AB(A,B):    #A y B tipo arrays
    
    B_raiz_log=sg.logm(B)*(1/2)
    B_raiz_exp=sg.expm(B_raiz_log)
    
    B_inverso=sg.inv(B_raiz_exp)
    
    c=np.dot(np.dot(B_inverso,A),B_inverso)
    A_exp=sg.expm(c)
    
    exp_B=np.dot(np.dot(B_raiz_exp,A_exp),B_raiz_exp)
    
    return exp_B
def media_error(x_media):    
    s=sg.eigh(x_media,check_finite=False,eigvals_only=True)
    for i in range(len(s)):
        if(s[i]<=0):
            s[i]=1
    x_abs=np.sum(np.log(s**2))
    return x_abs
def suma_log(P,m):
    suma=0
    for i in P:
        suma=suma+log_AB(i,m)
    suma=suma/len(P)
    return suma
def log_AB(A,B):       #A y B tipo arrays
   
    B_raiz_log=sg.logm(B)*(1/2)
    B_raiz_exp=sg.expm(B_raiz_log)
    
    B_inverso=sg.inv(B_raiz_exp)
    
    c=np.dot(np.dot(B_inverso,A),B_inverso)
    A_log=sg.logm(c)
    
    log_B=np.dot(np.dot(B_raiz_exp,A_log),B_raiz_exp)
    
    return log_B

In [33]:
def mean(P,tol): 
    
    m0=np.identity(P[0].shape[0])
    x0=suma_log(P,m0)
    m1=exp_AB(x0,m0)
    x_abs0=media_error(x0)
    i=0
    #print(x_abs0,"  ",i)
    while(x_abs0>tol and i<100):        
        x1=suma_log(P,m1)
        m2=exp_AB(x1,m1)
        x_abs0=media_error(x1)
        m1=m2
        i=i+1
        #print(x_abs0,"  ",i)
    return log_AB(m1,np.identity(P[0].shape[0]))

### Trayectory covariance
- Entradas Super tensores, posiciones de la trayectoria
<br>`Ignorar`

In [30]:
def trayectory_covariance_elements(P,Q,y1,x1,y2,x2):
    #El tensor P
    #El tensor Q
    #Las coordenadas de los puntos para las regiones de interes
    d=P.shape[0]
    col=x2-x1
    fil=y2-y1
    m_cov=np.zeros((d,d))
    n1=(1/((fil*col)-1))
    n2=(1/(fil*col))
    pos=0
    ten_matrix=np.zeros((d,d,3))
    for i in range(d):
        for j in range(i,d):
            q=Q[pos][y1,x1]+Q[pos][y1+fil,x1+col]-Q[pos][y1,x1+col]-Q[pos][y1+fil,x1]
            p_1=(P[i][y1,x1]+P[i][y1+fil,x1+col]-P[i][y1,x1+col]-P[i][fil+y1,x1])/(fil*col)
            p_2=(P[j][y1,x1]+P[j][y1+fil,x1+col]-P[j][y1,x1+col]-P[j][fil+y1,x1])/(fil*col)
            ten_matrix[0,i,j]=q
            ten_matrix[1,i,j]=p_1
            ten_matrix[2,i,j]=p_2
            pos=pos+1
            if(i!=j):
                ten_matrix[0,j,i]=ten_matrix[0,i,j]
                ten_matrix[1,j,i]=ten_matrix[1,i,j]
                ten_matrix[2,j,i]=ten_matrix[2,i,j]
    return ten_matrix

In [31]:
import numpy as np
temp_list=[]
temp_matrix=np.zeros((3,3,3))
temp_matrix[0,0,0]=1
temp_matrix[1,0,0]=2
temp_matrix[2,0,0]=3
temp_matrix[0,0,1]=4
temp_matrix[0,0,2]=5
temp_list.append(temp_matrix)
temp_matrix2=np.zeros((3,3,3))
temp_matrix2[0,0,0]=1
temp_matrix2[1,0,0]=2
temp_matrix2[2,0,0]=3
temp_list.append(temp_matrix2)

In [32]:
#temp_matrix=np.zeros((3,3,3))
temp_list[0][0]

array([[1., 4., 5.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [33]:
temp_list[0].shape[0]

3

In [34]:
#n1=ran2
#n2=ran1
for i in range(8):
fig.add_subplot(int(str(52)+str(i+1)))    
n1=ran2
#n1=numero del frame
img=cv2.imread('Trayectorias_densas/25_5_4/frame'+str(n1)+'.jpg')
if(len(covs[n1])-1==0):
    #n2=numero de la trayectoria
    n2=0
else:
    n2=ran1
y=int(matrix[n2][n1+33])
x=int(matrix[n2][(n1)+34])
plt.title("frame "+str(n1+17)+" trayectoria "+str(n2)+" ("+str(x)+","+str(y)+")")
cv2.circle(img,(np.float32(x),np.float32(y)),1,(0,255,0),-1)
cv2.rectangle(img,(np.float32(x-20),np.float32(y+20)),(np.float32(x+20),np.float32(y-20)),(0,0,255),2)

plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB);
#plt.imshow(normalized_draw((covs[n1+1][n2]),max_matrix,min_matrix),cmap="gist_rainbow")
#plt.axis("Off")
#plt.box('Off')
#plt.colorbar(ticks=[1,0.8,0.6,0.4,0.2,0,-0.2,-0.4,-0.6,-0.8,-1])
ran2=ran2+2
#img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB);

IndentationError: expected an indented block (<ipython-input-34-dbce5eafb0fb>, line 4)

In [ ]:
vidcap = cv2.VideoCapture('Datasets/datasets/ut/segmented_set1/'+list_elements2[17][:-4]+'.avi')
success,image = vidcap.read()
count = 1
success = True
while success:
        filename = "FramesTemp/frame%d.jpg" % count
        cv2.imwrite(filename, image)     # save frame as JPEG file     
        x,y,z=cv2.imread("FramesTemp/frame1.jpg").shape
        conv1_im = tf.contrib.keras.preprocessing.image.load_img(filename, target_size=(224, 224))
        conv1_im = tf.contrib.keras.preprocessing.image.img_to_array(conv1_im)
        conv1_im = np.expand_dims(conv1_im, axis=0);
        features = model_ResNet50.predict(conv1_im)
        features_conv1 = model_resNet50_conv1.predict(conv1_im)
        convimg=[]
        for i in range(15):
            convimg.append(cv2.resize(features_conv1[0,:,:,perm[i]], (y, x),interpolation = cv2.INTER_CUBIC))
        #Guardo las 20 caracteristicas de cada frame
        np.save('binaryfe/'+ list_elements1[17].split("/")[-1][:-4]+"/"+filename.split("/")[-1][:-4],convimg)    
        success,image = vidcap.read()
        #print('Read a new frame: ', success)
        count += 1